# **Cryptography Research Project**
You will spend the week studying cryptography and cryptanalysis: the means and processes used to create, translate, decrypt, and attack portions of plain or encrypted text. In this project, you will learn and employ a number of cryptologic techniques developed over the past few thousand years using the Python programming language. Both symmetric and asymmetric key cryptosystems will be studied. Your research project leader will assign one cryptosystem at a time in chronological order of development, and you will complete some or all of the following tasks: 

1. Choose a key for the specified cryptosystem. Write a script which encrypts a message from a .txt file and writes it to another .txt file with correct cryptographic formatting.
2. Given a specific cipher configuration, write a Python script which decrypts an encoded message sent by someone else on your research team. 
3. Determine the order of the key space associated to the given encryption technique. 
4. Estimate the amount of time required to complete a brute-force attack on your own cipher by hand. 
5. Write a Python script which attacks and successfully converts an enciphered message from the project staff to plain text without knowledge of the key.

You will develop a basic understanding of elementary number theory and abstract algebra.


---


## **Project Format**
First, we'll complete a brief introduction to cryptography via introduction of the Vernam cipher. Following that, the research project is formatted in the following manner. For each cipher technique we'll cover, this document will guide you through the following:

1. Description of cryptosystem
2. Mathematics of cryptosystem
3. Encryption of chosen message
4. Decryption of chosen message
5. Discussion and computation of key space size
6. Attack encrypted text

We will be working through some (or all) of the following cryptosystems:

1. Vernam Cipher (in use since 1918 AD)
2. Shift Cipher (in use c. 50 BC)
3. Affine Cipher (unknown usage)
4. Monoalphabetic Substitution Cipher (since antiquity, broken c. 800 AD)
5. Polyalphabetic Substitution Cipher (special case in use since 1553 AD, last military use 1945 - Enigma machine)
6. Diffie-Hellman Key Exchange (classified use since 1969, published widely 1976)
7. Rivest–Shamir–Adleman (RSA) Cryptosystem (published in 1976, still in use today)

---

# **Introduction to Cryptography**
What is cryptography? Cryptography is the practice and study of secure communications. In other words, the goal of cryptography is to enable two parties to communicate with each other while mitigating the risk that a third party, called an **adversary**, can successfully read the message. While **physical security** concerns itself with ensuring that an adversary will not even have the chance to intercept a message, a cryptographer assumes *a priori* that the adversary will be able to intercept the message.

Thus the central focus of cryptography is making a message *uninterpretable* to any third party.

Let us now state some defintions to lay the groundwork for the rest of this week. We will assume that all messages this week consist *only* of English letters and English words (i.e. no numbers, no non-English characters or words).

> Definition 1. $\quad$ **Plaintext** is a sequence of readable text. Mathematically, we denote a string of plaintext by $p$. It is convention to write plaintext in $\tt{lowercase \,\, letters}$. 

In [1]:
# In Python, to initialize a plaintext message such as "hello world" as a 
# string, we would write something like
plaintext = 'hello world'

# Running this bit of code will return the plaintext message.
print(plaintext)

hello world



> Definition 2. $\quad$ **Ciphertext** is a sequence of unreadable text that correspond to some string of plaintext. Mathematically, we denote a string of ciphertext by $c$. It is convention to write ciphertext in $\tt{UPPERCASE\,\, LETTERS}$.

> Definition 3. $\quad$ **Encryption** is the process of converting plaintext to ciphertext.

> Definition 4. $\quad$ **Decryption** is the process of converting ciphertext to plaintext.

> Definition 5. $\quad$ A **cryptographic key**, or just **key**, is a sequence of alphanumeric characters which respectively enables encryption and/or decryption of plaintext or ciphertext. Mathematically, we represent a cryptographic key by $k$. 

> Definition 6. $\quad$ An **encryption function** is an algorithm which maps plaintext to ciphertext. Likewise, a **decryption function** is an algorithm which maps ciphertext to plaintext. We typically denote an encryption function by $e$ and a decryption function by $d$.

Evaluating the plaintext $p$ under the encryption function yields the ciphertext $c$. Likewise, evaluating the ciphertext $c$ under the decryption function yields the plaintext. In mathematical language,

$$\begin{align*}
e(p) = c \qquad \textrm{and} \qquad d(c) = p
\end{align*}$$

It follows from these properties that

$$\begin{align*}
d(e(p)) = p \qquad \textrm{and} \qquad e(d(c)) = c
\end{align*}$$

This means that $d$ and $e$ are each other's **inverse functions**.

*Note: encryption and decryption functions typically accept both plaintext/ciphertext and a cryptographic key as arguments. That is, they are functions of two variables. In plain English, this means that - for example - if you pass a string of plaintext $p_1$ to an encryption function with a given key $k_1$, then the resulting ciphertext will look different than if you had passed a different string of plaintext $p_2$ through the encryption function with a different key $k_2$. However, this is not always the case, especially for simple cryptosystems.*

*A simple example to exhibit this behavior: define a function $f$ by the rule $f(p,k) = p+3k$, where $p$ and $k$ are integers. Try varying $p$ and $k$ to see how the output of $f$ changes.*

In [2]:
# Define a function f with arguments p and k such that f(p,k) = p + 3k.
def f(p,k):
  return p + 3*k

# You can change the values of p and k to experiment with the outputs of f.
#
# Can you find two different sets of values for p and k which give you the same
# output f(p,k)?

# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

p = 1     
k = 1

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE

print(f(p,k))

4


> Definition 7. $\quad$ A **cryptosystem** is a method for encrypting plaintext and decrypting ciphertext. A cryptosystem is defined by the form of its cryptographic key and its encryption/decryption functions.

> Definition 8. $\quad$ A **cipher** is a particular configuration of a cryptosystem. In most cases - and in every case this week - this just means that we've chosen a specific cryptographic key to use with a particular cryptosystem.

You may note that we often use the term "cipher" when we should technically be using "cryptosystem." The definitions are what they are, but it's ok to swap these words when we aren't trying to be extremely specific.

Given a particular cryptosystem, the collection of all possible...
> plaintexts is denoted $\mathcal{M}$.

> ciphertexts is denoted $\mathcal{C}$.

> keys is denoted $\mathcal{K}$.

> encryption functions is denoted $\mathcal{E}$.

> decryption functions is denoted $\mathcal{D}$.



---
$$$$
## Modular Arithmetic

Now we lay the mathematical foundation for cryptography: modular arithmetic. We'll begin with a definition.

> Definition 9. $\quad$ Let $a$ and $b$ be integers. We say that $a$ **divides** $b$ if there exists an integer $d$ such that $a\cdot d = b$, and we write $a \mid b$ to mean "$a$ divides $b$." If instead $a$ does not divide $b$, we would write "$a \nmid b$." 

*Example* $\quad$ Let $a = 3$ and $b = 27$. Then $a \mid b$ since $d = 9$ is an integer satisfying
$$$$

$$a \cdot d = b \qquad \textrm{i.e.} \qquad 3\cdot 9 = 27$$

$$$$

*Example* $\quad$ Let $a = 2$ and $b = 27$. Then $a \nmid b$ since the only number $d$ satisfying $2d = 27$ is
$$$$
$$d = 13.5$$

... which is not an integer.$$$$

> Definition 10. $\quad$ Let $a$ and $b$ be integers, and let $n$ be an integer greater than 1. We say that $a$ **is congruent to** $b$ **modulo** $n$ if $n$ divides $(a-b)$. Alternatively, if the latter holds, we write $a \equiv b \pmod n$. The number $n$ is called the **modulus** of the congruence.

Let's unpack this a bit. The concept of divisibility was invoked in the definition of integer congruence, so let's substitute the definition of the former into the latter. 

$$\begin{align*} a \equiv b \pmod n \qquad &\Longleftrightarrow \qquad n \,\,\textrm{divides}\,\, (a - b) \\\\
&\Longleftrightarrow \qquad \textrm{there exists an integer $d$ such that}\,\,(a-b)= n \cdot d 
\end{align*}
$$

Let's work through some examples.$$$$

*Example.* $\quad$ Suppose that $a = 25$, $b = 97$, and $n = 9$. Is it true that $a \equiv b \pmod n$?

We'll apply the definition of congruence. We want to know if it is true that $n$ divides $(a - b)$. In other words, we want to know if there exists an integer $d$ such that $a-b = n\cdot d$. We have

$$ \begin{align*}
&&a - b &= n\cdot d \qquad \Longrightarrow \qquad 25 - 97 = 9d \qquad \Longrightarrow \qquad -72 = 9d \\\\
&\Longrightarrow & d &= -\frac{72}{9} = -8
\end{align*}$$

Since $d = -8$ is an integer, we conclude that 25 **is** congruent to 97 modulo 9.$$$$

*Example.* $\quad$ Let now $a = 5$, $b = -16$, and $n = 10$. Is it true that $a \equiv b \pmod n$?

Again we apply the definition of congruence. We want to know if it is true that $n$ divides $(a-b)$. In other words, we want to know if there exists an integer $d$ such that $a-b = n\cdot d$. We have

$$\begin{align*}
&&a - b &= n\cdot d \qquad \Longrightarrow \qquad 5 - (-16) = 10d \qquad \Longrightarrow \qquad 21 = 10d \\\\
&\Longrightarrow & d &= \frac{21}{10} = 2.1
\end{align*}$$

Since $d = 2.1$ is not an integer, we conclude that $5$ **is not** congruent to $-16$ modulo 10.

Interestingly, addition, subtraction, and even multiplication work just fine when we replace an equality symbol with a congruence modulo $n$.

$$$$

*Example.* $\quad$ Notice that $35 + 28 = 63$. Also observe that

$$\begin{align*}
35 \equiv 9 \pmod{26} \qquad \textrm{and}\qquad   28 \equiv 2 \pmod{26}
\end{align*}$$

Now, $9 + 2 = 11$. So we'd expect that $63 \equiv 11 \pmod{26}$ for everything to work out as claimed. Indeed, 63 is congruent to 11 modulo 26 (verify this yourself)! This supports our claim that congruence modulo $n$ preserves addition. The same is true of subtraction and multiplication. Division, however, is not as well-behaved; there is a subtlety here that we will return to later.

This very nice behavior is the motivation behind the following theorem, which guarantees that addition and subtraction behave nicely.

*Theorem 1.* $\quad$ Let $a, b, c$, and $d$ be integers, and let $n$ be an integer greater than 1. If $a \equiv c \pmod n$ and $b \equiv d \pmod n$, then

$$a + b \equiv c + d \pmod n$$

*Proof.* $\quad$ Since $a \equiv c \pmod n$, there exists an integer $d_1$ such that $a-c = n\cdot d_1$ by the definition of congruence. Likewise, since $b \equiv d \pmod n$, there exists an integer $d_2$ such that $b - d = n \cdot d_2$. Taking the sums of the left and right-hand sides of these equations yields

$$\begin{align*}
&& (a-c) + (b-d) &= n\cdot d_1 + n \cdot d_2 \\\\
&\Longrightarrow & a + b - c - d &= n \cdot (d_1 + d_2) \\\\
&\Longrightarrow & (a+b) - (c + d) &= n \cdot (d_1 + d_2)
\end{align*}$$

Since $d_1$ and $d_2$ are both integers, their sum is also an integer. Then there exists an integer $d$ (namely $d = d_1 + d_2$) such that $(a+b) - (c+d) = n\cdot d$. That is, $n$ divides $(a+b) - (c+d)$. Therefore $ a+b \equiv c+d \pmod n$. $\quad \blacksquare$

The most important consequence of this theorem is that, for example, in evaluating the expression $25 + 891 \pmod{20}$, it does not matter if you first take the sum of 25 and 891 and then reduce it modulo 20, or first reduce 25 and 891 modulo 20 and then take their sum.

$$\begin{align*} 25 + 891 &\equiv 5 + 11 = 16 \pmod{20} \\\\
25 + 891 &= 916 \equiv 16 \pmod{20} \end{align*}
$$

*Note: Sometimes you'll need to reduce the numbers via their given modulus more than once to see that this works. In this case, we didn't have to do that.*$$$$

Python can be used to perform modular arithmetic for you. 

In [3]:
# We can also do modular arithmetic in Python.
# The syntax is as follows.
# To add two integers a and b modulo n in Python, we write:

# (a + b) % n

# Try adding 1053 and 14993 modulo 26.
# You should get 4.

# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

(1053 + 14993) % 26

4

Whenever you ask Python to compute an expression modulo an integer $n$, it will return a number between 0 and $n-1$. For the rest of the week, when you are required to perform modular arithmetic, you should always express your answer like this. This motivates the following definition:$$$$

*Definition 11.* $\quad$ Let $a$ be an integer and $n$ be an integer greater than 1. Suppose that $r$ is an integer satisfying $0 \leq r < n$ and $a \equiv r \pmod n$. Then $r$ is called the **least residue class of** $a$ **modulo** $n$.$$$$

---
$$$$
Finally, to each letter of the alphabet we associate an integer between 0 and 25, inclusive, as follows:$$$$

A | B | C | D | E | F | G | H | I | J | K | L | M | N | O | P | Q | R | S | T | U | V | W | X | Y | Z
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---
0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 | 25
$$$$

The purpose of doing this is to convert a string of text to a sequence of numerical values; this makes them ready for input into an encryption/decryption function. $$$$

*Example.* $\quad$ Convert the plaintext $\tt{cryptography}$ to a sequence of integers.

Referencing the table above gives

$$2, 17, 24, 15, 19, 14, 6, 17, 0, 15, 7, 24$$
$$$$

---
$$$$
## The Vernam Cipher
The Vernam cipher, invented by Gilbert Vernam in 1918 while employed by the American Telegraph and Telephone Company (AT&T), was the first cryptosystem featuring what is now called Shannon [perfect secrecy](https://www.cs.miami.edu/home/burt/learning/Csc609.011/Perfect/) after mathematician Claude Shannon (b. 1916 - d. 2001). Perfect secrecy is exactly what it sounds like: it is perfectly secret. However, this secrecy comes at a cost.

The security of *any* cryptosystem relies on the key being kept, well, secret. Therein lies the difficulty: secure key exchange. If keys cannot be exchanged securely under the watchful eye of an adversary, then secure communication is impossible. (Or is it? We'll return to this later.)

Nevertheless, under the hypotheses that key exchange can be executed securely, the key is chosen randomly, and keys are never reused, the Vernam cipher is **impervious to attack**. Because keys used with the Vernam cipher must be kept secret and discarded after one use, they are sometimes referred to as [**one-time pads**](https://en.wikipedia.org/wiki/One-time_pad). Let's examine this cryptosystem in detail.

*The Vernam Cipher.* $\quad$ Suppose Alice wishes to transmit a plaintext message $p$ of alphabetic characters with length N to Bob. To encrypt, send, and decrypt a message with the Vernam cipher, they complete the following steps.

> 1. Alice converts $p$ to a list of integers modulo 26.
> 2. Alice chooses a key $k$ consisting of randomly-chosen integers modulo 26 and delivers it to Bob using physical security measures.
> 3. Alice uses the encryption function $e_k(p_i) \equiv p_i + k_i \pmod{26}$ to encrypt $p$, yielding the ciphertext $c$.
> 4. Alice transmits $c$ to Bob in the clear.
> 5. Bob uses the key $k$ and the decryption function $d_k(c_i) \equiv c_i - k_i \pmod{26}$ to resolve the plaintext $p$ from $c$.
> 6. Alice and Bob destroy the key they used to encrypt/decrypt the message.

$$$$

*Example.* $\quad$ Suppose our plaintext $p$ is $\tt{attack \,\, at \,\, dawn}$. Step 1 tells us to convert $p$ to a list of integers modulo 26. The corresponding numerical plaintext $p$ is

$$p = [0, 19, 19, 0, 2, 10, 0, 19, 3, 0, 22, 13]$$

It will be useful to have a function handy which will accept a string of plaintext and produce the corresponding list of numbers.

In the cell below, we have a function called **plain_to_num** which accepts a string of plaintext and converts it to a list of numbers. This function ignores punctuation, spacing, and capitalization.

$$\textrm{plain\_to\_num(alphabetic plaintext)} \qquad \longrightarrow \qquad \textrm{numerical plaintext}$$

Test it on the plaintext $\tt{attack \,\, at \,\, dawn}$ to confirm that you obtain the result above.

In [63]:
# Define the function plain_to_num().
def plain_to_num(plaintext):

  # Initialize a list which stores the letters of the alphabetic (in order)
  #     in lowercase form.
  plainAlphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                   'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                   'y', 'z']

  # Initialize an empty list to store our plaintext in numerical form.
  plaintextNum = []  

  # Iterate over the plaintext, ignoring characters that are not letters, and
  # converting those that are to numbers. Store them in in plaintextNum.
  for i in range(len(plaintext)):

    # Call the .lower() function to ensure letters in 'plaintext' are evaluated 
    # as lowercase when compared to letters in plainAlphabet. If the object 
    # is a letter, allow the next line of code to be executed.
    if plaintext[i].lower() in plainAlphabet: 

      # For each letter in plaintext, append the corresponding number to 
      # plaintextNum by searching plainAlphabet for that letter and 
      # returning its index, which conveniently happens to be the number it 
      # corresponds to in cryptography.
      plaintextNum.append(plainAlphabet.index(plaintext[i].lower()))
  
  return plaintextNum

# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

plaintext = 'attack at dawn'

print(plain_to_num(plaintext))

[20, 21, 11, 11, 12, 19, 16, 5, 5, 8, 10, 5, 15, 24, 10, 16, 22, 22, 20, 14, 6, 8, 24, 16, 9, 12, 3, 6, 3, 14]


Let $p_i$ denote the $i^{\textrm{th}}$ element of the plaintext and $k_i$ denote the $i^{\textrm{th}}$ element of the key. Then $p_1 = 0$, $p_2 = 19$, $p_3 = 19$, ..., and $p_{12} = 13$. We do not worry about spaces between words. Since the length of the plaintext message is $N = 12$, our key must have length *at least* 12. For reasons which will become clear later, we want to round the length of the key up to the nearest multiple of 5. So we take the length of the key to be 15. For cryptographic security, the elements of the key must be chosen randomly. There is a Python package called [random](https://docs.python.org/3/library/random.html) which is suitable for this purpose.

Here is some example code to produce a random key of length 15.

In [5]:
# Import the Python package RANDOM.
import random

N = 15     # Set the length of the plaintext message.
key = []     # Initialize an empty list to serve as our key.

# Iterate over the length of the plaintext message, appending
# a random integer between 0 and 25 at each step.
#
# The random.randint(a,b) function gives us a random integer
# x satisfying a <= x <= b. Here we choose a = 0 and b = 25.
#
# This is equivalent to choosing a random integer modulo 26.
for i in range(N):                
  key.append(random.randint(0,25))  

# Print the key.
print(key)

[6, 25, 0, 17, 21, 3, 13, 0, 4, 23, 0, 8, 16, 16, 21]


We'll use the following key, which was generated using the code above:

$$k = [17, 5, 10, 16, 20, 13, 11, 18, 9, 22, 6, 16, 3, 17, 24]$$
$$$$

As stated before, the encryption function is $e_k(p_i) \equiv p_i + k_i \pmod{26}$. This just means that we sum each element of the plaintext with the corresponding element of the key modulo 26 to produce the corresponding element of the ciphertext. For example, if we want to know what the 3rd element $c_3$ of the ciphertext is, we take the third element of the plaintext (which is $p_3 = 19$) and the third element of the key (which is $k_3 = 10$) and add them together modulo 26.

$$ c_3 = e_k(p_3) = p_3 + k_3 = 19 + 10 = 29 \equiv 3 \pmod{26}$$

Since 3 corresponds to the letter $\tt{D}$, the third letter of the ciphertext is $\tt{D}$. 

Continuing on in this fashion, Alice obtains the numerical ciphertext $c = [17, 24, 3, 16, 22, 23, 11, 11, 12, 22, 2, 3]$. We could either transmit this to Bob as a sequence of numbers or as a sequence of letters - it makes no difference. We'll convert it back to letters.

$$\tt{RYDQW\,\,XLLMW\,\,CD}
$$

It is common practice in cryptography to transmit ciphertexts in groups of five letters. It is also common practice to add random trailing letters to the final block if it does not have a full five letters so that an adversary can't know for certain where the message terminates. There is a good reason for doing this which we will discuss later. For now, let's randomly select some additional letters for the end of the message. The ciphertext becomes:

$$\tt{RYDQW\,\,XLLMW\,\,CDPVU}$$

It will be useful to have a function which accepts a list of numerical ciphertext and outputs a string of properly formatted alphabetic ciphertext. In other words, we want this function to accomplish three things:

> 1. Convert a list of numerical ciphertext to alphabetic ciphertext.
> 2. If the length of the ciphertext is not a multiple of 5, randomly insert as many additional letters at the end as necessary to ensure it is. This number will be somewhere between 0 and 4 additional characters.
> 3. Place the text into 5-block form, i.e. insert spaces where appropriate.

To this end, below is a function called **num_to_cipher()** which satisfies the preceding conditions.

$$\textrm{num\_to\_cipher(numerical ciphertext)} \qquad \longrightarrow \qquad \textrm{alphabetic ciphertext}$$

Test it on the numerical ciphertext $c = [17, 24, 3, 16, 22, 23, 11, 11, 12, 22, 2, 3]$ to confirm that you obtain the result above.

In [6]:
# Define the function num_to_cipher().
def num_to_cipher(ciphertextNum):

  # This function will need the Python package RANDOM to append padding.
  import random

  # Initialize a list to store uppercase letters.
  cipherAlphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
                   'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
                   'Y', 'Z']

  # This block of code checks to see if the ciphertext will require padding.
  # If its length modulo 5 is 0, then it does not.
  if ((len(ciphertextNum) % 5) is not 0):

    # The length mod 5 is not 0. Store the number of additional characters
    # required in numLetters.

    numLetters = 5 - (len(ciphertextNum) % 5)
    # Iterate over the number of additional letters required, randomly appending
    # one on each iteration.

    for i in range(numLetters):
      ciphertextNum.append(random.randint(0,25))

  # Initialize an empty list to store our ciphertext.
  ciphertext = [] 

  # Iterate over the length of ciphertextNum.
  #
  # Check to see if the index we are at corresponds to the end of a 'block'
  # of ciphertext. If so, add a space ' '.
  #
  # Then append an element of the ciphertext by referencing cipherAlphabet.
  #
  # The numbers in ciphertextNum correspond to the index of the associated letter.
  for i in range(len(ciphertextNum)):
    if ((i % 5) == 0) and (i > 1):
      ciphertext.append(' ')
    ciphertext.append(cipherAlphabet[ciphertextNum[i]])

  # Finally, we'll concatenate the alphabetic ciphertext using the JOIN function  
  # so that the output of num_to_cipher() is a single string of ciphertext 
  # letters.
  #
  # The syntax of the JOIN function is:

  # [String appearing between each element of ListOfStrings].join([ListOfStrings])
  
  ciphertext = ''.join(ciphertext)
  
  # num_to_cipher() returns the ciphertext.
  return ciphertext

#-------------------------------------------------------------------------------

# Test that the function works correctly.
ciphertextNum = [17,24,3,16,22,23,11,11,12,22,2,3]

print(num_to_cipher(ciphertextNum))

RYDQW XLLMW CDIMI


We are now in a position to write an encryption function for the Vernam cipher.

In [7]:
# Define a function vernam_encrypt() to encrypt a plaintext message
#     using the Vernam cipher with a given key.
def vernam_encrypt(plaintext,key):  

  plaintextNum = plain_to_num(plaintext)

  ciphertextNum = []

  # Iterate over the length of the plaintext, computing an element of
  # the numerical ciphertext at each step, and appending it to ciphertextNum.
  for i in range(len(plaintextNum)):
    ciphertextNum.append((plaintextNum[i]+key[i]) % 26)

  ciphertext = num_to_cipher(ciphertextNum)

  # The function vernam_encrypt() returns the ciphertext.
  return ciphertext


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# We call our plaintext p.
plaintext = 'attack at dawn'

# We call our key k.
key = [17, 5, 10, 16, 20, 13, 11, 18, 9, 22, 6, 16, 3, 17, 24]  

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Use the encryption function to encrypt p with k.
vernam_encrypt(plaintext,key)

'RYDQW XLLMW CDVUC'

After transmitting this ciphertext to Bob, he will convert it back to a sequence of integers modulo 26 (which is precisely what we had before). Bob will now use the decryption function and the key to decrypt the ciphertext. The $i^{\textrm{th}}$ letter of the plaintext $p_i$ can be resolved using the decryption function as follows:

$$p_i \equiv d_k(c_i) \equiv p_i - k_i \pmod{26} $$

To automate this process using Python, we'll need two functions: **cipher_to_num()** to convert alphabetic ciphertext to numerical ciphertext, and **num_to_plain()** to convert numerical plaintext to alphabetic plaintext. 

We present code for these two functions below.

$$\textrm{cipher\_to\_num(alphabetic ciphertext)} \qquad \longrightarrow \qquad \textrm{numerical ciphertext}$$

$$\textrm{num\_to\_plain(numerical plaintext)} \qquad \longrightarrow \qquad \textrm{alphabetic plaintext}$$

In [8]:
# Below is a function which accepts alphabetic ciphertext and returns
# numerical ciphertext.

#-------------------------------------------------------------------------------

def cipher_to_num(ciphertext):

  # Initialize a list to store uppercase letters.
  cipherAlphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
                   'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
                   'Y', 'Z']

  # Initialize an empty list to store our ciphertext in numerical form.
  ciphertextNum = []  

  # Iterate over the ciphertext, ignoring characters that are not letters, and
  # converting those that are to numbers. Store them in in ciphertextNum.
  for i in range(len(ciphertext)):

    # Use the .upper() function to ensure letters in 'ciphertext' are evaluated 
    # as uppercase when compared to letters in cipherAlphabet. If the object 
    # is a letter, allow the next line of code to be executed.
    if ciphertext[i].upper() in cipherAlphabet: 

      # For each letter in ciphertext, append the corresponding number to 
      # ciphertextNum by searching cipherAlphabet for that letter and 
      # returning its index, which conveniently happens to be the number it 
      # corresponds to in cryptography.
      ciphertextNum.append(cipherAlphabet.index(ciphertext[i].upper()))
  
  return ciphertextNum

#-------------------------------------------------------------------------------

# Test cipher_to_num() on the given ciphertext.
ciphertext = 'RYDQW XLLMW CDPVU'

print(cipher_to_num(ciphertext))

[17, 24, 3, 16, 22, 23, 11, 11, 12, 22, 2, 3, 15, 21, 20]


In [9]:
# Below is a function which accepts numerical plaintext and returns 
# alphabetic plaintext in 5-block form.

#-------------------------------------------------------------------------------

def num_to_plain(plaintextNum):

  plainAlphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                   'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                   'y', 'z']

  # Initialize an empty string to store alphabetic plaintext.
  plaintext = []

  # Iterate over the length of plaintextNum.
  #
  # Check to see if the index we are at corresponds to the end of a 'block'
  # of plaintext. If so, add a space ' '.
  #
  # Then append an element of the plaintext by referencing plainAlphabet.
  # The numbers in plaintextNum correspond to the index of the associated letter.
  for i in range(len(plaintextNum)):
    if ((i % 5) == 0) and (i > 1):
      plaintext.append(' ')
    plaintext.append(plainAlphabet[plaintextNum[i]])
  
  # Finally, we'll concatenate the list 'plaintext' using the .join() function so 
  # that the output of num_to_plain() is a single string of plaintext 
  # letters.
  #
  # The syntax of the .join() function is:
  #
  # [String appearing between each element of ListOfStrings].join([ListOfStrings])
  #
  # In our case, we don't want anything to appear between the letters in the
  # plaintext, so we use an empty string ''.
  plaintext = ''.join(plaintext)

  # The function num_to_plain() returns the plaintext.
  return plaintext

#-------------------------------------------------------------------------------
# Test the function num_to_plain() using the numerical plaintext given above.
plaintextNum = [0,19,19,0,2,10,0,19,3,0,22,13]

print(num_to_plain(plaintextNum))

attac katda wn


We are now in a position to present a decryption function for the Vernam cipher.

In [10]:
# Define a function vernam_decrypt() to decrypt a ciphertext message
#     using the Vernam cipher with a given key.
def vernam_decrypt(ciphertext,key):  

  # Convert the alphabetic ciphertext to numerical ciphertext.
  ciphertextNum = cipher_to_num(ciphertext) 

  # Initialize an empty list to store our numerical plaintext.
  plaintextNum = []

  # Iterate over the length of the numerical ciphertext, computing an element of
  #     the plaintext at each step, and appending it to our plaintext list.
  for i in range(len(ciphertextNum)):
    plaintextNum.append((ciphertextNum[i]-key[i]) % 26)

  # Now we'll replace each entry of the plaintext, which is currently
  #     an integer modulo 26, with its corresponding letter.
  plaintext = num_to_plain(plaintextNum)

  # The function VERNAM_DECRYPT returns the plaintext.
  return plaintext


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Define a string to contain our ciphertext.
ciphertext = 'RYDQW XLLMW CDPVU'  

# Define our key.
key = [17, 5, 10, 16, 20, 13, 11, 18, 9, 22, 6, 16, 3, 17, 24]

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Use the decryption function to decrypt the ciphertext with the key.
vernam_decrypt(ciphertext,key)

'attac katda wnmew'

Our code returned the plaintext $\tt{attac\,\, katda\,\, wnmew}$ which, after discarding the padding and adjusting the spaces, reads $\tt{attack\,\, at \,\, dawn}$, the original message.

At this point, Bob would destroy his one-time pad containing the key, as Alice should have already done.$$$$

----


$$$$
## Exercise: The Lincolnshire Poacher
You have been assigned to a small CIA signals intelligence (SIGINT) unit tasked with using any means necessary to decrypt messages sent by the [numbers station](https://en.wikipedia.org/wiki/Numbers_station) nicknamed [The Lincolnshire Poacher](https://en.wikipedia.org/wiki/Lincolnshire_Poacher_(numbers_station)). A CIA field unit reported that they captured a spy who was in possession of a one-time pad containing Vernam cipher keys and instructions for use. Your unit suspects that the spy used the Lincolnshire Poacher to receive covert messages.

You are tasked with decrypting [this transmission](https://www.youtube.com/watch?v=QnXPqUU6fI0) using this [one-time pad](https://drive.google.com/file/d/1sRN_TLy2DrAuH9LSC2iWML_HqEDIaheP/view?usp=sharing). 

**Decrypt the covert message.**

*The [substantive portion of the transmission begins at 0:55](https://youtu.be/QnXPqUU6fI0?t=55). Be careful and follow the instructions closely. Take care that you write down the ciphertext correctly and use the correct decryption function. You should edit the code cell below to automate the decryption.*


In [11]:
# Use this space to complete a decryption script for The Lincolnshire Poacher.
# You will need to define two lists called "key" and "ciphertext" containing
# the key and ciphertext, respectively. 


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

key = [ 8, 13, 21, 16,  0, 
       22,  7, 12, 11,  1, 
       15, 10, 22,  7,  5, 
       21, 18, 13, 25,  6, 
        4,  1,  3, 21,  0, 
        8,  7, 14,  0, 25, 
       22, 17,  4, 23, 10, 
        5, 14, 25, 13, 16, 
       23, 25,  8, 14,  9, 
        5, 14,  3,  0,  3, 
       19,  2, 15, 20,  3]

ciphertext = [63, 69, 47, 71, 55, 
              13, 99, 27, 71, 45, 
              93, 29,  7, 21, 85, 
              73, 89, 47, 91, 45, 
              23, 49, 17, 41, 65, 
              63, 89, 57, 41, 25, 
              13, 79, 57,  1, 75, 
              33, 99, 37, 91, 85, 
              33, 29, 37, 41, 85, 
              53, 99, 37, 71, 85, 
              73,  9, 67, 31, 65]

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# We iterate over the length of ciphertext and map each integer to its least
# residue class.
for i in range(len(ciphertext)):
  ciphertext[i] = ciphertext[i] % 26

# Now that the numerical ciphertext is expressed in terms of least residue 
# classes, we are ready to convert it to alphabetic form using the function
# num_to_cipher().
ciphertext = num_to_cipher(ciphertext)

# Finally, we send the ciphertext and key through vernam_decrypt().
vernam_decrypt(ciphertext, key)

'deadd ropis atloc ation twoun derpa rkben chmar kedby white chalk'

---
$$$$

# **The Shift Cipher (Caesar's Cipher)**
The *shift cipher*, also known as *Caesar's cipher*, is one of the earliest recorded instances of the use of cryptography in securing communications. Encryption via the shift cipher is straightforward. The letters in the plaintext are shifted by a fixed amount to yield the ciphertext. This can be visualized using an encryption wheel (see below). The outer wheel corresponds to letters in the plaintext and the inner wheel corresponds to letters in the ciphertext. 
$$$$
![Caesar cipher encryption/decryption wheel](https://images-na.ssl-images-amazon.com/images/I/61K6UvP2XxL.png)
$$$$

*Note: This image has swapped our chosen convention for using uppercase letters for ciphertext and using lowercase letters for plaintext.*

To encrypt the letter $\tt{b}$, for example, one would find $\tt{b}$ on the outer wheel and record the letter adjacent to it on the inner wheel. In this case, the plaintext $\tt{b}$ would encrypt to the ciphertext letter $\tt{U}$.



## Mathematics of the Shift Cipher
The form of the key for the shift cipher is the least residue class of an integer modulo 26 (or equivalently, a letter of the alphabet). Let $k$ be the least residue class of an integer modulo 26; in other words, we take $k$ to be a single element of the set $\{0, 1, 2, 3, \ldots, 24, 25\}$. Then the encryption and decryption functions for the shift cipher are as follows:

$$ \begin{align*} e_k(p_i) &\equiv p_i + k \pmod{26} \\\\
d_k(c_i) &\equiv c_i - k \pmod{26} 
\end{align*}$$

where $p_i$ and $c_i$ are respectively the $i^{\textrm{th}}$ elements of the plaintext and ciphertext. Note that these are similar to the encryption and decryption functions for the Vernam cipher. The difference is that in the Vernam cipher, the key is a list of integers modulo 26, whereas the key for the shift cipher is a single integer modulo 26 which is applied uniformly to every element of the cipher/plaintext.

The decryption function can be derived from the encryption function by noting that $c_i = e_k(p_i)$ and $d_k(c_i) = p_i$. Substituting these into the first equation above yields

$$\begin{align*}
e_k(p_i) &\equiv p_i + k \pmod{26} \qquad \Longrightarrow \qquad c_i \equiv d_k(c_i) + k \pmod{26} \\\\
\Longrightarrow \qquad c_i - k &\equiv d_k(c_i) \pmod{26} \qquad \Longrightarrow \qquad d_k(c_i) \equiv c_i - k \pmod{26}
\end{align*}$$

... which is exactly the decryption function we claimed.

$$$$

---

$$$$

## Encryption with the Shift Cipher with Example

Suppose we wish to encrypt the phrase $\tt{from\,\, the\,\,window\,\,to\,\, the\,\, wall}$ using the shift cipher.

First, we choose a key, which for the shift cipher is just the least residue class of an integer modulo 26. Let's use $k = 10$. 

Next, we'll write our plaintext in 5-block form.

$$\tt{fromt\,\,hewin\,\, dowto \,\,thewa \,\,ll}$$

Next, we'll reference the letter-number conversion chart and convert this to a string of numbers.

$$\begin{align*} 
p = [&5, 17, 14, 12, 19,\\ &7, 4, 22, 8, 13,\\ &3, 14, 22, 19, 14,\\ &19, 7, 4, 22, 0, \\ &11, 11]
\end{align*}$$

Then we'll use our encryption function to encrypt the message. We'll do the first few elements of $p$ explicitly; the remainder of the message is left as an exercise.

$$\begin{align*}
c_1 &= e_k(p_1) \equiv p_1 + k \pmod{26} \equiv 5 + 10 \pmod{26} \\
&\equiv 15 \\\\
c_2 &= e_k(p_2) \equiv p_2 + k \pmod{26} \equiv 17 + 10 \pmod{26} \equiv 27 \pmod{26} \\&\equiv 1 \pmod{26} \\\\
c_3 &= e_k(p_3) \equiv p_3 + k \pmod{26} \equiv 14 + 10 \pmod{26} \\&\equiv 24 \pmod{26}
\end{align*}$$
$$$$

... and so on. The full ciphertext in numerical form is

$$\begin{align*}
c = [&15, 1, 24, 22, 3, \\&17, 14, 6, 18, 23, \\&13, 24, 6, 3, 24, \\&3, 17, 14, 6, 10, \\&21, 21]
\end{align*}$$

We once again reference our chart to convert this back to letters in 5-block form.

$$\tt{PBYWD \,\, ROGSX \,\, NYGDY \,\, DROGK \,\, VV}$$

As discussed previously, it is advised to add additional random text to the end of the last block if it does not contain exactly five letters. This is called **padding** the ciphertext. 

The reason for doing this is that terminating a block exactly where the last word ends gives an adversary information about the message. It turns out that English words are more likely to end with some letters than with others. We'll add $\tt{RCV}$ to the end of our ciphertext. The recipient of the message will know to discard the gibberish after decrypting.

So our ciphertext becomes

$$\tt{PBYWD \,\, ROGSX \,\, NYGDY \,\, DROGK \,\, VVRCV}$$

---

## Python: Encryption with the Shift Cipher
In the cell below is a function called $\texttt{shift\_encrypt()}$ which accepts a string of plaintext and an integer key and encrypts the string of text with the Shift cipher.

$$ \textrm{shift\_encrypt(plaintext, key)} \quad \longrightarrow \quad \textrm{ciphertext}$$

In [12]:
# In the space below is a Python function called shift_encrypt() which accepts
# a string of plaintext and a Shift cipher key and outputs ciphertext
# encrypted using the Shift cipher.

#-------------------------------------------------------------------------------

# Define a function shift_encrypt() to encrypt a plaintext message
# using the shift cipher with a given key.
def shift_encrypt(plaintext,key):  

  # Convert the alphabetic plaintext to numerical plaintext using
  # plain_to_num().
  plaintextNum = plain_to_num(plaintext)

  # Initialize an empty list to store our numerical ciphertext.
  ciphertextNum = []

  # Iterate over the length of plaintextNum, computing an element of
  # the ciphertext at each step, and appending it to ciphertextNum.
  for i in range(len(plaintextNum)):
    ciphertextNum.append((plaintextNum[i]+key) % 26)

  # Convert the numerical ciphertext to alphabetic ciphertext using 
  # num_to_cipher().
  ciphertext = num_to_cipher(ciphertextNum)

  # The function shift_encrypt() returns the ciphertext.
  return ciphertext

# Enter the plaintext and key from the example in the previous section and then 
# run this script to confirm that shift_encryp() works as expected.


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

plaintext = 'from the window to the wall'     # Define our plaintext.
key = 10                                      # Define our key.

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Use the encryption function to encrypt the plaintext with the key and print 
# the output.
print(shift_encrypt(plaintext,key))

PBYWD ROGSX NYGDY DROGK VVTSV


---
$$$$

## Decryption with the Shift Cipher with Example

Let's decrypt the ciphertext from our message earlier. The ciphertext is

$$\tt{PBYWD \,\, ROGSX \,\, NYGDY \,\, DROGK \,\, VVRCV}$$

Converting this to a list of numbers yields

$$\begin{align*}
c = [&15, 1, 24, 22, 3, \\&17, 14, 6, 18, 23, \\&13, 24, 6, 3, 24, \\&3, 17, 14, 6, 10, \\&21, 21, 17, 2, 21]
\end{align*}$$

We apply the decryption function to each element of the ciphertext with the key $k = 10$. We'll do the first few; the rest are left as an exercise.

$$\begin{align*}
p_1 &\equiv d_k(c_1) \equiv c_1 - k \pmod{26} \equiv 15 - 10 \pmod{26} \\
&\equiv 5 \pmod{26}\\\\
p_2 &\equiv d_k(c_2) \equiv c_2 - k \pmod{26} \equiv 1 - 10 \pmod{26} \equiv -9 \pmod{26} \\
&\equiv 17 \pmod{26}\\\\
p_3 &\equiv d_k(c_3) \equiv c_3 - k \pmod{26} \equiv 24 - 10 \pmod{26} \\
&\equiv 14 \pmod{26}
\end{align*}$$

Referencing our chart, we see that the first three letters of the plaintext are $\tt{fro}$. Continuing on in this fashion yields the plaintext:

$$\tt{fromt \,\, hewin \,\, dowto \,\, thewa \,\, llhsl}$$

Discarding the padding at the end and adding spaces where appropriate yields the original message:

$$\tt{from \,\, the \,\, window \,\, to \,\, the \,\, wall}$$

$$$$

---

$$$$

## Python: Decryption with the Shift Cipher
In the space below is a Python function called **shift_decrypt** which accepts a string of ciphertext and a shift cipher key and outputs plaintext.

$$ \textrm{shift\_decrypt(ciphertext, key)} \quad \longrightarrow \quad \textrm{plaintext}$$


In [13]:
# In the space below is a Python function called shift_decrypt() which accepts
# a string of ciphertext and a Shift cipher key and outputs the corresponding
# plaintext which was encrypted using the Shift cipher.

#-------------------------------------------------------------------------------

# Define a function shift_decrypt() to decrypt a plaintext message
#     using the shift cipher with a given key.
def shift_decrypt(ciphertext,key):  

  # Convert our alphabetic ciphertext to numerical ciphertext.
  ciphertextNum = cipher_to_num(ciphertext)

  # Initialize an empty list to store our numerical plaintext.
  plaintextNum = []

  # Iterate over the length of ciphertextNum, computing an element of
  # the plaintext at each step, and appending it to plaintextNum.
  for i in range(len(ciphertextNum)):
    plaintextNum.append((ciphertextNum[i]-key) % 26)

  # Convert the numerical plaintext to alphabetic plaintext.
  plaintext = num_to_plain(plaintextNum)

  # The function shift_decrypt() returns the plaintext.
  return plaintext

# Enter the ciphertext and key from the example in the previous section to 
# confirm that shift_decrypt() functions as expected.


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

ciphertext = 'PBYWD ROGSX NYGDY DROGK VVRCV'     # Define our ciphertext.
key = 10                                         # Define our key.

#-------------------------------------------------------------------------------
# EDIT ABOVE  THIS LINE


# Use the decryption function to decrypt the ciphertext with the key.
print(shift_decrypt(ciphertext,key))

fromt hewin dowto thewa llhsl


---
$$$$
## Exercise: Encrypted Message Exchange with Shift Cipher
Your research project leader has now placed you in teams. Each team will separately choose a key and a message to encrypt and send to another team. **You will only send the key and the ciphertext in 5-block form to the other team.** 

After receiving ciphertext and a key from another team, decrypt their message. You may use the space below to write an encryption/decryption script. Feel free to use the encryption/decryption functions from the preceding sections.

In [14]:
# Use this space to write an encryption script.

In [15]:
# Use this space to write a decryption script.

---
$$$$

## Shift Cipher: Order of the Key Space
We begin this discussion with some definitions.

> *Definition 12.* $\quad$ A **set** is a mathematical object containing other mathematical objects. The objects contained in a set are called its **elements**.

We use curly brackets $\{$ and $\}$ to denote inclusion in a set. 

$$$$
*Example.* $\quad$ $\{2,4,6\}$ is the set containing the numbers 2, 4, and 6. The numbers 2, 4, and 6 are the elements of this set.

*Example.* $\quad$ $\{\pi, -10\}$ is a set containing the numbers $\pi \approx 3.1415926$ and $-10$, which are the elements of this set.

*Example.* $\quad$ We can assign a symbol to represent a set so that we don't have to write out every single element of a set to reference it. For example, we could say that $X = \{0, 1, 2\}$, so that whenever I talk about $X$, you know that I am talking about the set containing the numbers 0, 1, and 2.

$$$$

> *Definition 13.* $\quad$ The **order** of a set is the number of elements it contains. We denote the order of a set $S$ by $|S|$.  

$$$$

*Example.* $\quad$ The order of the set $S$ referenced earlier is $|S| = 3$ because it contains three elements: the numbers 2, 4, and 6.

$$$$

We previously said that $\mathcal{K}$ was called the **key space** for a given cryptosystem. The key space is the set of all possible (unique) keys for a given cryptosystem.

*Question:* What is the order of the key space $|\mathcal{K}|$ for the shift cipher?

*Answer:* A key for the shift cipher is the least residue class of an integer modulo 26. Recall that a least residue class $r$ of an integer modulo 26 is an integer between 0 and 25. Since there are 26 integers between 0 and 26 inclusive, the order of the key space for the shift cipher is $|\mathcal{K}| = 26$.

---
$$$$
## Cryptanalysis of the Shift Cipher
Cryptographic analysis, or just **cryptanalysis**, is the study and practice of decrypting enciphered text *without knowledge of the key*, and in most cases, without knowledge of what cryptosystem was used.

There are [many different types of cryptanalysis](https://en.wikipedia.org/wiki/Attack_model). We will concern ourselves with [ciphertext-only attacks](https://en.wikipedia.org/wiki/Ciphertext-only_attack). Such an attack is the type made when an adversary only has access to ciphertext - it is among the worst-case scenarios for the attacker. We will also assume that we know which cryptosystem was used to perform the encryption.

The simplest (but often the most unwieldy) type of ciphertext-only attack is called the **brute force attack**, in which the attacker simply tries using every single key in the key space for the encryption method. Indeed, the larger the key space, the more computationally intensive a brute force attack becomes. 

*Question:* Do you think it is reasonable to use a brute force attack on text encrypted with a shift cipher?

$$$$

---

$$$$

## Exercise: Cryptanalysis of the Shift Cipher
Execute a brute-force attack on the ciphertext linked below which was encrypted using a Shift cipher and an unknown key.

$$$$

> [Shift ciphertext in 5-block form.](https://drive.google.com/file/d/1I0XQzPVS5ntOu0QJfUtp2XnbUMxE8Mcv/view?usp=sharing)

> [Shift ciphertext without spaces](https://drive.google.com/file/d/13s-_IdB27XzEmNTy--huekv3-8Rg1D1H/view?usp=sharing).

$$$$

There is a function called **shift_attack** in the cell below which automatically executes a brute-force attack on given ciphertext. Use it to execute a brute-force attack on the supplied ciphertext.


In [16]:
# Below is a function called shift_attack() which executes a brute-force attack
# on ciphertext encrypted with the Shift cipher.

#-------------------------------------------------------------------------------

# Define a function called shift_attack() which executes a brute-force attack
# on ciphertext assuming it was encrypted with the Shift cipher.
def shift_attack(ciphertext):

  # Call the function shift_decrypt() on the ciphertext for every element of the 
  # key space.
  for candidateKey in range(26):
    print('Candidate Plaintext, key = %s: %s' % (candidateKey,shift_decrypt(ciphertext,candidateKey)))



# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Initialize a list to store the ciphertext.
ciphertext = '''QVWKM IVQII BBPMX ZMAMV BLIGA KQMVK MQVBP MWTLA MVAMP IAITU WABKM IAMLB 
    WMFQA BQVVM EAXMI SBPMZ MQAVW EWZLN WZAKQ MVKMB PMMUX QZQKI TUMBP WLWNB 
    PWCOP BWVEP QKPIT TBPMA KQMVB QNQKI KPQMD MUMVB AWNBP MXIAB EMZMN WCVLM 
    LQAWX XWAML BWBPM UWABN CVLIU MVBIT XZQVK QXTMA WNQVO AWKIV LMDMV BMKPV 
    WTWOQ KITXZ WOZMA AWVTG PIXXM VAEPM VQBAX ZWLCK BAKIV QVAWU MEIGJ MCAML 
    NWZBP MLQUQ VCBQW VWNPC UIVTQ JMZBG QVITT BPMCA MNCTI ZBABP MEWZT LQAMQ 
    BPMZA BIVLQ VOABQ TTWZO WQVOJ IKSEI ZLABP MNQMT LAIZM KCTBQ DIBML EQBPP 
    WZAMX TWCOP AEPQT MJWWS AIZME ZQBBM VJGUI KPQVM ZGJCB QVUIB BMZAW NDQBI 
    TQUXW ZBIVK MUMIV QVOQV MNNMK BEIZI VLXWT QKMMA XQWVI OMBPM MUXQZ QKITI 
    XXZWI KPQAA BQTTM VKWCZ IOMLW ZIBTM IABBW TMZIB MLBPM BEWIQ UAWNB PMXIZ 
    BGIZM BWKWV YCMZB PMEPW TMACZ NIKMW NBPMM IZBPI VLBWM FBQVO CQAPW VKMIV 
    LNWZI TTBPM XWAAQ JQTQB GWNQV LMXMV LMVBB PWCOP BRIIQ'''

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Run the shift cipher decryption function using every key in the key space.
shift_attack(ciphertext)

Candidate Plaintext, key = 0: qvwkm ivqii bbpmx zmamv bliga kqmvk mqvbp mwtla mvamp iaitu wabkm iamlb wmfqa bqvvm eaxmi sbpmz mqavw ewzln wzakq mvkmb pmmux qzqki tumbp wlwnb pwcop bwvep qkpit tbpma kqmvb qnqki kpqmd mumvb awnbp mxiab emzmn wcvlm lqawx xwaml bwbpm uwabn cvliu mvbit xzqvk qxtma wnqvo awkiv lmdmv bmkpv wtwoq kitxz wozma awvtg pixxm vaepm vqbax zwlck bakiv qvawu meigj mcaml nwzbp mlquq vcbqw vwnpc uivtq jmzbg qvitt bpmca mncti zbabp mewzt lqamq bpmza bivlq voabq ttwzo wqvoj iksei zlabp mnqmt laizm kctbq dibml eqbpp wzamx twcop aepqt mjwws aizme zqbbm vjgui kpqvm zgjcb qvuib bmzaw ndqbi tquxw zbivk mumiv qvoqv mnnmk beizi vlxwt qkmma xqwvi ombpm muxqz qkiti xxzwi kpqaa bqttm vkwcz iomlw zibtm iabbw tmzib mlbpm bewiq uawnb pmxiz bgizm bwkwv ycmzb pmepw tmacz nikmw nbpmm izbpi vlbwm fbqvo cqapw vkmiv lnwzi ttbpm xwaaq jqtqb gwnqv lmxmv lmvbb pwcop briiq
Candidate Plaintext, key = 1: puvjl huphh aaolw ylzlu akhfz jpluj lpuao lvskz luzlo hzhst vzajl hzlka vlepz apuul dzwlh raol

---
$$$$
# **The Affine Cipher**
The affine cipher is a generalization of the shift cipher. Whereas the key for the shift cipher was the least residue class of an integer modulo 26, the key for the affine cipher is a pair of integers $(a,b)$ with $0 \leq a, b < 26$. 

## Encryption with the Affine Cipher

The encryption function for the affine cipher is

$$e_{ab}(p_i) \equiv a\cdot p_i + b \pmod{26}$$

However, there are some additional constraints we need to place on $a$. We will also need some new mathematics to derive the decryption function from the encryption function.

## Mathematics for Decryption


> *Definition 14.* $\quad$ Let $x$ and $y$ be integers not both zero. An integer $d$ is called the **greatest common divisor** of two integers $a$ and $b$ if it satisfies all three of the following conditions: 
1. $d$ divides $x$.
2. $d$ divides $y$.
3. If $c$ is a divisor of both $x$ and $y$, then $c \leq d$.

We denote the greatest common divisor of $x$ and $y$ by ${\textrm{gcd}}(x,y)$.

$$$$

*Example.* $\quad$ Let $x = 90$ and $y = 84$. Then ${\textrm{gcd}}(x,y) = 6$. To see this, note that the prime factorizations of $x$ and $y$ are

$$\begin{align*}
90 = 2 \cdot 3^2 \cdot 5 \qquad \textrm{and} \qquad 84 = 2^2 \cdot 3 \cdot 7
\end{align*}$$

The largest product of prime powers appearing in both prime factorizations is $2^1 \cdot 3^1 = 6$.

$$$$

This week, if you want to know the greatest common divisor of two numbers, go to [wolframalpha.com and type gcd([first number], [second number])](https://www.wolframalpha.com/input/?i=gcd%2884%2C90%29). Alternatively, use the python function GCD.

In [17]:
from math import gcd

# Try changing x and y to experiment.
x = 90
y = 84

# Call the GCD function and evaluate it at x and y.
gcd(x,y)

6

> *Definition 15.* $\quad$ Two integers $x$ and $y$ are said to be **coprime** or **relatively prime** if ${\textrm{gcd}}(x,y) = 1$.

$$$$

*Example.* $\quad$ The integers 25 and 16 are coprime. The only divisors of 25 are 1, 5, and 25 and the only divisors of 16 are 1, 2, 4, 8, and 16. The largest element common to both lists is 1. 

$$$$

*Example.* $\quad$ The integers 84 and 90 from a previous example are *not* coprime since their greatest common divisor is 6.

$$$$

We are now in a position to state the special condition we need to place on the value of $a$ in the affine cipher key. We'll need $a$ to be coprime with 26.

What is the reason for this? Let's try to derive the affine cipher's decryption function from the encryption function.

$$\begin{align*}
&& e_{ab}(p_i) &\equiv a\cdot p_i + b \pmod{26} \\\\
&\Longrightarrow & c_i &\equiv a\cdot d_{ab}(c_i) + b \pmod{26} \\\\
&\Longrightarrow & c_i - b &\equiv a \cdot d_{ab}(c_i) \pmod{26}
\end{align*}$$

At this point we'd be tempted to divide both sides of the congruence through by $a$. If we did, we'd end up with the following **false** decryption function:

$$ d_{ab}(c_i) \equiv \frac{c_i - b}{a} \pmod{26}$$

Why is this bad? There is no guarantee that the fraction on the right-hand side is an integer. What does it even mean to decide whether an integer (the left-hand side) is congruent to a fraction (the right-hand side) modulo 26? This is a completely meaningless congruence.

It turns out that if an integer is coprime with 26, then it is "invertible" modulo 26. 

> *Definition 16.* $\quad$ Let $a$ and $n$ be integers with $n>1$. If there exists an integer $q$ such that $a\cdot q \equiv 1 \pmod{n}$, then $q$ is called a **multiplicative inverse** of $a$, and we write $q = a^{-1}$. If $a$ has a multiplicative inverse, then it is said to be **invertible**.

$$$$

*Example.* $\quad$ Let $a = 5$ and $n=26$. Then $a$ is invertible modulo $n$, and its multiplicative inverse is $a^{-1} = 21$. 

$$a\cdot a^{-1} = 5 \cdot 21 = 105 \equiv 1 \pmod{26}$$

$$$$

*Example.* $\quad$ Let $a = 2$ and $n = 10$. Then $a$ is NOT invertible modulo $n$. Try making a list of products of $2$ with every integer modulo $10$ and you'll see that you never end up with 1. You'll also notice a pattern forming when you try writing these products out in order. For example:

$$\begin{align*}
2\cdot 0 &\equiv 0 \pmod{10} &\qquad  2\cdot 5 &= 10 \equiv 0 \pmod{10} \\
2\cdot 1 &\equiv 2 \pmod{10} &\qquad  2\cdot 6 &= 12 \equiv 2 \pmod{10} \\
2\cdot 2 &\equiv 4 \pmod{10} &\qquad  2\cdot 7 &= 14 \equiv 4 \pmod{10} \\
2\cdot 3 &\equiv 6 \pmod{10} &\qquad  2\cdot 8 &= 16 \equiv 6 \pmod{10} \\
2\cdot 4 &\equiv 8 \pmod{10} &\qquad  2\cdot 9 &= 18 \equiv 8 \pmod{10} \\
\end{align*}$$

Compare the results in the left and right-hand columns above. The results of the products begin to repeat - they will indeed continue to do so forever. The same would happen if you multiplied 2 by negative integers.

$$$$

> *Theorem 2.* $\quad$ Multiplicative inverses modulo a fixed integer $n > 1$ are unique. 

*Proof.* $\quad$ Let $a$ and $n$ be integers with $n > 1$. Assume, for the sake of contradiction, that $a$ has two distinct inverses; call them $a^{-1}_1$ and $a^{-1}_2$. Then $a_1^{-1} \not\equiv a^{-1}_2 \pmod n$. We have

$$\begin{align*}
a^{-1}_1 &\equiv  a^{-1}_1\cdot 1 \pmod{n}&& \textrm{by defn. of multiplicative identity} \\\\
&\equiv a^{-1}_1 \cdot (a \cdot a_2^{-1})\pmod{n}  && \textrm{by defn. of $a_2^{-1}$}\\\\
&\equiv (a^{-1}_1 \cdot a) \cdot a_2^{-1} \pmod{n}&&\textrm{by associativity of multiplication}\\\\
&\equiv 1 \cdot a_2^{-1} \pmod{n}&&\textrm{by defn. of $a_1^{-1}$}\\\\
&\equiv a_2^{-1} \pmod{n}&&\textrm{by defn. of multiplicative identity}
\end{align*}$$

Hence $a_1^{-1} \equiv a_2^{-1} \pmod{n}$. But we assumed that $a_1^{-1}$ was not congruent to $a_2^{-1}$ modulo $n$. This is a contradiction. Therefore multiplicative inverses are unique. $\quad \blacksquare$

$$$$

The useful consequence of this theorem is that once you have found **one** multiplicative inverse for a given integer modulo $n$, you know that you have found **all** of them.

For this week, if you want to know the multiplicative inverse of an integer $a$ modulo another integer $n$, go to [wolframalpha.com and type "inverse of [insert $a$ here] modulo [insert $n$ here]"](https://www.wolframalpha.com/input/?i=inverse+of+5+modulo+26).

Finally, we get to the reason we said that the first part of the affine cipher's key needs to be coprime with 26.

> *Theorem 3.* $\quad$ Let $a$ and $n$ be integers with $n > 1$. Then $a$ is invertible if and only if $a$ and $n$ are coprime.

The proof of Theorem 3 is beyond the scope of this project. If you are feeling intrepid, you may reference [this page](https://proofwiki.org/wiki/B%C3%A9zout%27s_Lemma#Proof_3) (see Proof 3) followed by [this page](https://proofwiki.org/wiki/Multiplicative_Inverse_in_Ring_of_Integers_Modulo_m) (see Proof 1). 

Below is a Python function which will compute the multiplicative inverse of an integer $a$ modulo $n$, if it exists. You will need this later.

In [18]:
# Define a function called inverseMod() which accepts an integer a and a modulus
# n and returns the inverse of a (mod n) or gives an error message when 
# a (mod n) fails to exist.
def inverseMod(a,n):

  # Ensure that a is mapped to its least residue class modulo n.
  a = a % n

  # Initialize a list of potential candidates for being the inverse of a (mod n).
  candidates = range(1,n+1)

  # Iterate over all candidate integers.
  for x in candidates:
    # If x has the value n, then the FOR loop has reached the end of available
    #     candidates, so a isn't invertible mod n.
    if x is n:
      # Print an error statement and return.
      print('%s is not invertible modulo %s' % (a,n))
      return
    # If x*a = 1 (mod n), then a is invertible and x is its inverse.
    if ((x*a) % n) is 1:
      # Return the inverse of a.
      return x


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Try checking invertibility/inverses with different values of a and n below.
a = 7
n = 26

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


inverseMod(a,n)

15

---

## Decryption with the Affine Cipher
Ok, great, why do we care? Let's return to our derivation of the decryption function. 

We will now assume that $a$ is coprime with 26. It follows that $a$ is invertible modulo 26, i.e. that there exists an $a^{-1}$ such that $a\cdot a^{-1} \equiv 1 \pmod{26}$.

We last left off at $c_i - b \equiv a \cdot d_{ab}(c_i) \pmod{26}$. Since $a$ is now invertible under our assumption that ${\textrm{gcd}}(a, 26) = 1$ by Theorem 3, we can invoke the existence of $a^{-1}$ and multiply both sides of the congruence by $a^{-1}$.

$$$$

$$\begin{align*}
&& c_i - b &\equiv a \cdot d_{ab}(c_i) \pmod{26} \\\\
&\Longrightarrow & a^{-1} \cdot (c_i - b) &\equiv a^{-1}\cdot \left(a\cdot d_{ab}(c_i)\right) \pmod{26} \\\\
&\Longrightarrow & a^{-1} \cdot (c_i - b) &\equiv (a^{-1}\cdot a)\cdot d_{ab}(c_i) \pmod{26} \\\\
&\Longrightarrow & a^{-1} \cdot (c_i - b) &\equiv (1)\cdot d_{ab}(c_i) \pmod{26} \\\\
&\Longrightarrow & a^{-1} \cdot (c_i - b) &\equiv d_{ab}(c_i) \pmod{26}
\end{align*}$$

$$$$

It follows that 
$$d_{ab}(c_i) \equiv a^{-1} \cdot (c_i - b) \pmod{26} $$

---
## Example: Encryption with the Affine Cipher
To summarize, for the affine cipher:

> The key is an ordered pair $(a,b)$ of least residue classes of integers modulo 26 such that $a$ is coprime with 26.

> The encryption function is $e_{ab}(p_i) \equiv a\cdot p_i + b \pmod{26}$.

> The decryption function is $d_{ab}(c_i) \equiv a^{-1} \cdot (c_i - b) \pmod{26}$.

One interesting fact to note is that the affine cipher reduces to the shift cipher in the case $a = 1$, so to keep things interesting we will not allow ourselves to choose $a = 1$.

Suppose that Alice wants to use the affine cipher to encrypt the following message and send it to Bob.

$$\tt{I've \,\, got \,\, friends \,\, in \,\, low \,\, places.}$$

Alice removes all the punctuation and writes the message in 5-block form.

$$\tt{ivego \,\, tfrie \,\, ndsin \,\, lowpl \,\, aces}$$

She converts it to a list of numbers $p$ using her reference table.

$$\begin{align*}
p = [&8, 21, 4, 6, 14, \\
&19, 5, 17, 8, 4, \\
&13, 3, 18, 8, 13 \\
&11, 14, 22, 15, 11 \\
& 0, 2, 4, 18]
\end{align*}$$

Before she can encrypt her plaintext, she needs to choose a key. We'll write a Python script to do this for us, keeping in mind that $a$ must be coprime with 26. We'll need the functions random.randint() and random.choice() from the Python package RANDOM.

In [19]:
# Import the Python package RANDOM.
import random

# Randomly choose an integer value for b so that 0 <= b < 25.
b = random.randint(0,25)

# Initialize an empty list to store least residue classes mod 26 coprime with 26.
coprimeList = []

# Iterate over the integers between 2 and 25. Dummy variable is x.
# Check if gcd(x, 26) = 1. 
# If it is, add it to the list 'coprimeList'. Otherwise, don't.
for x in range(2,26):
  if gcd(x,26) is 1:
    coprimeList.append(x)

# Randomly choose an element of coprimeList and assign its value to a.
a = random.choice(coprimeList)

# Print the values of a and b.
print('a = %s and b = %s' % (a,b))

a = 5 and b = 2


Suppose Alice chooses as her key $(a,b) = (7,3)$. 

She proceeds to encrypt the plaintext using the encryption function. We'll do the first three characters below; the rest are left to the reader.

$$\begin{align*}
e_{ab}(p_1) &= a\cdot p_1 + b = 7\cdot 8 + 3 = 59\\
&\equiv 7 \pmod{26} \\\\
e_{ab}(p_2) &= a\cdot p_2 + b = 7\cdot 21 + 3 = 150\\
&\equiv 20 \pmod{26} \\\\
e_{ab}(p_3) &= a\cdot p_3 + b = 7\cdot 4 + 3 = 31\\
&\equiv 5 \pmod{26} \\\\
\end{align*}$$

The complete ciphertext $c$ is

$$\begin{align*}
c = [&7, 20, 5, 19, 23, \\
&6, 12, 18, 7, 5, \\
&16, 24, 25, 7, 16, \\
&2, 23, 1, 4, 2, \\
&3, 17, 5, 25, 23]
\end{align*}$$

Converting this back to letters yields the ciphertext she'll send to Bob:

$$\begin{align*}
\tt{HUFTX \,\, GMSHF \,\, QYZHQ \,\, CXBEC \,\, DRFZX}
\end{align*}$$

---
## Python: Encryption with the Affine Cipher
In the cell below is a Python function called **affine_encrypt** which accepts a string of plaintext and an affine cipher key and outputs text encrypted with the affine cipher.

$$$$

$$\begin{align*} &\textrm{affine\_encrypt(plaintext, key)} \quad \longrightarrow \quad \textrm{ciphertext} \\\\ &\textrm{key} = [a,b] \end{align*}$$

Use this function to encrypt the sample plaintext using the key we chose.

In [20]:
# Below is a Python function called affine_encrypt() which accepts plaintext and
# an affine cipher key and encrypts the former using the latter.

#-------------------------------------------------------------------------------

# Define a function affine_encrypt() to encrypt a plaintext message
# using the affine cipher with a given key.
def affine_encrypt(plaintext,key):  

  # Convert the alphabetic plaintext to numerical plaintext.
  plaintextNum = plain_to_num(plaintext)

  # Initialize an empty list to store the numerical ciphertext.
  ciphertextNum = []

  # Iterate over the length of plaintextNum, computing an element of
  # the ciphertext at each step, and append it to ciphertextNum.
  for i in range(len(plaintextNum)):
    ciphertextNum.append((key[0]*plaintextNum[i]+key[1]) % 26)

  # Convert the numerical ciphertext to alphabetic ciphertext.
  ciphertext = num_to_cipher(ciphertextNum)

  # The function affine_encrypt returns the ciphertext.
  return ciphertext


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Use this space to test affine_encrypt() using the plaintext and key from
# the example in the previous section.

# Define the plaintext..
plaintext = 'Ive got friends in low places.'    

# Define the key
key = [7,3]      

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Use the encryption function to encrypt p with k and print the output.
print(affine_encrypt(plaintext, key))

HUFTX GMSHF QYZHQ CXBEC DRFZY


---
## Example: Decryption with the Affine Cipher
Bob receives Alice's message encrypted with the affine cipher:

$$\begin{align*}
\tt{HUFTX \,\, GMSHF \,\, QYZHQ \,\, CXBEC \,\, DRFZX}
\end{align*}$$

He converts it to numerical form:

$$\begin{align*}
c = [&7, 20, 5, 19, 23, \\
&6, 12, 18, 7, 5, \\
&16, 24, 25, 7, 16, \\
&2, 23, 1, 4, 2, \\
&3, 17, 5, 25, 23]
\end{align*}$$

Bob then uses the key (which was previously exchanged using physical security measures) and the decryption function to decrypt the ciphertext. We'll do the first three elements of the ciphertext here.

Recall that we need the multiplicative inverse of $a$ to plug into the decryption function. [Wolfram Alpha gives](https://www.wolframalpha.com/input/?i=inverse+of+7+mod+26) $7^{-1} \equiv 15 \pmod{26}$.

$$\begin{align*}
d_{ab}(c_1) &= a^{-1}\cdot(c_1 - b) = 15 \cdot (7 - 3) = 60\\
&\equiv 8 \pmod{26} \\\\
d_{ab}(c_2) &= a^{-1}\cdot(c_2 - b) = 15 \cdot (20 - 3) = 255\\
&\equiv 21 \pmod{26} \\\\
d_{ab}(c_3) &= a^{-1}\cdot(c_3 - b) = 15 \cdot (5 - 3) = 30\\
&\equiv 4 \pmod{26} \\\\
\end{align*}$$

Bob references his cryptographic conversion table, which yields the full plaintext after doing the rest of the calculations.

$$\tt{ivego \,\, tfrie \,\, ndsin \,\, lowpl \,\, aceso}$$

Bob discards the padding (only the last letter), and after introducing sensible spacing, the plaintext is

$$\tt{ive \,\, got \,\, friends \,\, in \,\, low \,\, places}$$

... as expected.



---
## Python: Decryption with the Affine Cipher
Below is a Python function called **affine_decrypt** which accepts a string of ciphertext and an affine cipher key and outputs plaintext.

$$$$

$$\begin{align*} &\textrm{affine\_decrypt(ciphertext, key)} \quad \longrightarrow \quad \textrm{plaintext} \\\\ &\textrm{key} = [a,b] \end{align*}$$

Use this function to decrypt the sample ciphertext using the key we chose.

In [21]:
# Use this space to write a function which will decrypt an arbitrary
# string of ciphertext with the affine cipher using a given key.

#-------------------------------------------------------------------------------

# Define a function affine_decrypt() to decrypt a plaintext message
# using the affine cipher with a given key.
def affine_decrypt(ciphertext,key):  

  # Compute the inverse of a.
  aInv = inverseMod(key[0],26)

  # Convert the alphabetic ciphertext to numerical ciphertext.
  ciphertextNum = cipher_to_num(ciphertext)

  # Initialize an empty list to store our plaintext in numerical form.
  plaintextNum = []  

  # Iterate over the length of ciphertextNum, computing an element of
  # the plaintext at each step, and appending it to plaintextNum.
  for i in range(len(ciphertextNum)):
    plaintextNum.append((aInv*(ciphertextNum[i]-key[1])) % 26)

  plaintext = num_to_plain(plaintextNum)

  # The function affine_decrypt() returns the plaintext.
  return plaintext


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Use this space to test affine_decrypt() using the plaintext and key from
# the example in the previous section.

ciphertext = 'HUFTX GMSHF QYZHQ CXBEC DRFZX'     # We call our ciphertext c.
key = [7,3]                               # We call our key k.

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Use the decryption function to decrypt the ciphertext with the key and 
# print the output.
print(affine_decrypt(ciphertext, key))

ivego tfrie ndsin lowpl aceso


---
$$$$
## Exercise: Encrypted Message Exchange with Affine Cipher
Your research project leader has now placed you in teams. Each team will separately choose a key and a message to encrypt and send to another team. **You will only send the key and the ciphertext in 5-block form to the other team.** 

After receiving ciphertext and a key from another team, decrypt their message. You may use the space below to write an encryption/decryption script. Feel free to use the encryption/decryption functions from the preceding sections.

In [22]:
# Use this space to write an encryption script.

In [23]:
# Use this space to write a decryption script.

---
## Affine Cipher: Order of the Key Space
Use the multiplicative rule of combinatorics to compute the order of the key space for the affine cipher.

> *Multiplicative Rule of Combinatorics:*  Let $A$ and $B$ be finite sets with orders $|A|$ and $|B|$. Then there are $|A|\cdot |B|$ ways of choosing one element each from both sets.

$$$$
*Example.* $\quad$ Suppose you are ordering a hamburger from a restaurant, and you are told you may select one type of cheese and one topping. How many unique combinations of toppings/cheese could you possibly order? The available cheeses and toppings are:

- No cheese / American cheese / cheddar cheese / pepperjack cheese
- No topping / lettuce / tomato / onion / jalepenos / mushrooms

You have four choices of cheese and six choices of topping. It follows that there are $4 \cdot 6 = 24$ unique combinations to choose from.

$$$$

With this in mind, the affine cipher's key has two independent pieces of information in it: the integer $a$ and the integer $b$. How many choices do you have for $a$? How many for $b$?

**Compute $|\mathcal{K}|$ for the affine cipher.**

Do you think it is reasonable to conduct a brute force attack on text encrypted with the affine cipher? How long do you think it would take you to sort through the candidate plaintexts after writing a script to generate them?

---
## Cryptanalysis of the Affine Cipher
Irrespective of your answer to the previous question, we will not be performing a brute force attack on the affine cipher. Instead, we will learn and employ another tenet of cryptanalysis. The following is the guiding principle behind nearly every type of ciphertext-only attack:

**Do everything in your power to reduce the effective size of the key space. Cheat and swindle your way to the decrypted plaintext.**

Practically speaking, this means that we are going to discard keys which would produce candidate plaintexts which are highly unlikely to be the correct plaintext. We will do this by abusing certain properties of the English language. 

As it turns out, when you count the appearances of each letter in a large volume of English text, the ratio of these appearances to the total number of letters counted tends to converge to a particular value. We present these values for the most common English letters below, but [a complete table can be found here](http://pi.math.cornell.edu/~mec/2003-2004/cryptography/subs/frequencies.html). (The higher the frequency, the more common the letter is.)

E | T | A | I | O | N 
--- | --- | --- | --- | --- | ---
12.02 | 9.10 | 8.12 | 7.68 | 7.31 | 6.95

The lower a letter's frequency, the closer its frequency tends to match its neighbors' frequencies. For this reason, it can be difficult to discern letters by their frequencies alone in a relatively small sample of text. Nevertheless, this is a powerful cryptanalytic tool which helped to carry cryptanalysis through the early 21st century.

Now, let's see how this can help us attack an affine ciphertext. Suppose we are given the following affine ciphertext (which, being a relatively short example, we have engineered to have just the right letter frequencies):

$$\tt{AJANG \,\, QRANE \,\, LAMLN \,\, GWRYL \,\, QLECU \,\, TAAHL \,\, ARMWJ \,\, AAATM}$$

We could manually count the letters, but it will be convenient to write a Python function which will do this for us. A function called **count_mono** which does this automatically for a single letter, and another function called **count_mono_all** which counts every single letter and prints the result is presented below. 


In [24]:
# Define a function count_mono() to count English letter monograms, i.e. single
# letters in a given text.
def count_mono(text, letter):

  #Initialize a variable to store the number of times the given letter appears.
  letterCount = 0

  # Iterate over the text, counting the number of times the given letter appears.
  # We evaluate each element of the text as lowercase so as to not miscount.
  for i in range(len(text)):
    if text[i].lower() == letter.lower():
      letterCount = letterCount + 1
  
  # count_mono() returns the number of occurrences of the given letter.
  return letterCount

#-------------------------------------------------------------------------------

# Define a function called count_mono_all() which counts all English monograms
# in a given text sample and prints the associated numbers.
def count_mono_all(text, dispFlag):

  # Initialize a list to contain all English letters.
  alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
              'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
              'y', 'z']

  # Iteratively invoke count_mono() on each letter in the alphabet, and print
  # the result.
  letterCount = []
  for i in range(len(alphabet)):
    letterCount.append(count_mono(text,alphabet[i]))
  if dispFlag == 1:
    for i in range(len(letterCount)):
        print('{0} : {1}'.format(alphabet[i].upper(), letterCount[i]))
  return letterCount


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Use this space to test count_mono() and/or count_mono_all() on the example
# ciphertext from the previous section.

text = 'AJANGQRANELAMLNGWRYLQLECUTAAHLARMWJAAATM'

letter = 'A'

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


print(count_mono(text, letter))

print(count_mono_all(text, 1))

10
A : 10
B : 0
C : 1
D : 0
E : 2
F : 0
G : 2
H : 1
I : 0
J : 2
K : 0
L : 5
M : 3
N : 3
O : 0
P : 0
Q : 2
R : 3
S : 0
T : 2
U : 1
V : 0
W : 2
X : 0
Y : 1
Z : 0
[10, 0, 1, 0, 2, 0, 2, 1, 0, 2, 0, 5, 3, 3, 0, 0, 2, 3, 0, 2, 1, 0, 2, 0, 1, 0]


Whether we did this manually or used the code above, we'd find that $\tt{A}$ is the most common ciphertext letter and that $\tt{L}$ is the second most common ciphertext letter. 

Referring to the frequency table, it seems reasonable to guess that ciphertext $\tt{A}$ corresponds to plaintext $\tt{e}$; likewise, ciphertext $\tt{L}$ corresponds to plaintext $\tt{t}$. It turns out that - if our guess is correct - this is all the information we need to solve for the key explicitly. 

In terms of the unknown key quantities $a$ and $b$, we can express the act of encrypting $\tt{e}$ and $\tt{t}$ respectively to $\tt{A}$ and $\tt{L}$ via this system of congruences:

$$\begin{align*}
{\tt{A}} &= e_{ab}({\tt{e}}) \equiv a\cdot {\tt{e}}+ b \pmod{26}\\\\
{\tt{L}} &= e_{ab}({\tt{t}}) \equiv a\cdot {\tt{t}}+ b \pmod{26}
\end{align*}$$

Replacing the ciphertext and plaintext letters with the least residue classes they represent yields the system of congruences

$$\begin{align*}
0  &\equiv a\cdot 4 + b \pmod{26} \tag{1}\\\\
11  &\equiv a\cdot 19 + b \pmod{26} \tag{2}
\end{align*}$$

Subtracting congruence (1) from congruence (2) will effectively get rid of the $b$ term on the right-hand side.

$$\begin{align*}
&&(11) - (0) &\equiv (19a + b) - (4a + b) \pmod{26} \\\\
&\Longrightarrow & 11 &\equiv (19a - 4a) + (b-b) \pmod{26} \\\\
&\Longrightarrow & 11 &\equiv 15a \pmod{26} \\\\
&\Longrightarrow & 15a &\equiv 11 \pmod{26} \tag{3} \\\\
\end{align*}$$

Now, unless 15 is invertible, we're stuck. If you've made a correct guess in the initial step, the coefficient of $a$ at this step will **always** be invertible. If it isn't, you made an arithmetic error or your guess was wrong.

We'll use our function INVERSEMOD from earlier to find $15^{-1}$.


In [25]:
inverseMod(15,26)

7

So $7\cdot 15 \equiv 1 \pmod{26}$. Multiplying both sides of congruence (3) by 7 yields

$$\begin{align*}
&&7\cdot(15a) &\equiv 7\cdot 11 \pmod{26} \\\\
&\Longrightarrow & (7\cdot 15)a &\equiv 25 \pmod{26} \\\\
&\Longrightarrow & a &\equiv 25 \pmod{26}
\end{align*}$$

Now that we know the value of $a$, we can plug it into either of congruences (1) or (2) and solve for $b$. We'll use congruence (1). First, we solve it for $b$, and then plug in known values.

$$\begin{align*}
&& 4a + b &\equiv 0 \pmod{26} \\\\
&\Longrightarrow & b &\equiv -4a \pmod{26} \\\\
&\Longrightarrow & b &\equiv -4\cdot25 \pmod{26} \\\\
&\Longrightarrow & b &\equiv 4 \pmod{26}
\end{align*}$$

To check that our solution is correct, we can run **affine_decrypt** on the ciphertext with the key $(25, 4)$ to see if we get readable plaintext.

In [26]:
# Set the key = [a, b] = [25, 4]
key = [25,4]

# Input our ciphertext as a string.
ciphertext = 'AJANG QRANE LAMLN GWRYL QLECU TAAHL ARMWJ AAATM'

# Invoke affine_decrypt on the ciphertext and suspected key.
affine_decrypt(ciphertext,key)

'every onera testr yingt otack leext ensiv eeels'

The decrypted plaintext reads

$$\tt{every \,\, onera \,\, testr \,\, yingt \,\, otack \,\, leext \,\, ensiv \,\, eeels}$$

After suitable rearrangment, the message is

$$\tt{everyone \,\, rates \,\, trying \,\, to \,\, tackle \,\, extensive \,\, eels}$$

$$$$

---

$$$$

## Exercise: Cryptanalysis of the Affine Cipher
Execute a cryptographic attack on the ciphertext linked below, which was encrypted using the affine cipher with an unknown key. 

$$$$

> [Affine ciphertext in 5-block form.](https://drive.google.com/file/d/1TnFPrrE2z0ajSu8z7jFr59hmk2t_F9s5/view?usp=sharing)

> [Affine ciphertext without spaces](https://drive.google.com/file/d/1SdweHDOnIVIN-q1kQeuMtTIpGrbZNPxD/view?usp=sharing).


$$$$

Below is a function called **affine_attack** which executes the attack described above on the given ciphertext. It produces a list of 25 of the most likely plaintexts in descending order of likelihood. It gives an error if the first entry of the guess for the key fails to be invertible modulo 26. You should use this to perform the cryptanalysis. You may also do it by hand.

$$$$

$$ \textrm{affine\_attack(ciphertext)} \quad \longrightarrow \quad \textrm{likely plaintexts}$$

In [27]:
# Below is a Python function called affine_attack() which executes an attack on 
# an arbitrary string of affine ciphertext.

#-------------------------------------------------------------------------------

# Define a function called affine_attack() which accepts a string of ciphertext
# encrypted with the affine cipher and returns the 25 most likely plaintexts
# using the method described in the preceding section.
def affine_attack(ciphertext):

  # Initialize a list where we will store the number of occurrences of each 
  # letter in the ciphertext. The index of each count will correspond to the
  # cryptographic number associated to corresponding letter.
  numLetters = []

  # Define a list containing the alphabet.
  alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
              'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
              'Y', 'Z'] 

  # Invoke the function COUNT_MONO defined previously to count the number of
  # occurrences of each ciphertext letter. Append this count to numLetters.
  for letter in alphabet:
    numLetters.append(count_mono(ciphertext,letter))
  
  # Initalize a list to contain the indices of the five most frequently appearing
  # ciphertext letters.
  frequentLetters = []
  
  # Iterate over the length of frequentLetters.
  for i in range(5):

    # Assign the highest letter count in numLetters to maxVal.
    # On subsequent iterations, this will be the highest letter count not already
    # considered.
    maxVal = max(numLetters)

    # Save the index of the letter corresponding to the count currently saved
    # in maxVal.
    indexOfFrequentLetter = numLetters.index(maxVal)

    # Append the index of this letter to frequentLetters.
    frequentLetters.append(indexOfFrequentLetter)

    # Assign the value -1 to numLetters whereever it was taken from maxVal so
    # we don't get repeated occurrences of the same letter on subsequent
    # iterations.
    numLetters[indexOfFrequentLetter] = -1

  # The outer FOR loop will iterate through the first four most frequent letter indices
  # in frequentLetters (out of a total of five).
  for firstLetterIndex in range(len(frequentLetters)-1):

    # After the outer FOR loop decides what letter will be assigned to 
    # firstLetterIndex, the variable secondLetterIndex is given the value
    # of the following letter in the list frequentLetters.
    secondLetterIndex = firstLetterIndex + 1

    # This while loop will iterate through every element of frequentLetters which
    # appears AFTER the element stored in firstLetterIndex. It will stop when
    # secondLetterIndex has a value exceeding the last index of frequentLetters.
    # (This value happens to be 5 since the last index is 4.)
    while secondLetterIndex < len(frequentLetters):

      # Solving the system of congruences described previously yields this value
      # for a. 
      a = 19*(frequentLetters[firstLetterIndex] - frequentLetters[secondLetterIndex])

      # If a is not invertible modulo 26, the decryption will fail, so we skip 
      # this particular decryption attempt if that is the case.
      if type(inverseMod(a,26)) is int:

        # Solving the system of congruences described previously yields this value
        # for b. 
        b = (frequentLetters[firstLetterIndex] - 4*a) % 26

        # Call the function AFFINE_DECRYPT on the ciphertext and guessed key value.
        print(affine_decrypt(ciphertext,[a,b]))

      # Repeat the code just executed, but swap the letter assignments. 
      # On the first iteration, this will make the assumption that T is the underlying
      # plaintext letter for the most common ciphertext letter and that E
      # corresponds to the second most common ciphertext letter.
      # (Normally, one would expect the opposite to be true.)
      a = 19*(frequentLetters[secondLetterIndex] - frequentLetters[firstLetterIndex])
      if type(inverseMod(a,26)) is int:
        b = frequentLetters[secondLetterIndex] - 4*a
        print(affine_decrypt(ciphertext,[a,b]))
        
      # Increase the value of secondLetterIndex by 1, and pass to the next iteration
      # of the inner WHILE loop.
      secondLetterIndex = secondLetterIndex + 1

  return


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------
# Define the ciphertext.
ciphertext = 'AUZLU BZGYM BKLBY WZHYR KVUBL UAUZD ZAURS RGGWH CBRKT BTUZM ZEHBW ABTAU BTOZM JTBDY WZRKZ AXPZK GMRDA UZMXA BRKXW DZLUX KBLTV UBLUB TAUXA DXAUZ DXABL XWFHX KABAJ YXMAB LHWXM WJZCA ZKTBR KDXJE ZLRKL ZBOZS XTNZK ZMXAZ SEJLR KABKH ZSWRL XWDRA BRKXK SAUXA XWWFH XKABA BZTVU XAZOZ MXAWZ XTAEJ XKXWR NJXKS XLLRD DRSXA BRKDX JEZLR KLZBO ZSXTN ZKZMX AZSXG AZMXW BPZDX KKZML RKTZF HZKAW JAUZM ZDHTA EZLRD YXMXA BOZOZ WRLBA BZTRG BKLMZ XTZXK SSZLM ZXTZS HMBKN THLUN ZKZMX ABRKT VURTZ MZWXA BRKTX MZGBC ZSXKS SZAZM DBKXE WZXKS DXJAU ZMZGR MZYMR EWZDX ABLXW WJEZY MRYRT ZSARE ZGRHK SAUBT YMREW ZDRHM XHAUR MUZMZ TRWOZ TEJAU ZUZWY RGXKR AUZMY MBKLB YWZKR AWZTT ZOBSZ KAVUB LUTHY YRTZT AUXAF HXKAB AJBTB KGBKB AZWJS BOBTB EWZRM AUXAB ADXJD ZKAXW WJXAW ZXTAT RGXML RKABK HXWWJ SBDBK BTUXT XAWXT AEZGR MZBAB TARAX WWJZC ABKNH BTUZS ARXMM BOZXA FHXKA BABZT AUXAD XJEZL XWWZS OXKBT UBKNF HXKAB ABZTR MVUBL UXMZB KGBKB AZWJW BAAWZ XKSWZ TTAUX KXKJX TTBNK XEWZF HXKAB AJRMB ATHYY RTZTA UXAVZ DXJGR MDXKR ABRKK RABKS ZZSRG XETRW HAZEH ARGMZ WXABO ZXKSL RDYXM XABOZ BKGBK BAJAB TXOZM JIHTA ZCLZY ABRKA RAUZD ZAURS RGBKS BOBTB EWZTX TXWTR ARAUZ GRMZB NKBKG BKBAZ TBDXW DZAUR SAUXA AUZJU XOZMZ LRHMT ZXARK LZARB KGBKB AZWJW BAAWZ FHXKA BABZT XKSBK GBKBA ZRMSZ MTXKS NMXSX ABRKT RGAUZ TZKRA MZWXA BOZWJ EHAXE TRWHA ZWJTH LUAUZ JXTTH DZAUZ TZFHX KABAB ZTTBD HWZAT ZDZWV BAURH AXKJL ZMZDR KJXTF HXKAB ABZTA UXAXL AHXWW JXKSR EOBRH TWJZC BTAXK SDXPZ LRDYH AXABR KTVBA UAUZD XLLRM SBKNW JAUZM ZTHWA RGVUB LUDHT AKZZS TEZXT YMZLX MBRHT XTAUZ XETRW HAZZC BTAZK LZRGA UZFHX KABAB ZTAUZ JXTTH DZXKS TRDZW XAZNZ RDZAM BLBXK TUXOZ LXMMB ZSAUZ TZTYZ LHWXA BRKTX ERHAM ZXWXK SXETR WHAZB KGBKB AJTAB WWDHL UGXMA UZMXK SUXOZ MXBTZ SBDXN BKXMJ TJTAZ DTRGB KGBKB AZWJN MZXAX KSBKG BKBAZ WJWBA AWZFH XKABA BZTXK SAUZB MTZOZ MXWRM SZMTX KSYMR YZMAB ZTVUB LUARX WWTRE ZMBKF HBMZM TBKAR DXAUZ DXABL XWAMH AUTDH TALZM AXBKW JXYYZ XMOZM JKRAB RKXWX KSOBT BRKXM JAUZM ZVBWW EZAUZ BKLRK OZKBZ KLZTA UXAVB WWXMB TZBGV ZSRKR AMBNU AWJSB TABKN HBTUE ZAVZZ KXETR WHAZX KSMZW XABOZ BKGBK BAJXE TRWHA ZBKGB KBAJX TTHLU LXKUX MSWJE ZAUZR EIZLA ZBAUZ MRGRH MLRKL ZYABR KTRML XWLHW XABRK TEHAM ZWXAB OZBKG BKBAJ DXJHK SZMXY MRYZM MZNHW XABRK RHMXH AURMR ETZMO ZTAUB TSBTA BKLAB RKTOZ MJTAM BLAWJ XKSBK AMRSH LZTKR KZEHA BKGBK BAZWJ WBAAW ZFHXK ABABZ TAUXA XMZMZ WXABO ZWJTR VUBLU UZXMM BOZTX AEJEZ NBKKB KNVBA UGBKB AZFHX KABAB ZTXKS YMRLZ ZSBKN EJXNM XSHXW XKSKZ LZTTX MJYMR NMZTT RGSBD BKHAB RKUBT LRDYH AXABR KTXWV XJTLR DDZKL ZEJGB KBAZX KSBKA ZWWBN BEWZF HXKAB ABZTX KSAUZ KXAWX TAUZB KFHBM ZTVUX AVBWW EZAUZ MZTHW ABKLZ MAXBK LBMLH DTAXK LZTVU ZKTHL URMTH LUFHX KABAB ZTXMZ SBDBK BTUZS BKBKG BKBAH DAUBT BTXLR KTAXK AYMXL ABLZZ OZKBK LRDDR KXWNZ EMXXK SNZRD ZAMJX KSBTK RDRMZ SZTLZ KSBKN GMRDX NZKZM XWYMR YRTBA BRKAR XYXMA BLHWX MLXTZ VUBLU BTLZM AXBKW JBKLW HSZSB KBAXK SGMRD AUZTZ ZXMWJ YMBKL BYWZT DXKXN ZSVBA UXOXT ASZXW RGTPB WWXKS TXNXL BAJUZ SZSHL ZTUBT DZAUR SRGGW HCBRK TVUBL UBGVZ LRKTB SZMRK WJTRG XMXTU ZUBDT ZWGUX TLXMM BZSBA ARNZA UZMVB AUAUZ XYYWB LXABR KUZUX TDXSZ RGBAZ BAUZM UZMZR MZWTZ VUZMZ SBMZL AWJRM BKSBM ZLAWJ ZCYMZ TTWJR MAXLB AZWJA RAUZD RTALH MBRHT SBTLR OZMBZ TBKXM AXKSK XAHMZ XKSAR AUZTH EWBDZ TAAUZ RMBZT VZDXJ SZTZM OZSWJ ZTAZZ DBAXT AUZNM ZXAZT AVRMP RGNZK BHTXK SXTAU ZKREW ZTAZG GRMAA UXAZO ZMVXT DXSZE JAUZU HDXKD BKSBK SZZSB TDHTA EZRVK ZSAUX ADXKJ HTZGH WBDYM ROZDZ KATXK SKZVX YYWBL XABRK TUXOZ EZZKT BKLZD XSZEJ RAUZM TXKSY MREXE WJVBW WEZTA BWWDX SZZOZ MJSXJ GRMBA BTKRD ZXKZC LZWWZ KLZRG AUBTD ZAURS AUXAB TBTSR HEAWZ TTTAB WWLXY XEWZR GXNMZ XAZMS ZNMZZ RGYZM GZLAB RKXKS VBWWX WVXJT XGGRM SXKBK ZCUXH TABEW ZGHKS RGLHM BRHTD XAAZM ARMZV XMSAU ZYXBK TRGAU ZBKNZ KBRHT XKSBK SHTAM BRHTX KXWJT AXTBX DSZTB MRHTA RDXPZ AUBTX TTXAB TGXLA RMJXT YRTTB EWZZT YZLBX WWJAR AUZOZ MJWZX MKZSX KSBKN ZKBRH TXHAU RMRGA UZSBT LRHMT ZLXWW ZSAUZ XKXWJ TBTVU RTZZD BKZKA AXWZK ATBXL PKRVW ZSNZD JTZWG ARUXO ZXNMZ XAOZK ZMXAB RKGRM BTUXW WUZMZ ZKSZX ORHMA RREOB XAZTR DZRGU BTYMB KLBYX WREIZ LABRK TARAU ZDZAU RSRGG WHCBR KTYXM ABLHW XMWJT HLUXT BUXOZ KRAAR HLUZS HYRKB KDJLR DDZKA VUBLU BTTRR KARGR WWRVU ZAUBK PTRHM XHAUR MUXTK RAYMR LZZSZ SBKXS ZDRKT AMXAB OZXKS TLBZK ABGBL XWDXA AZMBK UBTYM BKLBY VUZMZ UZSZS HLZTA UZDRD ZKARG XMZLA XKNWZ VURTZ TBSZT XMZTH YYRTZ SAREZ OXMBX EWZWB KZTBT UXWWM ZYMZT ZKAAU ZDXAA ZMXKX WJABL XWWJA UHTXN MZZXE WJBAU BKPAR AUZDB KSRGA UZXHA URMWZ ACXKS JEZAV ROXMB XEWZW BKZTR MFHXK ABABZ TVUBL UXASB GGZMZ KAYZM BRSTR GABDZ XLFHB MZSBG GZMZK AOXWH ZTEJG WRVBK NRMBK LMZXT BKNLR KABKH XWWJZ BAUZM ZFHXE WJRMX WBPZB KZFHX EWJAU BTAUZ MZGRM ZBTAU ZBMHW ABDXA ZMXAB RAUZM XABRR GAUZB MDRDZ KATGW HCBRK TRMOZ WRLBA BZTEJ VUBLU CXKSC KLRKA BKHXW WJBKL MZXTZ RMSZL MZXTZ KRVAR XMNHZ GMRDX NZKZM XWAUZ RMZDA RXYXM ABLHW XMLXT ZLRKA XBKZS HKSZM BABTL ZMAXB KWJRK ZRGAU ZDRTA WZNBA BDXAZ XKSWR NBLXW XTVZW WXTRK ZRGAU ZDRTA HTHXW XKSHT ZGHWV XJTRG XMNHB KNBKA UZVUR WZLRD YXTTR GAUZD XAUZD XABLT ARREI ZLAUZ MZAUX AXGAZ MVZUX OZDXS ZCXKS JARTA XKSGR MTRDZ FHXKA BAJVZ XMZKR AXAWB EZMAJ ARDXP ZAUZD KRAUB KNRMK RFHXK ABAJR MOXKB TUBKN FHXKA BABZT BTKRA XKREI ZLABR KXNXB KTAAU ZDZAU RSRGG WHCBR KTEHA XNXBK TAAUZ LRDDR KXKXW JTBTA UBTDZ AURSR KWJXS RYATA UBTVX JRGXM NHBKN XTXLR KTAXK AYMXL ABLZB KAUZO HWNXM XWNZE MXXKS MZGZM TBTAU BAUZM GRMAU ZYMRR GRGBA BGVZU XOZXK ZFHXA BRKXK JURVL RDYRT ZSRGA UZNZK ZMXWK HDEZM TXELB AUXTX WVXJT EZZKA XHNUA AUXAV ZDXJB KAZMY MZAAU ZTZEJ XKJYX MABLH WXMKH DEZMT XAYWZ XTHMZ RMZOZ KEJQZ MRYMR OBSZS AUXAA UZZFH XABRK RMAUZ LRKSB ABRKT RGAUZ FHZTA BRKSR KRAZC YMZTT WJMZF HBMZA UZLRK AMXMJ GRMNZ KZMXW KHDEZ MTXTT HLUDX JTAXK SGRMX KJSZG BKBAZ KHDEZ MTBKA UZVUR WZKHD ZMBLX WTLXW ZVUBL UTLXW ZBAUB KPDXJ EZAUH TLRDD RSBRH TWJMZ YMZTZ KAZSK ZNXAB OZAUM ZZKZN XABOZ AVRKZ NXABO ZRKZQ ZMRRK ZAVRA UMZZZ ALVUZ MZXWW YRTTB EWZGM XLABR KXWKH DEZMT BKAZM DZSBX AZARA UZTZU ZMZZC YMZTT ZSXMZ AREZL RKLZB OZSXT BKAZM YRWXA ZSEHA BKAUB TTLXW ZAUZA ZMDQZ MRBTX TDHLU XAZMD RMKHD EZMXT XKJRA UZMXK SUXTB ATXKX WRNRH TYMRY ZMABZ TBKLR DDRKV BAUAU ZMZTA VZXMZ WBPZV BTZAR WSAUX AVZDX JKRAN BOZTH LUOXW HZTAR NZKZM XWTJD ERWTX GAZMV XMSTX TAUZJ LRHWS KRAMZ LZBOZ XAGBM TAVUB LUBGX SDBAA ZSBTB AUBKP KRAUB KNARA UZYMZ TZKAY HMYRT ZCXVY'

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Call affine_attack() on the given ciphertext.
affine_attack(ciphertext)

thech iefpr incip leupo nwhic hthem ethod offlu xions isher ebuil tisth isver ysimp leone taken fromt herat ional mecha nicsw hichi sthat mathe matic alqua ntity parti cular lyext ensio nmayb econc eived asgen erate dbyco ntinu edloc almot ionan dthat allqu antit ieswh ateve ratle astby analo gyand accom modat ionma ybeco nceiv edasg enera tedaf teral ikema nnerc onseq uentl yther emust becom parat iveve locit iesof incre asean ddecr eased uring suchg enera tions whose relat ionsa refix edand deter minab leand mayth erefo repro blema tical lybep ropos edtob efoun dthis probl emour autho rhere solve sbyth ehelp ofano therp rinci pleno tless evide ntwhi chsup poses thatq uanti tyisi nfini telyd ivisi bleor thati tmaym ental lyatl easts ofarc ontin ually dimin ishas atlas tbefo reiti stota llyex tingu ished toarr iveat quant ities thatm aybec alled vanis hingq uanti tieso rwhic harei nfini telyl ittle andle sstha nanya ssign ableq uanti tyori tsupp osest hatwe mayfo rmano tionn otind eedo

---
# The Monoalphabetic Substitution Cipher

The monoalphabetic substitution cipher - which, for brevity, we will just call the substitution cipher - is an even further generalization of the ciphers we have seen so far. In fact, both the shift and affine ciphers are special cases of the substitution cipher.

You might be bracing yourself for some wild mathematics to set up our description of this cryptosystem, but relax - this is, in some sense, much easier to work with than the affine cipher (to cryptanalyze it is a different story). We will describe this cryptosystem mostly in plain English because we lack both the mathematical language to describe it precisely and the time to cover the former.

Seeing an example key for the substitution cipher will tell you everything you need to know. One way to view this is that we write the alphabet down (in alphabetical order) to represent plaintext. Below that, we write down the alphabet one more time, but out of order. Such a "mixing up" of an ordered list is called a **permutation** of that list.

$$$$ 

Plaintext | a | b | c | d | e | f | g | h | i | j | k | l | m | n | o | p | q | r | s | t | u | v | w | x | y | z
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- 
Ciphertext | J | P | B | F | I | C | E | Q | K | W | D | U | X | H | L | O | N | V | S | Z | T | A | Y | R | M | G

$$$$

To encrypt a message, you find each plaintext letter in the top row and write down the letter appearing below it. Decryption works the same, but you search for the ciphertext letter in the bottom row and write down the letter above it.

---
$$$$
## Example: Encryption with the Monoalphabetic Substitution Cipher
Suppose that Alice wants to send the message

$$\tt{Take \,\, your \,\, protein \,\, pills \,\, and \,\, put \,\, your \,\, helmet \,\, on.}$$

She converts the message to 5-block form.

$$\tt{takey \,\, ourpr  \,\, otein  \,\, pills  \,\, andpu  \,\, tyour  \,\, helme \,\, ton}$$

Using the example key we wrote down previously, she begins converting each plaintext letter to ciphertext.

$$\begin{align*}
e({\tt{t}}) &= {\tt{Z}} & e({\tt{a}}) &= {\tt{J}} & e({\tt{k}}) &= {\tt{D}} & e({\tt{e}}) &= {\tt{I}} \quad ... \textrm{etc.}
\end{align*}$$

After converting the plaintext to ciphertext, Alice chooses random letters to append to the end of the message as padding. The complete ciphertext is

$$\tt{ZJDIM \,\, LTVOV \,\, LZIKH \,\, OKUUS \,\, JHFOT \,\, ZMLTV \,\, QIUXI \,\, ZLHOF}$$

Finally, Alice transmits the ciphertext to Bob.

In preparation for our next exercise, we present a function which will automatically generate a random monoalphametic substitution cipher key. Since Python lists are ordered, we can generate and interpret a substitution cipher key as follows.

1. Start with a list consisting of a ciphertext alphabet in numerical.
2. Randomly permute the list, i.e. shuffle it around so that alphabetic order is not preserved.
3. The **indices** of this list correspond to a plaintext letter. In other words, index 0 corresponds to plaintext $\tt{a}$, index 1 to $\tt{b}$, index 2 to $\tt{c}$, and so forth.
4. The value of each entry of the list corresponds to the ciphertext letter the plaintext letter associated to that index maps to. 

For example, if the first few entries of our key look like

$$[13,\,\,\,21,\,\,\, 3,\,\,\, 23,\,\,\,  24,\,\,\,\ldots]$$

Then plaintext $\tt{a}$ maps to ciphertext $\tt{N}$, plaintext $\tt{b}$ maps to ciphertext $\tt{V}$, plaintext $\tt{c}$ maps to ciphertext $\tt{D}$, etc.

Our key-generating function is presented below.

In [28]:
# Define a function substitution_key() which has no arguments and returns
# a random monoalphabetic substitution cipher key.
def substitution_key():

  # Import the Python package random.
  import random

  # Define a list to contain the ciphertext alphabet.
  key = list(range(26))

  # Use the function random.shuffle() to permute the numerical ciphertext alphabet.
  random.shuffle(key)
  
  # Return the key.
  return key

# Call substitution_key() and print the output.
print(substitution_key())

[0, 18, 15, 16, 19, 6, 25, 24, 22, 11, 2, 13, 4, 7, 21, 1, 8, 12, 14, 23, 3, 20, 17, 5, 9, 10]


---
$$$$
## Python: Encryption with the Monoalphabetic Substitution Cipher
Below is a Python function called **substitution_encrypt()** which accepts a string of plaintext and a substitution cipher key and outputs text encrypted with the substitution cipher.

$$$$

$$\begin{align*} &\textrm{substitution\_encrypt(plaintext, key)} \quad \longrightarrow \quad \textrm{ciphertext} \end{align*}$$

Test it on the sample plaintext with the key we used given above.

In [29]:
# Below is a function which will encrypt an arbitrary string of plaintext with
# the substitution cipher.

#-------------------------------------------------------------------------------

# Define a function substitution_encrypt() which accepts plaintext and a 
# substitution cipher key as arguments and returns ciphertext encrypted with
# the monoalphabetic substitution cipher.
def substitution_encrypt(plaintext, key):

  # Convert alphabetic plaintext to numerical plaintext using plain_to_num.().
  plaintextNum = plain_to_num(plaintext)

  # Initialize an empty list to store numerical ciphertext.
  ciphertextNum = []

  # Iterate over the length of plaintextNum.
  for i in range(len(plaintextNum)):

    # Use the key to convert each plaintext letter to ciphertext and append it
    # to ciphertextNum.
    ciphertextNum.append(key[plaintextNum[i]])

  # Convert the numerical ciphertext to alphabetic ciphertet using 
  # num_to_cipher.().
  ciphertext = num_to_cipher(ciphertextNum)

  # Return the alphabetic ciphertext.
  return ciphertext


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Define a key.
key = [9,15,1,5,8,2,4,16,10,22,3,20,23,7,11,14,13,21,18,25,19,0,24,17,12,6]

# Define the plaintext.
plaintext = 'Take your protein pills and put your helmet on.'

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Call substitution_encrypt() on the plaintext and key and print the result.
print(substitution_encrypt(plaintext,key))

ZJDIM LTVOV LZIKH OKUUS JHFOT ZMLTV QIUXI ZLHRP


---
$$$$
## Example: Decryption with the Monoalphabetic Substitution Cipher
After Bob receives the ciphertext from Alice, he can begin decrypting it. As with encryption, decryption by hand is straightforward. Bob uses the original key table as he begins this process.

$$$$ 

Plaintext | a | b | c | d | e | f | g | h | i | j | k | l | m | n | o | p | q | r | s | t | u | v | w | x | y | z
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- 
Ciphertext | J | P | B | F | I | C | E | Q | K | W | D | U | X | H | L | O | N | V | S | Z | T | A | Y | R | M | G

$$$$

Recall that the ciphertext was

$$\tt{ZJDIM \,\, LTVOV \,\, LZIKH \,\, OKUUS \,\, JHFOT \,\, ZMLTV \,\, QIUXI \,\, ZLHOF}$$

Bob searches the bottom row of the table for each ciphertext letter and writes down the corresponding plaintext letter directly above it. The first few decryptions are:

$$\begin{align*}
d({\tt{Z}}) &= {\tt{t}} & d({\tt{J}}) &= {\tt{a}} & d({\tt{D}}) &= {\tt{k}} & d({\tt{I}}) &= {\tt{e}} \quad \ldots \textrm{etc.}
\end{align*}$$

The full plaintext is

$$\tt{takey \,\, ourpr \,\, otein \,\, pills \,\, andpu \,\, tyour \,\, helme \,\, tonpd}$$ 

After introducing punctuation, capitalization, and removing padding, Bob obtains the message.

$$\tt{Take \,\, your\,\,protein\,\,pills\,\,and\,\,put\,\,your\,\,helmet\,\,on.}$$

---
$$$$
## Python: Decryption with the Monoalphabetic Substitution Cipher
Below is a Python function called **substitution_decrypt** which accepts a string of ciphertext and a substitution cipher key and outputs plaintext.

$$$$

$$\begin{align*} &\textrm{substitution\_decrypt(ciphertext, key)} \quad \longrightarrow \quad \textrm{plaintext} \end{align*}$$

Test it on the sample ciphertext with the key we used given above.

In [30]:
# Below is a Python function which will decrypt an arbitrary string of 
# ciphertext which was encrypted using a substitution cipher. 

#-------------------------------------------------------------------------------

# Define a function substitution_decrypt() which accepts ciphertext and a 
# substitution cipher key as arguments and returns plaintext.
def substitution_decrypt(ciphertext, key):

  # Convert alphabetic ciphertext to numerical ciphertext using cipher_to_num.().
  ciphertextNum = cipher_to_num(ciphertext)

  # Initialize an empty list to store numerical ciphertext.
  plaintextNum = []

  # Iterate over the length of plaintextNum.
  for i in range(len(ciphertextNum)):

    # Use the key to convert each ciphertext letter to plaintext and append it
    # to plaintextNum.
    plaintextNum.append(key.index(ciphertextNum[i]))

  # Convert the numerical plaintext to alphabetic plaintext using 
  # num_to_plain.().
  plaintext = num_to_plain(plaintextNum)

  # Return the alphabetic plaintext.
  return plaintext


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Define a key.
key = [9,15,1,5,8,2,4,16,10,22,3,20,23,7,11,14,13,21,18,25,19,0,24,17,12,6]

# Define the plaintext.
ciphertext = 'ZJDIM LTVOV LZIKH OKUUS JHFOT ZMLTV QIUXI ZLHOF'

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Call substitution_decrypt() on the ciphertext and key and print the result.
print(substitution_decrypt(ciphertext,key))

takey ourpr otein pills andpu tyour helme tonpd


---
$$$$
## Monoalphabetic Substitution Cipher: Order of the Key Space
Now we consider the order of the key space for the monoalphabetic subsitution cipher. 

Recall that a key for the substitution cipher is a shuffled, ordered list containing the alphabet. So to count keys, we need to find a way to count the number of unique ways we can shuffle the alphabet.

Let's first consider a few simpler examples which might lead us to a solution. 

$$$$

*Example.* $\quad$ How many ways are there to order two objects $A$ and $B$? We can write down these unique orderings explicitly.

$$AB \qquad BA$$

So there are two ways to order two objects.

$$$$

*Example.* $\quad$ How many ways are there to order three objects $A, B$, and $C$? 

$$ABC \qquad ACB \qquad BAC$$
$$BCA \qquad CAB \qquad CBA$$

There are six ways to order three objects. 

$$$$

*Example.* $\quad$ How many ways are there to order four objects $A, B, C$, and $D$? 

We could write down all the ways of doing this as in previous examples, but there's a simpler way to look at this. For a given ordering of $A, B$, and $C$, say $ABC$, there are four places we can insert $D$.

$$DABC \qquad ADBC \qquad ABDC \qquad ABCD$$

Since there are six ways of ordering $A, B$, and $C$, there must be $4 \cdot 6 = 24$ unique orderings of all four objects. We could have used the same argument when going from two to three objects; we'd have multiplied the number of places we could have put $C$ in and around $A$ and $B$, which is 3, by the number of unique orderings of $A$ and $B$, which is 2. Hence there are $3\cdot 2 = 6$ orderings of three objects.

$$$$

It turns out that this pattern holds no matter how many objects you are considering.

> *Definition 17.* $\quad$ Let $n$ be a positive integer. Then the number $n!$, read "$n$ factorial," is the number of ways of ordering $n$ unique objects, and

$$ n! = n\cdot(n-1) \cdot(n-2) \cdot (n-3) \cdots 3 \cdot 2 \cdot 1$$

$$$$

*Question 1:* $\quad$ How many unique ways are there of ordering the alphabet? Hence, what is the order of the key space for the substitution cipher? Use Wolfram Alpha to perform this computation.

*Question 2:* $\quad$ Suppose you are able to check one substitution cipher key per second in the course of performing a brute force attack. How long would it take you to complete a brute force attack on ciphertext encrypted with a substitution cipher? Assume you need to check every key. Express your answer in years and in multiples of the age of the universe (13.8 billion years). [Wolfram Alpha](https://www.wolframalpha.com/) is suitable for this calculation.

---
$$$$
## Cryptanalysis of the Monoalphabetic Substitution Cipher
Despite the seemingly overwhelming odds stacked against you, it is possible to break the monoalphabetic substitution cipher *by hand*. Someone who is unpracticed in cryptanalysis could break a sufficiently long sample of ciphertext within several hours, while someone adept at breaking substitution ciphers might be able to do it within a half hour. As you will see, a computer is capable of doing it very quickly.

One of the techniques we will use is something you have already dipped your toes into: **letter frequency analysis**. We will extend this generally to $\mathbf{n}$**-gram analysis**, where instead of counting the frequencies of single letters, you count the frequencies of so-called **bigrams**, which are sequences of two consecutive letters, **trigrams**, which are sequences of three consecutive letters, and so on. (Note: single letters are also called **monograms**.) 

It is typically enough to break a substitution cipher to only consider monograms and bigrams, but trigrams and quadgrams are useful if these are not enough.

For this reason, we will need functions which are capable of counting monograms, bigrams, trigrams, and quadgrams. (We already have one which counts monograms.)

In the three code cells below are functions called **count_bi()**, **count_tri()**, and **count_quad()** which return the number of times a given bigram, trigram, or quadgram appears in the supplied text.




In [31]:
# Define a function called count_bi() which accepts a string of text and a 
# bigram and returns the number of times the bigram appears in the text.
def count_bi(text, bigram):

  # Initialize a list to store the alphabet in alphabetical order.
  alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
              'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
              'y', 'z']

  # Initialize a variable to store the number of times the chosen bigram appears
  # in the given text.
  count = 0 

  # Initialize an empty list which will store the supplied text with all spaces,
  # punctuation, and other miscellaneous characters removed.
  formattedText = []

  # This for loop will iterate over the supplied text string and append all
  # of its alphabetic characters to formattedText in lowercase.
  for i in range(len(text)):
    if text[i].lower() in alphabet:
      formattedText.append(text[i].lower())

  # This for loop will iterate over the length of formattedText and count the
  # number of times the chosen bigram appears.
  for i in range(len(formattedText) - 1):
    textBigram = ''.join(formattedText[i:(i+2)])
    if textBigram == bigram:
      count = count + 1

  # The function count_bi() returns the number of occurrences of the given
  # bigram in the supplied text string.
  return count

In [32]:
# Define a function called count_tri() which accepts a string of text and a 
# trigram and returns the number of times the trigram appears in the text.
def count_tri(text, trigram):

  # Initialize a list to store the alphabet in alphabetical order.
  alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
              'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
              'y', 'z']

  # Initialize a variable to store the number of times the chosen trigram appears
  # in the given text.
  count = 0 

  # Initialize an empty list which will store the supplied text with all spaces,
  # punctuation, and other miscellaneous characters removed.
  formattedText = []

  # This for loop will iterate over the supplied text string and append all
  # of its alphabetic characters to formattedText in lowercase.
  for i in range(len(text)):
    if text[i].lower() in alphabet:
      formattedText.append(text[i].lower())

  # This for loop will iterate over the length of formattedText and count the
  # number of times the chosen trigram appears.
  for i in range(len(formattedText) - 2):
    textTrigram = ''.join(formattedText[i:(i+3)])
    if textTrigram == trigram:
      count = count + 1

  # The function count_tri() returns the number of occurrences of the given
  # trigram in the supplied text string.
  return count

In [33]:
# Define a function called count_quad() which accepts a string of text and a 
# quadgram and returns the number of times the quadgram appears in the text.
def count_quad(text, quadgram):

  # Initialize a variable to store the number of times the chosen quadgram 
  # appears in the given text.
  alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
              'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
              'y', 'z']

  # Initialize an empty list which will store the supplied text with all spaces,
  # punctuation, and other miscellaneous characters removed.
  count = 0 

  # Initialize an empty list which will store the supplied text with all spaces,
  # punctuation, and other miscellaneous characters removed.
  formattedText = []

  # This for loop will iterate over the supplied text string and append all
  # of its alphabetic characters to formattedText in lowercase.
  for i in range(len(text)):
    if text[i].lower() in alphabet:
      formattedText.append(text[i].lower())

  # This for loop will iterate over the length of formattedText and count the
  # number of times the chosen quadgram appears.
  for i in range(len(formattedText) - 3):
    textQuadgram = ''.join(formattedText[i:(i+4)])
    if textQuadgram == quadgram:
      count = count + 1

  # The function count_quad() returns the number of occurrences of the given
  # quadgram in the supplied text string.
  return count

---
$$$$
### Discussion of Technique (Optional)

The technique for cryptanalyzing the monoalphabetic substitution cipher by hand can be messy as it relies heavily on human intuition. Consequently, it is not immediately obvious how to instruct a computer to carry out an attack. We will need to make use of a class of algorithm called a [genetic algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm) to do so.

A genetic algorithm is a method of selecting an optimal solution to a problem from a collection of candidate solutions. In our case, the problem is to find a substitution cipher key which decrypts (or partially decrypts) the supplied ciphertext. The candidate solutions are the set of all possible keys, and an optimal solution is a key which fully decrypts the ciphertext.

*Question: Can you think of a situation in which there exists more than one key which fully decrypts ciphertext encrypted with a monoalphabetic substitution cipher?*

Genetic algorithms rely on the existence and specification of a **fitness function** $f: S \rightarrow \mathbb{R}$ which maps each candidate solution to a real number. The fitness function must do this in such a way that a candidate solution $s_1$ is "better than" another candidate solution $s_2$ whenever $f(s_1) > f(s_2)$. Assuming we can find a function satisfying this property, the genetic algorithm proceeds as follows:

1.   Randomly select a candidate solution $P$ from the set of all possible candidate solutions. $P$ is called the "parent."
2.   Compute $f(P)$, the parent's fitness.
3.   Make a copy of the parent solution and alter it slightly; call this slightly-altered solution $C$ the "child."
4.   Compute $f(C)$, the child's fitness.
5.   If $f(C) > f(P)$, resassign $C$ to $P$. In other words, the child becomes the parent. If instead $f(C) \leq f(P)$, do nothing.
6.   If the desired number of iterations has been reached, stop. Otherwise, go back to step 3. 

If the fitness function $f$ has been chosen appropriately, the parent solution will (hopefully) converge to an optimal solution given a sufficient number of iterations. So how do we apply this to cryptanalysis of the monoalphabetic substitution cipher? 

We first obtain lists of $\mathbf{n}$-gram frequencies for desired values of $\mathbf{n}$ from a very large sample of English text $\mathcal{S}$. In our case, we'll use $\mathbf{n} = 1, 2, 3, 4$, which are called monograms, bigrams, trigrams, and quadgrams, respectively. Let $k$ be a monoalphabetic substitution cipher key, let $d_k$ be the decryption function associated to this key, and let $c$ be the given ciphertext. Then $d_k(c)$ is the image of the ciphertext under the decryption function associated to the candidate key. Let $x$ be a particular $\mathbf{n}$-gram. Define

$$
\lambda_{\mathbf{n}}(x) = \frac{\textrm{number of times $x$ appears in $\mathcal{S}$}}{\textrm{total number of $\mathbf{n}$-grams in $\mathcal{S}$}}
$$

In other words, $\lambda_{\mathbf{n}}(x)$ is the proportion of $\mathbf{n}$-grams in $\mathcal{S}$ which are exactly $x$. Then for each $\mathbf{n}$ and each candidate key $k$, we define a fitness function

$$
f_{\mathbf{n}}(k) = -\sum_{x \in d_k(c)}\begin{cases} \log_{10}\left[\lambda_{\mathbf{n}}(x)\right] & \textrm{if $x \in \mathcal{S}$} \\\\
-10 & \textrm{otherwise}
\end{cases}
$$

where each $x$ is taken to be an $\mathbf{n}$-gram appearing in $d_k(c)$. The result is that $\mathbf{n}$-grams in $d_k(c)$ which appear frequently in English text will contribute almost nothing to the value of $f_{\mathbf{n}}(k)$, but those that appear infrequently in English text will contribute a comparatively large negative value to it. The less the value of the fitness function is, the less likely it is to be plaintext. If $d_k(c)$ is English text, it should have a value very close to zero (but will still be negative).

The reason we have taken the logarithm of $\lambda(x)$ in the sum is that very different values of $\lambda$ could result in massive floating point arithmetic errors in our algorithm, leading to inaccurate results. For example, a computer with 8-digit precision might attempt to compute $10^{10} + 10$ and erroneously return $10^{10}$, but $\log_{10}(10^{10}) + \log_{10}(10) = 10 + 1 = 11$, which is exact.

The sum itself is multiplied by $-1$ to ensure that candidate plaintexts which more closely resemble written English in terms of $\mathbf{n}$-gram frequency have higher fitnesses than those that do not. 

---
$$$$
### Fitness Function and $\mathbf{n}$-gram Data

We present a function below called **logFitness()** which accepts a string of text and another string specifying the $\mathbf{n}$-gram type and returns the fitness of the text with respect to that $\mathbf{n}$-gram type.

*Note: There are some additional cells titled, for example, "English Monogram Frequencies." You'll need to performed the following procedure and then run these for **logFitness()** to work.*



1.   Navigate to [this folder](https://drive.google.com/drive/folders/1yRosII1zLh0nY29f-7Orj14EPxRHN8gW?usp=sharing) and download the four text files it contains.
2.   On the extreme left side of your browser window there is a small icon that looks like a folder. Click on it.
3.   Just to the right of the icon there is small file directory. You should see a folder with the text "sample_data" next to it. 
4.   Select all of the text files you downloaded in step 1 and drag them into the empty white area underneath the folder mentioned in step 3. The files should upload and you should see them appear underneath the file called "sample_data."
5.   The four code cells below should now allow you to run them without any errors. If they do not, tell your research project leader.

In [34]:
#@title English Monogram Frequencies
import os
monograms = "/content/monograms.txt"
#mono = open('/Users/Dan/Desktop/PC/WSP/WSP 2020/Cryptography (2020)/Letter Frequencies/monograms.txt', 'r')
mono = open(monograms, "r")
engMonogramRaw = mono.readlines()

engMonogramFrequencies = []
for line in range(len(engMonogramRaw)):
  engMonogramFrequencies.append([])

for line in range(len(engMonogramRaw)):
  engMonogramFrequencies[line].append(engMonogramRaw[line][0:1])
  engMonogramFrequencies[line].append(int(engMonogramRaw[line][2:]))

mono.close()

# Initialize a list to contain the alphabet.
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
            'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
            'y', 'z']

# Initialize a list to contain English monograms in alphabetical order.
monograms = []

# Initialize a list to contain English monogram raw counts from source data.
engMonogramData = []

# Append the English monogram raw counts from source data to engMonogramData
# and the corresponding monograms to 'monograms'.
for i in range(len(engMonogramFrequencies)):
  monograms.append(engMonogramFrequencies[i][0].upper())
  engMonogramData.append(engMonogramFrequencies[i][1])

FileNotFoundError: ignored

In [ ]:
#@title English Bigram Frequencies
import os
bigrams = "/content/bigrams.txt"
bi = open(bigrams, "r")
#bi = open('/Users/Dan/Desktop/PC/WSP/WSP 2020/Cryptography (2020)/Letter Frequencies/bigrams.txt', 'r')
engBigramRaw = bi.readlines()

engBigramFrequencies = []
for line in range(len(engBigramRaw)):
  engBigramFrequencies.append([])

for line in range(len(engBigramRaw)):
  engBigramFrequencies[line].append(engBigramRaw[line][0:2])
  engBigramFrequencies[line].append(int(engBigramRaw[line][3:]))

bi.close()

# Store bigrams in descending order of frequency.
bigrams = []
for index in range(len(engBigramFrequencies)):
  bigrams.append(engBigramFrequencies[index][0].upper())

# Compute the number of bigrams evaluated in the imported data.
sumOfBigrams = 0
for index in range(len(engBigramFrequencies)):
  sumOfBigrams = sumOfBigrams + engBigramFrequencies[index][1]

# Initialize a list to store English bigram proportions. Compute and store them.
engBigramProportions = []
for index in range(len(engBigramFrequencies)):
  proportion = float(engBigramFrequencies[index][1])/float(sumOfBigrams)
  engBigramProportions.append(proportion)

In [ ]:
#@title English Trigram Frequencies
import os
trigrams = "/content/trigrams.txt"
tri = open(trigrams, "r")
#tri = open('/Users/Dan/Desktop/PC/WSP/WSP 2020/Cryptography (2020)/Letter Frequencies/trigrams.txt', 'r')
engTrigramRaw = tri.readlines()

engTrigramFrequencies = []
for line in range(len(engTrigramRaw)):
  engTrigramFrequencies.append([])

for line in range(len(engTrigramRaw)):
  engTrigramFrequencies[line].append(engTrigramRaw[line][0:3])
  engTrigramFrequencies[line].append(int(engTrigramRaw[line][4:]))

tri.close()

trigrams = []
for index in range(len(engTrigramFrequencies)):
  trigrams.append(engTrigramFrequencies[index][0].upper())

sumOfTrigrams = 0
for index in range(len(engTrigramFrequencies)):
  sumOfTrigrams = sumOfTrigrams + engTrigramFrequencies[index][1]

engTrigramProportions = []
for index in range(len(engTrigramFrequencies)):
  proportion = float(engTrigramFrequencies[index][1])/float(sumOfTrigrams)
  engTrigramProportions.append(proportion)

In [ ]:
#@title English Quadgram Frequencies
import os
quadgrams = "/content/quadgrams.txt"
quad = open(quadgrams, "r")
#quad = open('/Users/Dan/Desktop/PC/WSP/WSP 2020/Cryptography (2020)/Letter Frequencies/quadgrams.txt', 'r')
engQuadgramRaw = quad.readlines()

engQuadgramFrequencies = []
for line in range(len(engQuadgramRaw)):
  engQuadgramFrequencies.append([])

for line in range(len(engQuadgramRaw)):
  engQuadgramFrequencies[line].append(engQuadgramRaw[line][0:4])
  engQuadgramFrequencies[line].append(int(engQuadgramRaw[line][5:]))

quad.close()

quadgrams = []
for index in range(len(engQuadgramFrequencies)):
  quadgrams.append(engQuadgramFrequencies[index][0].upper())

sumOfQuadgrams = 0
for index in range(len(engQuadgramFrequencies)):
  sumOfQuadgrams = sumOfQuadgrams + engQuadgramFrequencies[index][1]

engQuadgramProportions = []
for index in range(len(engQuadgramFrequencies)):
  proportion = float(engQuadgramFrequencies[index][1])/float(sumOfQuadgrams)
  engQuadgramProportions.append(proportion)

In [ ]:
# Below is a function called logFitness() which accepts a string of text and 
# a string specifying the n-gram type (monogram, bigram, trigram, or quadgram) 
# and returns the n-gram fitness of the text.

#-------------------------------------------------------------------------------

# Import the function log10() from the Python package math.
from math import log10

# Define the function logFitness.
def logFitness(text, fitnessType):

  # This section of code removes spaces and punctuation from the input text
  # and capitalizes every letter.
  tempText = text[:].lower()
  returnText = []
  for i in range(len(tempText)):
    if tempText[i].lower() in alphabet:
      returnText.append(tempText[i].upper())
  text = ''.join(returnText)

  #----------------------------MONOGRAMS----------------------------------------

  # If the selected fitness type is monogram, this section of code will run.
  if fitnessType == 'monogram':

    # Compute the text monogram fitness and return it.
    monoFitnesses = []

    for i in range(len(text)):
      currMonogram = text[i]
      if currMonogram in monograms:
        currMonogramIndex = monograms.index(currMonogram)
        if engMonogramProportions[currMonogramIndex] == float(0):
          monoFitnesses.append(float(-10))
        else:
          monoFitnesses.append(log10(engMonogramProportions[currMonogramIndex]))
    monoFitness = sum(monoFitnesses)

    return monoFitness

  #----------------------------BIGRAMS------------------------------------------

  # If the selected fitness type is bigram, this section of code will run.
  if fitnessType == 'bigram':

    # Compute the text bigram fitness and return it.
    biFitnesses = []

    for i in range(len(text)-1):
      currBigram = text[i:(i+2)]
      if currBigram in bigrams:
        currBigramIndex = bigrams.index(currBigram)
        if engBigramProportions[currBigramIndex] == float(0):
          biFitnesses.append(float(-10))
        else:
          biFitnesses.append(log10(engBigramProportions[currBigramIndex]))
    biFitness = sum(biFitnesses)

    return biFitness

  #----------------------------TRIGRAMS-----------------------------------------

  # If the selected fitness type is trigram, this section of code will run.
  if fitnessType == 'trigram':

    # Compute the text trigram fitness and return it.
    triFitnesses = []

    for i in range(len(text)-2):
      currTrigram = text[i:(i+3)]
      if currTrigram in trigrams:
        currTrigramIndex = trigrams.index(currTrigram)
        if engTrigramProportions[currTrigramIndex] == float(0):
          triFitnesses.append(float(-10))
        else:
          triFitnesses.append(log10(engTrigramProportions[currTrigramIndex]))
    triFitness = sum(triFitnesses)

    return triFitness

  #----------------------------QUADGRAMS----------------------------------------

  # If the selected fitness type is quadgram, this section of code will run.
  if fitnessType == 'quadgram':

    # Compute the text trigram fitness and return it.
    quadFitnesses = []

    for i in range(len(text)-3):
      currQuadgram = text[i:(i+4)]
      if currQuadgram in quadgrams:
        currQuadgramIndex = quadgrams.index(currQuadgram)
        if engQuadgramProportions[currQuadgramIndex] == float(0):
          quadFitnesses.append(float(-10))
        else:
          quadFitnesses.append(log10(engQuadgramProportions[currQuadgramIndex]))
    quadFitness = sum(quadFitnesses)

    return quadFitness


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Specify the string of text whose fitness is to be evaluated.
text = '''I'm going back some day
          Come what may
          To Blue Bayou
          Where the folks are fun
          And the world is mine
          On Blue Bayou
          Where those fishing boats
          With their sails afloat
          If I could only see
          That familiar sunrise
          Through sleepy eyes
          How happy I'd be'''

# Specify the n-gram type to compute fitness for.
fitnessType = 'bigram'

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Call logFitness() on the given text and n-gram fitness type.
logFitness(text, fitnessType)

---
$$$$ 
## Exercise: Cryptanalysis of the Monoalphabetic Substitution Cipher
We present a function called **substitution_attack** which executes the attack described above on the given ciphertext.

$$$$

$$ \textrm{substitution\_attack(ciphertext)} \quad \longrightarrow \quad \textrm{likely plaintext}$$

$$$$

Use the function substitution_attack to attack the following ciphertexts which were encrypted with the monoalphabetic substitution cipher with an unknown key. The keys used for each ciphertext are different. 

$$$$

> [Substitution ciphertext in 5-block form (short)](https://drive.google.com/file/d/1Qq26IMrdgPJTLv2ImyN3scJ9iHyxJCEA/view?usp=sharing)

> [Substitution ciphertext without spaces (short)](https://drive.google.com/file/d/1HOpjMaoC0S8t4rXFV1BtdXduypixGzmc/view?usp=sharing).

> [Substitution ciphertext in 5-block form (long)](https://drive.google.com/file/d/1jqoAY-IwPuC_YG8JgdXMMgaK-Mq-2eOm/view?usp=sharing)

> [Substitution ciphertext without spaces (long)](https://drive.google.com/file/d/1iBG6BPWgsOxE9XGBEdOEyDVAQZIgf0BF/view?usp=sharing)

$$$$

**Experiment with different numbers of bigram, trigram, and quadgram iterations on the ciphertexts given above. How does the length of the ciphertext affect the attack in accuracy and duration? Is it true that a longer text sample leads to a more accurate and faster attack?**

In [ ]:
# Define our attack function substitution_attack() which accepts a string of 
# ciphertext encrypted with a monoalphabetic substitution cipher and outputs
# likely plaintext.
#
# The arguments of this function are as follows:
#
#     1. CIPHERTEXT       --- Ciphertext to be attacked
#     2. FLAG             --- If TRUE, remaining arguments are ignored and
#                             attacker is prompted to input desired iterations
#                             for each n-gram.
#     3. NUMBI            --- Desired number of bigram iterations.
#     4. NUMTRI           --- Desired number of trigram iterations.
#     5. NUMQUAD          --- Desired number of quadgram iterations.

def substitution_attack(ciphertext, flag, numBi, numTri, numQuad):

  # Import the Python package RANDOM and the function log10() from MATH.
  import random
  from math import log10
  import time

  # Start a stopwatch to time execution of this function.
  start = time.time()

  # Convert the alphabetic ciphertext to numerical ciphertext. 
  # We only do this so we can get an accurate count of the total number of
  # letters in the ciphertext.
  ciphertextNum = cipher_to_num(ciphertext)

  # Store the total number of letters in the ciphertext to the variable totalMono.
  totalMono = len(ciphertextNum)

  # Store the total number of bigrams in the ciphertext to the variable 
  # totalBigrams.
  totalBigrams = len(ciphertextNum) - 1

  # Store the total number of trigrams in the ciphertext to the variable 
  # totalTrigrams.
  totalTrigrams = len(ciphertextNum) - 2

  # Store the total number of quadgrams in the ciphertext to the variable 
  # totalQuadgrams.
  totalQuadgrams = len(ciphertextNum) - 3


  #_____________________________________________________________________________
  #
  #                                 MONOGRAMS
  #_____________________________________________________________________________


  # Initialize a list to store ciphertext monogram frequencies.
  cipherMonogramData = []

  # Record the ciphertext monogram frequencies.
  for i in range(len(alphabet)):
    count = count_mono(ciphertext,alphabet[i])
    cipherMonogramData.append(count)
  
  # Initialize a list to store ciphertext monograms in descending order of
  # frequency.
  orderedCipherMonograms = []

  # Run a FOR loop to store ciphertext monograms in descending order of
  # frequency in orderedCipherMonograms.
  for i in range(len(alphabet)):
    currMaxCount = max(cipherMonogramData)
    indexOfCurrMaxCount = cipherMonogramData.index(currMaxCount)
    cipherLetter = alphabet[indexOfCurrMaxCount]
    orderedCipherMonograms.append(cipherLetter)
    cipherMonogramData[indexOfCurrMaxCount] = -1

  # Initialize a list to store our guess for the key after applying
  # the letter frequency analysis technique.
  key = []

  # Append the required key values resulting from letter frequency analysis.
  for i in range(len(alphabet)):
    keyNum = alphabet.index(orderedCipherMonograms[monograms.index(alphabet[i].upper())])
    key.append(keyNum)

  # Print the initial best guess for the key.
  print(key)


  #_____________________________________________________________________________
  #
  #                                  BIGRAMS
  #_____________________________________________________________________________


  # In preparation for bigram frequency analysis, rename some of the variables
  # and generate a new list of plaintext using the key obtained from 
  # letter frequency analysis.
  parentKey = key
  parentPlaintext = substitution_decrypt(ciphertext,parentKey)
  print(parentPlaintext)

  # Compute the overall bigram fitness of the candidate plaintext.
  parentBiFitness = logFitness(parentPlaintext, 'bigram')

  # If the variable FLAG is true, user is prompted to enter the number of 
  # desired bigram iterations. Otherwise, the quantity stored in numBi is used.
  if flag == True:
    numBi = int(input('Enter the desired number of bigram iterations:  '))

  # Initialize the genetic algorithm for bigram fitness.
  for iter in range(numBi):
    if (iter % 500) == 0:
      print('Bigram iteration = %s' % iter)

    # Randomly select two integers between 0 and 25 which are not the same.
    reducedAlphabet = list(range(0,26))

    a = random.choice(reducedAlphabet)
    reducedAlphabet.remove(a)
    b = random.choice(reducedAlphabet)

    # Initialize a list called childKey which is (right now) just a copy of parentKey.
    childKey = parentKey[:]

    # Randomly swap two of the key values in childKey. The original parentKey
    # is not affected.
    childKey[a], childKey[b] = childKey[b], childKey[a]

    # Attempt to decrypt the ciphertext using the childKey.
    childPlaintext = substitution_decrypt(ciphertext, childKey)

    # Compute the bigram fitness of the child plaintext.
    childBiFitness = logFitness(childPlaintext, 'bigram')

    # If the proportions of n-grams appearing in the child plaintext are
    # closer to normal English than the parent plaintext, we discard parentKey
    #  nd replace it with the new childKey.
    # This will (hopefully) successively improve the quality of the decrypted
    # text.
    if childBiFitness > parentBiFitness:
      parentKey = childKey
      parentBiFitness = childBiFitness
      parentPlaintext = childPlaintext

  #_____________________________________________________________________________
  #
  #                                 TRIGRAMS
  #_____________________________________________________________________________


  # The code in this section is identical to that of the previous section, 
  # except operations are done on trigrams instead of bigrams.
  
    # Compute the overall trigram fitness of the candidate plaintext.
  parentTriFitness = logFitness(parentPlaintext, 'trigram')
  print(parentPlaintext)

  # If the variable FLAG is true, user is prompted to enter the number of 
  # desired trigram iterations. Otherwise, the quantity stored in numTri is used.
  if flag == True:
    numTri = int(input('Enter the desired number of trigram iterations:  '))

  # Initialize the genetic algorithm for trigram fitness.
  for iter in range(numTri):
    if (iter % 500) == 0:
      print('Trigram iteration = %s' % iter)

    # Randomly select two integers between 0 and 25 which are not the same.
    reducedAlphabet = list(range(0,26))

    a = random.choice(reducedAlphabet)
    reducedAlphabet.remove(a)
    b = random.choice(reducedAlphabet)

    # Initialize a list called childKey which is (right now) just a copy of parentKey.
    childKey = parentKey[:]

    # Randomly swap two of the key values in childKey. The original parentKey
    # is not affected.
    childKey[a], childKey[b] = childKey[b], childKey[a]

    # Attempt to decrypt the ciphertext using the childKey.
    childPlaintext = substitution_decrypt(ciphertext, childKey)

    # Compute the trigram fitness of the child plaintext.
    childTriFitness = logFitness(childPlaintext, 'trigram')

    # If the proportions of n-grams appearing in the child plaintext are
    # closer to normal English than the parent plaintext, we discard parentKey
    #  nd replace it with the new childKey.
    # This will (hopefully) successively improve the quality of the decrypted
    # text.
    if childTriFitness > parentTriFitness:
      parentKey = childKey
      parentTriFitness = childTriFitness
      parentPlaintext = childPlaintext


  #_____________________________________________________________________________
  #
  #                                  QUADGRAMS
  #_____________________________________________________________________________


  # The code in this section is identical to that of the previous section, 
  # except operations are performed on quadgrams instead of trigrams or bigrams.

    # Compute the overall quadgram fitness of the candidate plaintext.
  parentQuadFitness = logFitness(parentPlaintext, 'quadgram')
  print(parentPlaintext)

  # If the variable FLAG is true, user is prompted to enter the number of 
  # desired quadgram iterations. Otherwise, the quantity stored in numQuad is used.
  if flag == True:
    numQuad = int(input('Enter the desired number of quadgram iterations:  '))

  # Initialize the genetic algorithm for quadgram fitness.
  for iter in range(numQuad):
    if (iter % 25) == 0:
      print('Quadgram iteration = %s' % iter)

    # Randomly select two integers between 0 and 25 which are not the same.
    reducedAlphabet = list(range(0,26))

    a = random.choice(reducedAlphabet)
    reducedAlphabet.remove(a)
    b = random.choice(reducedAlphabet)

    # Initialize a list called childKey which is (right now) just a copy of parentKey.
    childKey = parentKey[:]

    # Randomly swap two of the key values in childKey. The original parentKey
    # is not affected.
    childKey[a], childKey[b] = childKey[b], childKey[a]

    # Attempt to decrypt the ciphertext using the childKey.
    childPlaintext = substitution_decrypt(ciphertext,childKey)

    # Compute the quadgram fitness of the child plaintext.
    childQuadFitness = logFitness(childPlaintext, 'quadgram')

    # If the proportions of n-grams appearing in the child plaintext are
    # closer to normal English than the parent plaintext, we discard parentKey
    #  nd replace it with the new childKey.
    # This will (hopefully) successively improve the quality of the decrypted
    # text.
    if childQuadFitness > parentQuadFitness:
      parentKey = childKey
      parentQuadFitness = childQuadFitness
      parentPlaintext = childPlaintext


#-------------------------------------------------------------------------------

  # The attack is now complete. Print the attack duration, final candidate key,
  # and final candidate plaintext to the command window.

  print('\n\n\n')
  print('***ATTACK COMPLETE***')
  print('\n')
  end = time.time()
  time = end - start
  print('Elapsed time (in seconds) = %s' % time)
  print('\n')
  print(parentKey)
  print('\n')
  return parentPlaintext


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# The variable 'flag' should be set to True if you'd like to enter bigram, 
# trigram, and quadgram iterations DURING the attack.
#
# Set this variable to False if you'd like to time how long it takes to complete
# the attack.
flag = False

# If the variable 'flag' is set to False, you'll need to enter the desired
# number of iterations below.

# Number of bigram iterations.
numBi = 5000

# Number of trigram iterations.
numTri = 2000

# Number of quadgram iterations.
numQuad = 300

ciphertext = 'CJAWI LGJGI FTUDA TUDAA BFDIE UHMUD NAUUF XRGDA JGEBN FPYGB AMDAT FDUIL FDGCE PGFTG RGKET DRGDD GJFTK DLGDL JGGUR AKETU ACDLG NEJDZ IEJFU NGEPG CJGGB AWFUU REVGJ ZFKTA JETPG FUUDJ GTKDL'

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


substitution_attack(ciphertext, flag, numBi, numTri, numQuad)

---

# The Polyalphabetic Substitution Cipher

The polyalphabetic substitution cipher shares much in common with its monoalphabetic cousin; indeed, the former is a generalization of the latter. You might already be familiar with a particularly famous example of the use of this cryptosystem: [the Enigma machine](https://en.wikipedia.org/wiki/Enigma_machine), a device used by the German military from 1926 through the end of World War II.

We assume that the reader has already familiarized themselves with the monoalphabetic substitution cipher; if this is not the case, then you should go back to the previous section to acquaint yourself.

To encrypt a plaintext message $p$ with the polyalphabetic substitution cipher, Alice would first select a positive integer $M$, which we call the **period** of the cipher. Then, we choose $M$ many monoalphabetic substitution cipher keys and form them into an ordered list:

$$
k = (k_0, k_1, k_2, \ldots, k_{M-1})
$$

From now on we will index lists starting from $i = 0$ to facilitate our definitions of various things; in this case, the encryption function. Specifically, some indices will need to be made amenable to modular arithmetic, and starting with $i = 1$ is problematic. 

Supposing that $p_i$ is the $(i+1)^{\textrm{th}}$ letter in the plaintext message and $c_i$ is to be the $(i+1)^{\textrm{th}}$ letter of the ciphertext, encryption proceeds as follows:

$$
c_0 = e_{k_0}(p_0) \\
c_1 = e_{k_1}(p_1) \\
c_2 = e_{k_2}(p_2) \\
\vdots \quad  \quad \vdots \qquad \vdots \\
c_{M-2} = e_{k_{M-2}}(p_{M-2})\\
c_{M-1} = e_{k_{M-1}}(p_{M-1})\\
c_{M} = e_{k_0}(p_{M})\\
\vdots \quad  \quad \vdots \qquad \vdots 
$$

$$$$

... and so on. Essentially, the encryption process amounts to using a different monoalphabetic subsitution cipher to encrypt each plaintext letter until all $M$ keys have been used up, at which time Alice would begin reusing keys in the same order she employed them the first time. The encryption function for the polyalphabetic substitution cipher can be written succinctly as follows:

$$
e_k(p_i) = e_{k_{i  \pmod{M}}}(p_i)
$$

where $k_{i \pmod{M}}$ is a monoalphabetic substitution cipher key in the list $k$, and hence $e_{k_{i \pmod{M}}}$ is the monoalphabetic substitution encryption function corresponding to that key. 

The decryption function $d_k$ is derived from the encryption function just as it was for the monoalphabetic substitution cipher. Let $d_{k_i}$ be the decryption function associated to the $i^{\textrm{th}}$ monoalphabetic substitution cipher constituting the given polyalphabetic substitution cipher. Then

$$
p_0 = d_{k_0}(c_0) \\
p_1 = d_{k_1}(c_1) \\
p_2 = d_{k_2}(c_2) \\
\vdots \quad  \quad \vdots \qquad \vdots \\
p_{M-2} = d_{k_{M-2}}(c_{M-2})\\
p_{M-1} = d_{k_{M-1}}(c_{M-1})\\
p_{M} = d_{k_0}(c_{M})\\
\vdots \quad  \quad \vdots \qquad \vdots 
$$

$$$$

It follows that the decryption function $d_k$ is given by

$$
d_k(c_i) = d_{k_i \pmod{M}}(c_i)
$$

as expected.

---

## Example: Encryption with the Polyalphabetic Substitution Cipher

Suppose that Alice would like to use a polyalphabetic substitution cipher to encrypt the message 

$$
\tt{This\,\, is\,\, getting\,\, really\,\, complicated.}
$$

As we have become accustomed to, Alice places the text into 5-block form and introduces padding as needed.

$$
\tt{thisi\,\,sgett\,\, ingre\,\, allyc\,\, ompli\,\, cated}
$$

Then, Alice chooses a positive integer $M$ to serve as the period of the cipher. In this case, she chooses $M = 3$. Note that the security of the cipher increases as $M$ increases. The 3-rotor Enigma machine, for example, had a period of over 19,000. Now Alice selects three monoalphabetic substitution cipher keys. We present a function **poly_substitution_key** below which will do this for us. Note that it has a dependency on the function **substitution_key** we used in the previous section.

In [ ]:
# Define a function poly_substitution_key() which accepts a positive integer M
# and returns a random polyalphabetic substitution cipher key with period M.
def poly_substitution_key(M):

  # Initialize an empty list to store the polyalphabetic substitution cipher
  # key.
  key = []

  # Initialize a FOR loop, invoking the function substitution_key at each step
  # and appending the resulting monoalphabetic substitution cipher key to the
  # list "key" on each iteration.
  for i in range(M):
    key.append(substitution_key())

  # The function poly_substitution_key() returns the polyalphabetic substitution
  # cipher key.
  return key


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# The variable M stores the period of the desired polyalphabetic substitution
# cipher.
M = 2

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Run poly_substitution_key() and print the result to the command window.
print(poly_substitution_key(M))

Alice chooses the following three monoalphabetic substitution cipher keys:

$$
k_0 = [8, 12, 14, 0, 2, 23, 11, 7, 5, 13, 1, 21, 16, 24, 22, 9, 20, 17, 19, 10, 4, 6, 25, 15, 3, 18]\\
k_1 = [2, 14, 15, 18, 7, 16, 5, 24, 11, 13, 12, 22, 25, 17, 4, 8, 23, 21, 10, 3, 20, 19, 1, 0, 9, 6] \\
k_2 = [24, 20, 5, 21, 12, 8, 3, 18, 4, 6, 23, 22, 7, 15, 1, 9, 14, 25, 0, 10, 17, 2, 13, 16, 19, 11]
$$

These correspond to the following three substitution tables:

$$$$

$k_0$ | Plaintext | a | b | c | d | e | f | g | h | i | j | k | l | m | n | o | p | q | r | s | t | u | v | w | x | y | z
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- 
| Ciphertext | I | M | O | A | C | X | L | H | F | N | B | V | Q | Y | W | J | U | R | T | K | E | G | Z | P | D | S

$$$$

$k_1$ | Plaintext | a | b | c | d | e | f | g | h | i | j | k | l | m | n | o | p | q | r | s | t | u | v | w | x | y | z
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- 
| Ciphertext | C | O | P | S | H | Q | F | Y | L | N | M | W | Z | R | E | I | X | V | K | D | U | T | B | A | J | G 

$$$$

$k_2$ | Plaintext | a | b | c | d | e | f | g | h | i | j | k | l | m | n | o | p | q | r | s | t | u | v | w | x | y | z
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- 
| Ciphertext | Y | U | F | V | M | I | D | S | E | G | X | W | H | P | B | J | O | Z | A | K | R | C | N | Q | T | L

$$$$

Alice is now in a position to encrypt her message $p$. To encrypt $p_0$, Alice refers to the first table since $0 \equiv 0 \mod{3}$. To encrypt $p_1$, Alice refers to the second table since $1 \equiv 1 \mod{3}$. Likewise, $p_2$ is encrypted using the third table. When Alice gets to $p_3$, she again refers to the first table since $3 \equiv 0 \mod{3}$. Alice continues in this manner until the entire message is encrypted.

$$
e_k({\tt{t}}) = e_{k_0}({\tt{t}}) = {\tt{K}} \\
e_k({\tt{h}}) = e_{k_1}({\tt{h}}) = {\tt{Y}} \\
e_k({\tt{i}}) = e_{k_2}({\tt{i}}) = {\tt{E}} \\
e_k({\tt{s}}) = e_{k_0}({\tt{s}}) = {\tt{T}} \\
\vdots \qquad \vdots \qquad \vdots \\
\textrm{etc.} \qquad \textrm{etc.} \qquad \textrm{etc.}
$$

The full ciphertext is

$$
\tt{KYETL\,\, ALHKK\,\, LPLVM\,\, IWWDP\,\, BQIWF\,\, PYKHV}
$$

Alice then transmits the ciphertext to Bob.

---
$$$$
## Python: Encryption with the Polyalphabetic Substitution Cipher
Below is a Python function called **poly_substitution_encrypt()** which accepts a string of plaintext and a polyalphabetic substitution cipher key and outputs text encrypted with the polyalphabetic substitution cipher.

$$$$

$$\begin{align*} &\textrm{poly\_substitution\_encrypt(plaintext, key)} \quad \longrightarrow \quad \textrm{ciphertext} \end{align*}$$

$$$$

Use it to encrypt the sample plaintext with the key given above. The key should be specified as a list of lists - in other words, a list of the individual monoalphabetic substitution cipher keys in the order they are to be used.

In [ ]:
# Below is a Python function which accepts plaintext and a polyalphabetic
# substitution cipher key and returns ciphertext encrypted with a polyalphabetic
# substitution cipher.

#-------------------------------------------------------------------------------
# Define a polyalphabetic substitution cipher encryption function called 
# poly_substitution_encrypt().
def poly_substitution_encrypt(plaintext, key):

  # Convert alphabetic plaintext to numerical plaintext using plain_to_num.().
  plaintextNum = plain_to_num(plaintext)

  # Initialize an empty list to store numerical ciphertext.
  ciphertextNum = []

  # Initialize a variable to store the period of the supplied key.
  M = len(key)

  # Iterate over the length of plaintextNum.
  for i in range(len(plaintextNum)):

    # Use the key to convert each plaintext letter to ciphertext and append it
    # to ciphertextNum. The apppropriate sub-key is chosen by sending i to its
    # least residue class modulo M.
    ciphertextNum.append(key[i % M][plaintextNum[i]])

  # Convert the numerical ciphertext to alphabetic ciphertet using 
  # num_to_cipher.().
  ciphertext = num_to_cipher(ciphertextNum)

  # Return the alphabetic ciphertext.
  return ciphertext


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Define a key.
key = [[8, 12, 14, 0, 2, 23, 11, 7, 5, 13, 1, 21, 16, 24, 22, 9, 20, 17, 19, 10, 4, 6, 25, 15, 3, 18], 
       [2, 14, 15, 18, 7, 16, 5, 24, 11, 13, 12, 22, 25, 17, 4, 8, 23, 21, 10, 3, 20, 19, 1, 0, 9, 6], 
       [24, 20, 5, 21, 12, 8, 3, 18, 4, 6, 23, 22, 7, 15, 1, 9, 14, 25, 0, 10, 17, 2, 13, 16, 19, 11]]

# Define the plaintext.
plaintext = 'This is getting really complicated.'

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Call substitution_encrypt() on the plaintext and key and print the result.
print(poly_substitution_encrypt(plaintext,key))

---
## Example: Decryption with the Polyalphabetic Substitution Cipher

After receiving the ciphertext from Alice, Bob proceeds to use his identical copy of the key to undo what Alice has done in *exactly* the same manner as Alice, except he reads the three conversion tables from bottom-to-top instead of top-to-bottom.

Referring to the three tables in the previous section, we have

$$
d_k({\tt{K}}) = d_{k_0}({\tt{K}}) = {\tt{t}} \\
d_k({\tt{Y}}) = d_{k_1}({\tt{Y}}) = {\tt{h}} \\
d_k({\tt{E}}) = d_{k_2}({\tt{E}}) = {\tt{i}} \\
d_k({\tt{T}}) = d_{k_0}({\tt{T}}) = {\tt{s}} \\
\vdots \qquad \vdots \qquad \vdots \\
\textrm{etc.} \qquad \textrm{etc.} \qquad \textrm{etc.}
$$

This yields the original plaintext once Bob has finished converting each ciphertext letter.

$$
\tt{thisi\,\,sgett\,\, ingre\,\, allyc\,\, ompli\,\, cated}
$$

After rearranging letters and adding punctuation, Bob can read the original message.

$$
\tt{This\,\, is\,\, getting\,\, really\,\, complicated.}
$$

---
$$$$
## Python: Decryption with the Polyalphabetic Substitution Cipher
Below is a Python function called **poly_substitution_decrypt** which accepts a string of ciphertext and a polyalphabetic substitution cipher key and outputs plaintext.

$$$$

$$\begin{align*} &\textrm{poly\_substitution\_decrypt(ciphertext, key)} \quad \longrightarrow \quad \textrm{plaintext} \end{align*}$$

In [ ]:
# Below is a Python function which will decrypt an arbitrary string of 
# ciphertext which was encrypted using a polyalphabetic substitution cipher. 

#-------------------------------------------------------------------------------
# Define a function poly_substitution_decrypt() which accepts ciphertext and a 
# polyalphabetic substitution cipher key as arguments and returns plaintext.
def poly_substitution_decrypt(ciphertext, key):

  # Convert alphabetic ciphertext to numerical ciphertext using cipher_to_num.().
  ciphertextNum = cipher_to_num(ciphertext)

  # Initialize an empty list to store numerical ciphertext.
  plaintextNum = []

  # Initialize a variable to store the period of the cipher.
  M = len(key)

  # Iterate over the length of plaintextNum.
  for i in range(len(ciphertextNum)):

    # Use the key to convert each ciphertext letter to plaintext and append it
    # to plaintextNum. The apppropriate sub-key is chosen by sending i to its
    # least residue class modulo M.
    plaintextNum.append(key[i % M].index(ciphertextNum[i]))

  # Convert the numerical plaintext to alphabetic plaintext using 
  # num_to_plain.().
  plaintext = num_to_plain(plaintextNum)

  # Return the alphabetic plaintext.
  return plaintext


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Define a key.
key = [[8, 12, 14, 0, 2, 23, 11, 7, 5, 13, 1, 21, 16, 24, 22, 9, 20, 17, 19, 10, 4, 6, 25, 15, 3, 18], 
       [2, 14, 15, 18, 7, 16, 5, 24, 11, 13, 12, 22, 25, 17, 4, 8, 23, 21, 10, 3, 20, 19, 1, 0, 9, 6], 
       [24, 20, 5, 21, 12, 8, 3, 18, 4, 6, 23, 22, 7, 15, 1, 9, 14, 25, 0, 10, 17, 2, 13, 16, 19, 11]]

# Define the plaintext.
ciphertext = 'KYETL ALHKK LPLVM IWWDP BQIWF PYKHV'

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Call poly_substitution_decrypt() on the ciphertext and key and print the result.
print(poly_substitution_decrypt(ciphertext,key))

---
## Polyalphabetic Substitution Cipher: Order of the Key Space

Computation of $|\mathcal{K}|$ for the polyalphabetic substitution cipher is similar to that of the monoalphabetic substitution cipher. You will want to combine Definition 17 (the definition of $n!$) with the Multiplicative Rule of Combinatorics. 

Recall that $n!$ is the number of ways of uniquely ordering $n$ objects and that the Multiplicative Rule of Combinatorics says that the number of ways of uniquely selecting one object each from two sets $A$ and $B$ is the product of the order of these sets, i.e. $|A| \cdot |B|$.

*Question 1:* $\quad$ Suppose that we are dealing with a polyalphabetic substitution cipher with period $M$ and with key $k = (k_0, k_1, \ldots, k_{M-1})$. How many ways are there of choosing $k_0$? How many ways are there of choosing $k_1$? Are these numbers different?

*Question 2:* $\quad$ Given your answer to Question 1, how many ways are there of choosing $k_0$ and $k_1$? 

*Question 3:* $\quad$ Given your answer to Question 2, how many ways must there be of choosing $k$?

*Question 4:* $\quad$ If $M = 3$, what is $|\mathcal{K}|$?

Your answer to question 4 should come out to be roughly $10^{80}$. For comparison, the order of the key space of the Enigma cryptosystem is approximately $1.59 \cdot 10^{20}$. The Enigma is a polyalphabetic substitution cipher with a period of 16900. Why is Enigma's key space so small? A naive calculation would indicate that its key space is larger than $10^{500000}$. 

It turns out that the Enigma's design is the culprit. The majority of the keys available to a general polyalphabetic substitution cipher are unavailable to the Enigma because of its physical components. The Enigma was electromechanical - not a fully digital device. Changing Enigma keys amounted to swapping internal and external components, but these swaps only permitted certain keys to be realized. Once the Allies understood the physical design of the Enigma, cryptanalysis of the Enigma became possible, though still difficult and time-consuming.

---
## Cryptanalysis of the Polyalphabetic Substitution Cipher

Because successful cryptanalysis of the Enigma relies heavily on understanding the particulars of its internal components and design, we will instead consider the general polyalphabetic substitution cipher. It turns out that if we can find a way to determine the period $M$ of the cipher, then cryptanalysis of the polyalphabetic substitution cipher effectively reduces to cryptanalysis of $M$ monoalphabetic substitution ciphers, which is a problem we have already solved. We will return to the issue of determining $M$ later; for now, assume this problem is solved.

Suppose we have the following polyalphabetic substitution ciphertext and that we have determined that its period is $M=2$:

$$
\tt{FDLMS\,\, YVYZY\,\, TOUOB\,\, UQGSO\,\, BIWDL\,\, HWTQG\,\, HGQSF\,\, HVYTJ\,\, ZSHIW\,\, UZHQY\\ SXUOZ\,\, UVOUO\,\, HUFZZ\,\, GMUVO\,\, WTGIW\,\, XFULD\,\, HBGJB\,\, YUAZO\,\, EDFNF\,\, ZZGTO\\ TUFZZ\,\, UFVHG\,\, TUVOF\,\, DLMSY\,\, ZRFBD\,\, GWZAY\,\, TCWDE\,\, UVOQN\,\, FZZGT\,\, WQJHO\\ ZCEWT\,\, WVJBI\,\, EPUJJ\,\, IHEWX\,\, EQFUV\,\, OROYX\,\, EYQCU\,\, JJIHE\,\, KNWUF\,\, YBIRO\\ YNQOZ\,\, ELPGT\,\, WJTUF\,\, VHJZO\,\, TUFZF\,\, CHIFT\,\, LUKNU\,\, OHUFZ\,\, ZYQOH\,\, IFOTH\\ ECFCB\,\, JGIFZ\,\, HOCUE}
$$

This means that **every other letter has been encrypted with the same monoalphabetic substitution cipher**. If we partition the text into two separate ciphertexts, then we'll have two monoalphabetic substitution ciphertexts which will be amenable to letter frequency analysis. The split ciphertext is:

$$
\textrm{Partition 1}
$$
$$
\tt{FLSVZ\,\, TUBQS\,\, BWLWQ\,\, HQFVT\,\, ZHWZQ\,\, SUZVU\,\, HFZMV\,\, WGWFL\,\, HGBUZ\,\, EFFZT\\ TFZFH\,\, TVFLS\,\, ZFDWA\,\, TWEVQ\,\, FZTQH\,\, ZETVB\,\, EUJHW\,\, EFVRY\,\, EQUJH\,\, KWFBR\\ YQZLG\,\, WTFHZ\,\, TFFHF\,\, LKUHF\,\, ZQHFT\,\, EFBGF\,\, HCE}
$$

$$$$

$$
\textrm{Partition 2}
$$

$$
\tt{DMYYY\,\, OOUGO\,\, IDHTG\,\, GSHYJ\,\, SIUHY\,\, XOUOO\,\, UZGUO\,\, TIXUD\,\, BJYAO\,\, DNZGO\\ UZUVG\,\, UODMY\,\, RBGZY\,\, CDUON\,\, ZGWJO\,\, CWWJI\,\, PJIEX\,\, QUOOX\,\, YCJIE\,\, NUYIO\\ NOEPT\,\, JUVJO\,\, UZCIT\,\, UNOUZ\,\, YOIOH\,\, CCJIZ\,\, OU}
$$

Now, we know that parts 1 and 2 were encrypted using monoalphabetic substitution ciphers. We can determine with a good degree of certainty which letters decrypt to ${\tt{e}}, {\tt{t}}, {\tt{a}}, {\tt{o}}, {\tt{i}}$, and ${\tt{n}}$. 

When dealing with a normal monoalphabetic substitution cipher, our next step would be to conduct bigram and trigram analysis on each partition of the ciphertext. However, there is a problem: a bigram in a partition of our polyalphabetic substitution ciphertext is *not* a bigram in the original ciphertext. For example, the first bigram in Partition 1 is ${\tt{FL}}$, but the first bigram of the original ciphertext is ${\tt{FD}}$. This is because the partitions alternately draw ciphertext letters from the original ciphertext.

It turns out that each plaintext bigram has up to $M$ unique manifestations in the ciphertext. Can you see why?

So how will we deal with this? If we were forced to do this by hand, we might first proceed as with the monoalphabetic substitution cipher by counting the number of times each ciphertext bigram appears. Since each plaintext bigram corresponds to $M = 2$ ciphertext bigrams, and each of the two ciphertext bigrams should appear with roughly equal probability, we could guess that the two most frequent ciphertext bigrams map to the most common plaintext bigram (${\tt{th}}$). Likewise, the third and fourth most common ciphertext bigrams might correspond to the second most common English bigram (${\tt{he}}$). 

We would continue in this fashion until usable bigrams are exhausted. We would repeat a similar process with the ciphertext trigrams; in this case as well, there could be $M$ unique ciphertext trigrams associated to any given plaintext trigram.

We might also need to use quadgrams (or even 5-grams) to whittle the key space down to a size which would permit us to use a brute force attack on the reduced key space in a reasonable amount of time. Provided we have enough ciphertext to work with, this would permit us to successfully attack the ciphertext.

$$$$

---
$$$$

### Determining the Period of a Polyalphabetic Substitution Cipher

Recall that we skipped a step in our cryptanalysis: determining the period of the cipher. If we have enough ciphertext available, there is a relatively straightforward procedure for computing the period $M$. 

> *Definition 18.* $\quad$ The **index of coincidence**, or **IOC**, of a text is the normalized probability of drawing a letter from a given string of text twice in a row if letters are chosen randomly and the second letter is chosen without replacing the first letter.

For English text with no punctuation or spaces, the probability of drawing a letter $k$ (here, $k$ does not denote the letter ${\tt{k}}$, but is instead a substitute for an arbitrary letter of the alphabet) twice in a row is

$$
\frac{n_k}{N} \cdot \frac{n_k-1}{N-1}
$$

where $n_k$ is the number of times $k$ appears in the text and $N$ is the total number of letters in the text. Then the index of coincidence is

$$
{\textrm{IOC}} = 26 \cdot \sum_{i=1}^{26} \left(\frac{n_i}{N} \cdot \frac{n_i-1}{N-1}\right)
$$

The coefficient 26 is the normalization constant (the number of available characters in the text. If $n_i < 2$ for some letter $i$, then the term corresponding to that letter is taken to be zero (if a given letter appears fewer than two times in the text, then there is no way to draw that letter from the text two times without replacement).

It turns out that the expected value of the index of coincidence for a random sample of English text is approximately 1.73. To determine the most likely value for $M$, we perform the following procedure:

1.   Choose a candidate value for $M$.
2.   Split the ciphertext into $M$ partitions as described previously.
3.   Compute the IOC for each partition separately.
4.   Compute the average IOC for all $M$ partitions.
5.   Repeat steps 1-4 until the maximum possible value of $M$ has been reached.

The value of $M$ which yields an average IOC closest to 1.73 is the period of the cipher.

Below is a Python function called **IOC()** which accepts a string of text and returns its index of coincidence.

In [ ]:
# Define a function called IOC() which accepts a string of text and returns
# its index of coincidence. We assume that the IOC is computed for the 26
# letters of the English alphabet.

def IOC(text):

  # Call the function count_mono_all() on the text and store it (the
  # number of times each English letters appears) in a vector called letterCounts.
  letterCounts = count_mono_all(text, 0)

  alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
              'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
              'y', 'z']
  
  # Compute the total number of English letters appearing in the text and
  # store this number in the variable N.
  N = 0
  for i in range(len(text)):
    if text[i].lower() in alphabet:
      N = N + 1
  
  # Compute the index of coincidence of the given text.
  indexOfCoincidence = 0
  for n in letterCounts:
    if (n != 0) and (n != 1):
      indexOfCoincidence = indexOfCoincidence + (n*(n-1))
  indexOfCoincidence = indexOfCoincidence*26/(N*(N-1))

  # The function IOC() returns the index of coincidence of the supplied text.
  return indexOfCoincidence

---
$$$$
## Exercise: Cryptanalysis of the Polyalphabetic Substitution Cipher

Below is a Python function called **poly_substitution_attack()** which accepts a string of ciphertext and returns a string of candidate plaintext obtained via a genetic algorithm. 

$$$$

$$ \textrm{poly\_substitution\_attack(ciphertext)} \quad \longrightarrow \quad \textrm{likely plaintext}$$

$$$$

The attack procedure is as described in the previous section. Use this function to attack the following ciphertexts:

> [Polyalphabetic Substitution Ciphertext (short)](https://drive.google.com/file/d/18hh8Beuk57FYzeocb0Du2YBwpY0X_NeN/view?usp=sharing)

> [Polyalphabetic Substitution Ciphertext (medium)](https://drive.google.com/file/d/14EMJM6jh9gQrkRyu5kzCiz-A2ONi3a9e/view?usp=sharing)

> [Polyalphabetic Substitution Ciphertext (long)](https://drive.google.com/file/d/1ei6-fyV3SrvlmBHVfoUDW3A8Dzdnyat9/view?usp=sharing)



**Experiment with different numbers of bigram, trigram, and quadgram iterations on the ciphertexts given above. How does the length of the ciphertext affect the attack in accuracy and duration? Is it true that a longer text sample leads to a more accurate and faster attack?**

In [ ]:
# Define our attack function poly_substitution_attack() which accepts a string
# of ciphertext encrypted with a monoalphabetic substitution cipher and outputs
# likely plaintext.
#
# The arguments of this function are as follows:
#
#     1. CIPHERTEXT       --- Ciphertext to be attacked
#     2. ITERFLAG         --- If TRUE, the NUM argument is ignored and attacker
#                             is prompted to input desired iterations for
#                             each n-gram.
#     3. SHAKE            --- Vector of length 4. Desired number of random key
#                             swaps, then number of bigram, trigram, and quadgram
#                             iterations permitted to pass before reducing number
#                             of random key swaps by 1.
#     4. NUM              --- Vector of length 3. Desired number of bigram,
#                             trigram, and quadgram iterations, in that order.
#     5. MAXPERIOD        --- Maximum value of M to be considered during IOC
#                             attack phase.
#     6. PERIODOVERRIDE   --- Flag which determines whether IOC attack phase will
#                             take place.
#     7. M                --- If periodOverride is set to True, then this variable
#                             is the period value the attack will assume.

def poly_substitution_attack(ciphertext, iterFlag, shake, num, maxPeriod, periodOverride, M):

  # Import the Python package RANDOM and the function log10() from MATH.
  import random
  from math import log10
  import time

  # Start a stopwatch to time execution of this function.
  start = time.time()

  alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
              'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
              'y', 'z']
  
  # Convert the alphabetic ciphertext to numerical ciphertext. 
  # We only do this so we can get an accurate count of the total number of
  # letters in the ciphertext.
  ciphertextNum = cipher_to_num(ciphertext)

  # Store the total number of letters in the ciphertext to the variable totalMono.
  totalMono = len(ciphertextNum)

  # Store the total number of bigrams in the ciphertext to the variable 
  # totalBigrams.
  totalBigrams = len(ciphertextNum) - 1

  # Store the total number of trigrams in the ciphertext to the variable 
  # totalTrigrams.
  totalTrigrams = len(ciphertextNum) - 2

  # Store the total number of quadgrams in the ciphertext to the variable 
  # totalQuadgrams.
  totalQuadgrams = len(ciphertextNum) - 3

  ciphertextAlpha = []
  for i in range(len(ciphertextNum)):
    ciphertextAlpha.append(alphabet[ciphertextNum[i]])

  indicesOfCoincidence = [float(0)]*(maxPeriod-1)

  for Mcandidate in range(1,maxPeriod):
    split = []
    for columns in range(Mcandidate):
      split.append([])
    for i in range(len(ciphertextAlpha)):
      split[i % Mcandidate].append(ciphertextAlpha[i])
    temp = []
    for i in range(Mcandidate):
      split[i] = ''.join(split[i])
      temp.append(IOC(split[i]))
    indicesOfCoincidence[Mcandidate-1] = sum(temp)/len(temp)
  print(indicesOfCoincidence)
  for k in range(len(indicesOfCoincidence)):
    indicesOfCoincidence[k] = abs(indicesOfCoincidence[k] - 1.73)

  if periodOverride == False:
    maxIOC = min(indicesOfCoincidence)
    M = indicesOfCoincidence.index(maxIOC)+1

  ciphertextSplit = []
  for columns in range(M):
    ciphertextSplit.append([])
  for i in range(len(ciphertextAlpha)):
    ciphertextSplit[i % M].append(ciphertextAlpha[i])
  for i in range(len(ciphertextSplit)):
    ciphertextSplit[i] = ''.join(ciphertextSplit[i])


  #_____________________________________________________________________________
  #
  #                                 MONOGRAMS
  #_____________________________________________________________________________


  key = []
  
  for column in range(M):

    # Initialize a list to store ciphertext monogram frequencies.
    cipherMonogramData = []

    # Record the ciphertext monogram frequencies.
    for i in range(len(alphabet)):
      count = count_mono(ciphertextSplit[column],alphabet[i])
      cipherMonogramData.append(count)
  
    # Initialize a list to store ciphertext monograms in descending order of
    # frequency.
    orderedCipherMonograms = []

    # Run a FOR loop to store ciphertext monograms in descending order of
    # frequency in orderedCipherMonograms.
    for i in range(len(alphabet)):
      currMaxCount = max(cipherMonogramData)
      indexOfCurrMaxCount = cipherMonogramData.index(currMaxCount)
      cipherLetter = alphabet[indexOfCurrMaxCount]
      orderedCipherMonograms.append(cipherLetter)
      cipherMonogramData[indexOfCurrMaxCount] = -1

    # Initialize a list to store our guess for the key after applying
    # the letter frequency analysis technique.
    splitKey = []

    # Append the required key values resulting from letter frequency analysis.
    for i in range(len(alphabet)):
      keyNum = alphabet.index(orderedCipherMonograms[monograms.index(alphabet[i].upper())])
      splitKey.append(keyNum)

    key.append(splitKey)

  #-----------------------------------------------------------------------------

  # Define the initial best guess for the key and attempt to decrypt the 
  # ciphertext with it.
  parentKey = key[:]
  parentPlaintext = poly_substitution_decrypt(ciphertext, key)


  #_____________________________________________________________________________
  #
  #                                  BIGRAMS
  #_____________________________________________________________________________


  # Initialize a list to store the fitnesses of candidate plaintext bigrams.
  parentBiFitness = logFitness(parentPlaintext, 'bigram')

  # Print the parent key and current candidate plaintext for examination.
  print(parentKey)
  print(parentPlaintext)

  # If the variable FLAG is true, user is prompted to enter the number of 
  # desired bigram iterations. Otherwise, the quantity stored in numBi is used.
  if iterFlag == True:
    num[0] = int(input('Enter the desired number of bigram iterations:  '))

  shakeBi = shake[0]

  # Initialize the genetic algorithm for bigram fitness.
  for iter in range(num[0]):
    if (iter % 250) == 0:
      print('Bigram iteration = %s' % iter)

    for s in range(shakeBi):
      [colNum] = random.sample(range(M),1)

      # Randomly select two integers between 0 and 25 which are not the same.
      [a,b] = random.sample(range(0,26), 2)

      # Initialize a list called childKey which is (right now) just a copy of parentKey.
      childKey = parentKey[:]

      # Randomly swap two of the key values in childKey. The original parentKey
      # is not affected.
      temp = childKey[colNum].copy()
      a_index = temp.index(a)
      b_index = temp.index(b)
      temp[a_index], temp[b_index] = temp[b_index], temp[a_index]
      childKey[colNum] = temp[:]
      temp = []

    # Attempt to decrypt the ciphertext using the childKey.
    childPlaintext = poly_substitution_decrypt(ciphertext, childKey)
    childBiFitness = logFitness(childPlaintext, 'bigram')

    # If the proportions of n-grams appearing in the child plaintext are
    # closer to normal English than the parent plaintext, we discard parentKey
    #  nd replace it with the new childKey.
    # This will (hopefully) successively improve the quality of the decrypted
    # text.
    if childBiFitness > parentBiFitness:
      parentKey = childKey
      parentBiFitness = childBiFitness
      parentPlaintext = childPlaintext
    
    percentBi = shake[1]
    if (shakeBi > 1) and (iter != 0) and ((iter % percentBi) == 0):
      shakeBi = shakeBi - 1


  #_____________________________________________________________________________
  #
  #                                  TRIGRAMS
  #_____________________________________________________________________________


  # Initialize a list to store the fitnesses of candidate plaintext trigrams.
  parentTriFitness = logFitness(parentPlaintext, 'trigram')

  # Print the parent key and current candidate plaintext for examination.
  print(parentKey)
  print(parentPlaintext)

  # If the variable FLAG is true, user is prompted to enter the number of 
  # desired trigram iterations. Otherwise, the quantity stored in numTri is used.
  if iterFlag == True:
    num[0] = int(input('Enter the desired number of trigram iterations:  '))

  shakeTri = shake[0]

  # Initialize the genetic algorithm for trigram fitness.
  for iter in range(num[1]):
    if (iter % 100) == 0:
      print('Trigram iteration = %s' % iter)

    for s in range(shakeTri):
      [colNum] = random.sample(range(M),1)

      # Randomly select two integers between 0 and 25 which are not the same.
      [a,b] = random.sample(range(0,26), 2)

      # Initialize a list called childKey which is (right now) just a copy of parentKey.
      childKey = parentKey[:]

      # Randomly swap two of the key values in childKey. The original parentKey
      # is not affected.
      temp = childKey[colNum].copy()
      a_index = temp.index(a)
      b_index = temp.index(b)
      temp[a_index], temp[b_index] = temp[b_index], temp[a_index]
      childKey[colNum] = temp[:]
      temp = []

    # Attempt to decrypt the ciphertext using the childKey.
    childPlaintext = poly_substitution_decrypt(ciphertext, childKey)
    childTriFitness = logFitness(childPlaintext, 'trigram')

    # If the proportions of n-grams appearing in the child plaintext are
    # closer to normal English than the parent plaintext, we discard parentKey
    #  nd replace it with the new childKey.
    # This will (hopefully) successively improve the quality of the decrypted
    # text.
    if childTriFitness > parentTriFitness:
      parentKey = childKey
      parentTriFitness = childTriFitness
      parentPlaintext = childPlaintext
    
    percentTri = shake[2]
    if (shakeTri > 1) and (iter != 0) and ((iter % percentTri) == 0):
      shakeTri = shakeTri - 1


  #_____________________________________________________________________________
  #
  #                                  QUADGRAMS
  #_____________________________________________________________________________


  # Initialize a list to store the fitnesses of candidate plaintext quadgrams.
  parentQuadFitness = logFitness(parentPlaintext, 'quadgram')

  # Print the parent key and current candidate plaintext for examination.
  print(parentKey)
  print(parentPlaintext)

  # If the variable FLAG is true, user is prompted to enter the number of 
  # desired quadgram iterations. Otherwise, the quantity stored in numQuad is used.
  if iterFlag == True:
    num[0] = int(input('Enter the desired number of quadgram iterations:  '))

  shakeQuad = shake[0]

  # Initialize the genetic algorithm for quadgram fitness.
  for iter in range(num[2]):
    if (iter % 20) == 0:
      print('Quadgram iteration = %s' % iter)

    for s in range(shakeQuad):
      [colNum] = random.sample(range(M),1)

      # Randomly select two integers between 0 and 25 which are not the same.
      [a,b] = random.sample(range(0,26), 2)

      # Initialize a list called childKey which is (right now) just a copy of parentKey.
      childKey = parentKey[:]

      # Randomly swap two of the key values in childKey. The original parentKey
      # is not affected.
      temp = childKey[colNum].copy()
      a_index = temp.index(a)
      b_index = temp.index(b)
      temp[a_index], temp[b_index] = temp[b_index], temp[a_index]
      childKey[colNum] = temp[:]
      temp = []

    # Attempt to decrypt the ciphertext using the childKey.
    childPlaintext = poly_substitution_decrypt(ciphertext, childKey)
    childQuadFitness = logFitness(childPlaintext, 'quadgram')

    # If the proportions of n-grams appearing in the child plaintext are
    # closer to normal English than the parent plaintext, we discard parentKey
    #  nd replace it with the new childKey.
    # This will (hopefully) successively improve the quality of the decrypted
    # text.
    if childQuadFitness > parentQuadFitness:
      parentKey = childKey
      parentQuadFitness = childQuadFitness
      parentPlaintext = childPlaintext
    
    percentQuad = shake[3]
    if (shakeQuad > 1) and (iter != 0) and ((iter % percentQuad) == 0):
      shakeQuad = shakeQuad - 1

  #-----------------------------------------------------------------------------


  # The attack is now complete. Print the attack duration, final candidate key,
  # and final candidate plaintext to the command window.

  print('\n\n\n')
  print('***ATTACK COMPLETE***')
  print('\n')
  end = time.time()
  time = end - start
  print('Elapsed time (in seconds) = %s' % time)
  print('\n')
  print(parentKey)
  print('\n')
  return parentPlaintext


  #-----------------------------------------------------------------------------


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Define a variable to store the ciphertext.
ciphertext = '''VMGWA TYPPV KHJZO YFEUA KXDHG SCIPV KHUNI ROEUA KXDHG UGGIW GHGGD VVCGH HEURD EKPVZ VPLHI AACYY JNPRO YZQGO KQBSS GQVGP CASYW PELHA USGZA PTATN CPYXE FSVOE KJGOA RJSSH HAKSS WMCVO FQBMS YEMYI WAZNG VZHHE URCEJ JEOAQ CVCRG IGYEZ TIIEY UAEZA OEKHG ZGHHE KMBSS QCCBM AWVAK SDZKW URUCG XEOYZ CECQC PYDJA GDGIS WJYTR GZLGL UGQCF SMAMI RZVZC GZGPS HPWLW EBESD QUPSS DYUHH EURCY LAKOZ AREUH ONPIG IZNEV UQYEC PEUGC VCKQY EMIJE WVEYK RJHOW CAXGD IGWAM GGJYU DUAMR BSZNE CMAWI MGUPE YXNBW SXDHU FGGUE AHHEA QUSYY CCSMB MKARF MQDDE YXZGW VLKRB MGDCP GYPVK HVCZN EICQX PZRBE AHEFD YSFEG BCBRA SZNEL WAKWA NGWAH HEEVI ZGGCP UAANU EEZZN EZGSV CYPKM ARBCP VSOTQ BXZAA ABRGI ZNESG WESVJ EYZHV FDRJS SHHYZ JGZZQ GOSZT VMHSR YLRAX RWNCD UTGVE GCQBS GDSOK MRIEL SEUHJ YZGRN CCEYU QBXBM AYUPW ECYGU GHWPE PZECR JWOJG RZNEL MRRSD MOZER JWVUL OGOCH SVBAU ECPGH WNPWV XORJT EHHSD VGUIQ BXDRV MYYJF SHCNU ITNOH HPYDR NCEGS ZRBSD RSOWQ JEUHP ECVGM AFEZC AJAUR VOAFE ZCAJE ZHEAY YJAES VSWNE MZNEC ZGOZV JSFZM OEEYC PGXSC EJAYZ RDPYY XEYPC PGJVC KGXEZ NONOE HPVJE ZKJVF GSVCW ATGMR CEAQB AZPBN UHSHG WVGKR VSSYD EZNEE XRBGS SJECQ BXSZU SYVUX YXETY ICNZN EGCRJ NLQBY ULENB GUAUE AEIVS XDHUP EYSOT DSCFH HEKQM ESZVH YVIWG LEOZR OEAAB SDRYA GSSOT PDZGR BSDGC CJPVA AFEZC AJMWA BIEDE OZWLP EPCPE YHYUD UFMGT GGDCA TNCWV UEPEY JPEPI YWBVO AHHEF YKFFW ECNNS SGYEM ZNEWE HCWGF EZCAJ EZHEC IACPT QOWKN VMGOZ ECQEO WRJSD RHLPR OCPGD EPGUC YEEIS VCPGZ SZKHC ALRSO ZNEAC WSRGP VOASE ZGPEW BLGOK LSNOP GRGVC PGCGH GYCYC IUEUP VSEAB NBQBC EDENO HBEKP CPGII VCQEM ZNEAC ESXTW ECYYJ FDGUE AABEY YGSDR OTEWJ WVGIN OHCPG QOHSH HECPD ZGGTA UEWEX ROEYL HEAOW EXROE YLHEA OWEXR QVERC FDQAM CRBCY QJBGV ONAQB EKNVZ PWLYC RLNOP KZGFE ZCAJM NNVSE PCPGV ESSUE UOHUV CRVCF RJBGV ONADX WYYDA UEKGY HCPGU KWTRG IBRVS OVEWG PUHGH VWQMU SOYJE CHHEW RSWEY XAZVV OZNEW GYXSD ARSDR ONSCJ AKGZG GGOAU ECPCA KXDHH ENGAW YHEAZ NEZGY JAZSV CYPAN UEVCZ NECDQ ZBGVO NADUF YVDEM IFOGS VSDQB XYUGV ZHHEZ NSFFV GMSZT EZGAE JLEGZ HHYZQ CTYPD YMWEM YCSFC AYYWH HYZAB EYPFE AQCQO RUSEA BCEZG OGSSC DRJSD GCABA BEAQJ OSHSS KHSWM NVMEH USYPF NBEKA AQBXZ NECDQ ZSSGZ ZGAOM GVEMA RUSEY VSEAB NBZEE AQBXS YEOGV XAGPR ZSCCP GXVZE AKCKM IXYWV FZQDG SSEZK HVSEA BCSZD NLJKS EYXSD REQOG CASYU ISVCP GNLGG VUGYH SYMFK HPPPE CVGMA GBMDQ UIYCS WVNVM SYALZ AWYEH VOAWS RGQBS DRDNL ZGZZG IWGVE CEDEO WRQVY VCECP GIZNE CDQZC SCENU RHYAA BFGHG WAFEZ CAJMZ NVSZN EYWGC SDREO AARHE LONXG DCZAG MBAOY OHGHY HSFWA TGOHE ZEYVO WQEOZ RBXMQ UPIMC PGSVC SYCPG RJXGA RISVX EZHSO TRYEU HHYZF EZCAJ AURUE VRUTG VEHSQ USYPU PGSVS WNEMZ NEREP SGMGC EELVO ZNEWP QCABR EWBMB OVGIN OHAEY XSOTR VZZNW PVZGZ PRCEK PVDGD EHYYJ EAFEZ CAJMN RHYAY GSDQB XZNEZ GSEWM NVRGR YECIC PEYXN UOLNO SGOZU EYMAB EVAKT SYCUG GZASY EECHH ECRVZ GAYEC GTAMW SNUJE NPWEN UHHEP WVOGH VWCRV MVEGN AWGZA AKZTV EYZEO YUDDP EWJZG YWAMW IEMAG DEYXI SVBEN SGZMD UUGLV VKRNT EWAUG AYECL ONNDE MZNEO YZCEC GOEBW EFZQY EPGKC GQCEM WLNOQ CCYWK FFICP EYXSD RDNLJ KSGVU TSVFE AAKSE YCECP CEMWV ZZVVR GWCPG SVLZN EZYLE AKEON NQBXE BBNNQ FOSSU YEDPE CVGME YEHEP EZYUA LQROZ SDESZ RSCYQ JGCAT GZWLN OVTAW VGRYL SCZNE UGPCH ELONX GDAUH HENAO WAQCP EYFCS HGNKG SMQRO ZSDJS SMUWE YXPGV HYEVS SNGUY UQDEB REWEY XSSNV RGGTA WVGRY LGIVA KZSSB YUDPE CVGMA SVCTW VMDRW YKJVZ ZARPE PXEUR OYZQG OYYJO SACPG VSODQ UIYHH ECPLN OHHSD RGOMI DNLJK SGVUP YDIEG YCZGC EOAAK CLGDP EYECZ GFAUE KGYNK OAVEM KTKYC RTAMR UNBWV OAHHE CRWYK ABWVA BEZAV GMGBE ZJAYU RCYCI VFKHH EVSEZ GLVWM RJSDR LPYDI EGYXZ SSSOT QBCEK ECZRV MEWLI SVVSD AKCYY JLGGO CYYJS DRBYM WVSSY DEWGT ECRRA URTEU HSOPW VFGAR SCGBC EPCNC PHYAL GHGCG WGLKW YVYYM XECKA CPYHE RGYCP GWVZT RUSPW VOGHV ZVGDF SMAMI RZVZQ BSSGU GYLEN UWLPY WRSDR YNMMT ESZVC PGDEK NSGQR OZSDJ IGWCV PWSIZ RJYKN EYMSV LKDSM NNEOD RCPSM XPZHH YZNSC SSBGG VUNUG AHELO NXGDT YPTYU ICALR UHSVE FSCZW ELVSG DCPYY CPGGB FERBS YYJGC QTAZQ YELMA SEXVF ZNVSD GJIEV USZGT EAHHE KMBYU DVWLA USYPD NLJAA WGCEA GUEYV CPKJA YURCY CXVFZ NEWYV XEKHC PYHHY AZSZK HUNMX EMZNE GCAIW GCGID IZECP ZYZQV WZVVR GWVOA NVMLG JEZVS GKHGS DRUSY VUGSP UAIWE CSCVO VPCYC PUNLG BLPWV OGHUC EEHEA FEZCA JAURI VKIWA ZNHEC AWOZN GVTNC CEPKG PAUEB GTAMQ ECNQA WIRXN EYXNO HCNUR WGMGB EZPRN CRYEC HHENG LTGGO EUAWO SHRNC RYECP VAAFE ZCAJM NQCPY PTAMR SSNQA WYWAC ZAZCS CEMYI IVZYG SBAOU EWAAS YUNBI EYCPT YUIIA MWSNU PERGY CPGPC YCPOV UDGTU IGVFY GTGYC ZSJLH OPCAU LOEYP ETDGC CGYCZ SJLMY DJLKN OAMWE MQROZ SDESZ RSAGY CZSJL WEHCW GPWEG HSCQM USYSG ZASHA WNTEY YUSDR VHSMB SSZOV UYSOT DGTUA RSDRK OEXEZ KRERG VLSDQ BXCMB CAAWO VAKDU AWWEB ELSMO WEHCW GSVWF QESYW FAGVG USHOE LRTUG VDYUH LNOFK CZJKS EYVOG SZNNR OVUQC WEBET EHHHV VGUSH CPGPC YCPVZ GHHEP AWECM BAZPJ EYVGO WRCPG IOETA BEZNE ZGGOE UATNC RZNNR OVUQC CQROZ SDESZ RSYZA BFGPE SOJVP SSAMS YCWGH CPGCJ YADLM SYCWG HCPGP CYCPO VUDGT UYGTM AGDNN VSVAK RGDGO GSHAK JEZGD PECVG MEYEE JGUGG VVSGD HNNSV CEHGD UAWAZ SGVMD RZEEH SGYCP GCPEC VGMAS HAKJE ZGDIY WBVCF HHELQ DZSXV FNGSW GDPEC VGMGH CEEGU DDQTP SSCNZ MOOZN ECZGO CSYVX YQBXS GHEYD UYEDP ECVGM EYEAZ SSWMT KAGHC PGCJN NYPEC VGMGH CEEQW YKUEX EYBAU ECNWV LYMPG BGVON ADUPC MXXGD BNNYG TDABE VPSWM GUDLQ DZSXV FAABS NAOZV NEWMH EWMMU PGGUD GDCPG CSFCA YYWGJ MEYXQ OQDDM IZZEY CSDRV OKSEZ QROZS DJFOJ ZEAHH EKHOA PAOSD QBFGW AVBQA HYYJC YQJFD REZBM AWVPE EUAWS DRTAW VGRYL UYVPS SNQAW ZGFEW GOESZ ERGVL SDQBX NNEOZ NESEC EFSCE CKAJN UHWNC VLBGV ONAQB EKGSM YYJOS SDPEW JZGYS SKHSH GZGZI RJTGW AUGQB NOVBE NNGHG PGNUF EZCAJ MCRVM ZNETS VJCSY CPGLE WMMRA MCVXY QBUGZ GZGDE CZVGL EYXAZ QBCOZ SFERB SAGCY BAOHG GBAUE RVMGB CNROZ'''

# If iterFlag is set to True, then the algorithm will ask the user to input
# the desired number of bigram, trigram, and quadgram iterations. If iterFlag
# is set to False, then the algorithm will use the values stored in the variable
# num.
iterFlag = False

# The first entry of shake is the number of random swaps to make between key 
# entries. The remaining entries are the number of bigram, trigram, and quadgram
# iterations to permit before reducing the number of random swaps by 1, respectively.
shake = [5,500,50,15]

# The entries of num are the numbers of bigram, trigram, and quadgram iterations
# to complete before passing control to the next portion of the attack script, 
# respectively.
num = [8000,1500,150]

# maxPeriod is the maximum value to be considered when attempting to deduce the
# period M of the cipher.
maxPeriod = 10

# If periodOverride is set to True, then the IOC attack will be skipped and 
# the algorithm will assume the value of M set below.
periodOverride = False
M = 1

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Execute an attack on the given ciphertext encrypted with a polyalphabetic
# substitution cipher.
poly_substitution_attack(ciphertext,iterFlag,shake,num,maxPeriod,periodOverride,M)

---
$$$$
# The Diffie-Hellman Key Exchange

The security of every cryptosystem developed through the end of the 1960s relied on the ability of two parties to exchange keys privately. This is a massive drawback. Is there a secure way to exchange cryptographic keys via a publicly accessible form of communication?

This is precisely the question that mathematicians [Whitfield Diffie](https://en.wikipedia.org/wiki/Whitfield_Diffie) and [Martin Hellman](https://en.wikipedia.org/wiki/Martin_Hellman) asked themselves in the early 1970s. The answer is yes, and the solution is elegant. It is named the Diffie-Hellman key exchange after its inventors and [it was published](https://ee.stanford.edu/~hellman/publications/24.pdf) in the academic journal *IEEE Transactions on Information Theory* in 1976. Actually, a British cryptographer Malcolm Williamson came up with the same solution seven years earlier, but the British government classified it. So was born the field known as **public key cryptography**.

The Diffie-Hellman key exchange is not, strictly speaking, a method to encrypt messages. It is a method to exchange keys for use in a symmetric key cryptosystem. In fact, the two parties exchanging the key have no control over what the key will look like! The exchange method is based on a computationally difficult mathematical problem called the **discrete logarithm problem**, or DLP.

---
$$$$
## Mathematics of the Diffie-Hellman Key Exchange

To understand the Diffie-Hellman key exchange, we'll need to understand the DLP, and to understand the DLP, we need some new mathematics.

> *Definition 19.* $\quad$ Let $G$ be a nonempty set equipped with a binary operation $\circ$ such that the following conditions are satisfied:
1.   Closure: If $a \in G$ and $b \in G$, then $a \circ b \in G$.
2.   Associativity: $a \circ (b \circ c) = (a \circ b) \circ c$ for every $a, b$, and $c$ in $G$.
3.   There exists an element $e$ in $G$ (called the **identity element**) such that $a \circ e = e \circ a = a$ for every $a \in G$.
4.   For each $a \in G$, there is an element $d \in G$ (called the **inverse** of $a$) such that $a \circ d = e$ and $d \circ a = e$. For any element $a \in G$, we denote its inverse by $a^{-1}$.

Whenever a set $G$ together with a binary operation $\circ$ form a group, we say that $G$ **is a group under** $\circ$.

*Example* $\quad$ Let $G = \mathbb{Z}$, the set of integers, be equipped with the binary operation $+$, or addition. Then $G$ is a group under $+$. Let's check that the axioms are satisfied via examples (we won't prove it). 

1.   If $a$ and $b$ are integers, say $a = 5$ and $b = -50$, then $a + b = 5 + (-50) = -45$, which is an integer, and therefore an element of $G$.
2.   If $a = 5$, $b = -50$, and $c = 3$, then

$$ \begin{align*}
a + (b + c) &= 5 + (-50 + 3) = 5 + (-47) = -42\\\\
(a + b) + c &= (5 + (-50)) + 3 = -45 + 3 = -42
\end{align*}
$$

3.   Observe that if $n$ is any integer, then $0 + n = n + 0 = n$. It turns out that $0$ is the identity element of $\mathbb{Z}$ under addition.
4.   If $n$ is any integer, then $-n$ is the inverse of $n$. Why? Note that $n + (-n) = n - n = 0$ and $-n + n = 0$. 

An interesting point is that in strict group notation, we would denote $-n$ as $n^{-1}$. The context is important.

$$$$

*Question: Can you think of another example of a group? You need to specify the set and the operation and ensure they satisfy the axioms 1-4 above.*

$$$$

*Example* $\quad$ Let $G = \mathbb{Z}$, the set of integers, be equipped with the binary operation $\cdot$, or multiplication. Then $G$ is **not** a group. It turns out that axioms (3) and (4) both fail to hold. Can you find counterexamples in both cases? 

*Breaking Axiom 3: Is there an integer such that when you multiply it by any other integer you always get the same answer?*

*Breaking Axiom 4: Notice that in the first example ( $\mathbb{Z}$ under addition), the notion of "subtraction" is related to inverses. What is the analogous "operation" to multiplication?*

$$$$

We now turn our attention to a particular class of groups called the **multiplicative groups**, which we define via a well-known theorem.

> *Theorem 4.* $\quad$ Let $n$ be a positive integer with $n > 1$ and let $G$ be the set of least residue classes $r$ of integers modulo $n$ such that $r$ belongs to $G$ if and only if $r$ is coprime with $n$. Then $G$ is a group under multiplication modulo $n$. 

If $G$ is a set of integers which form a group under multiplication modulo a positive integer $n > 1$, then $G$ is called a **multiplicative group modulo $\mathbf{n}$**, and we write $G = \mathbb{Z}_n^*$.

This type of group is central to the Diffie-Hellman key exchange.

$$$$

*Example* $\quad$ Let $n = 12$. Then $G = \{1, 5, 7, 9, 11 \}$ forms a group under multiplication modulo $12$. 

*Exercise: Verify that the group referenced above satisfies the group axioms 1-4. Why is it that if we include, for example, the number 2 in $G$, then $G$ fails to be a group under multiplication modulo 12?*

*Question: Under what condition on $n$ will $\mathbb{Z}_n^*$ contain* **every** *integer between $1$ and $n-1$? (Hint: think carefully about the condition a least residue class must satisfy for inclusion. Refer to Theorem 4.)*

$$$$

We now define exponentiation in a general group with an unspecified binary operator.

> *Definition 20.* $\quad$ Let $G$ be a finite group under $\circ$, let $g$ be an element of $G$, and let $k$ be an integer. Then we define exponetiation of $g$ to the power $k$ as

$$\begin{align*}
g^k = \underbrace{g \circ g \circ \cdots \circ g}_{\textrm{$n$ copies of $g$}}
\end{align*}$$

Negative exponents indicate positive powers of a group element's inverse.

> *Definition 21.* $\quad$ If $G$ is a group with finitely many elements, then $G$ is called a **finite group**.

> *Definition 22.* $\quad$ Let $G$ be a group with identity element $e$ and let $g \in G$. The smallest positive integer $k$ such that $g^k = e$ is called the **order** of $g$, and we write $|g| = k$. 

When we speak of the order of the group itself (as opposed to one of its elements), what we mean is the order in the sense of Definition 13, i.e. the order of a *set*. The notation is the same: the order of a group $G$ is written $|G|$.

> *Definition 23.* $\quad$ If $G$ and $H$ are groups equipped with the same binary operation and $H$ is a subset of $G$ (i.e. every element of $H$ is also an element of $G$), then we say that $H$ is a **subgroup** of $G$.

*Example* $\quad$ Let $H = \{1, 2, 4\}$ be equipped with multiplication modulo $7$. Then $H$ is a subgroup of $\mathbb{Z}_7^*$. To see this, note that $H$ has the multiplicative identity $1$, is closed under multiplication modulo $7$ (test this by performing every possible multiplication of $1, 2$, and $4$; there are only 6 pairings to check), every element of $H$ has an inverse, and that $H$ is a subset of $\{1, 2, 3, 4, 5, 6\}$. $H$ inherits associativity of multiplication from $G$.

$$$$

> *Definition 24.* $\quad$ Suppose that $g$ is an element of a group $G$. We define a group $\langle g \rangle$, called the **cyclic subgroup of $G$ generated by $g$**, as the group generated by repeatedly applying the group operation of $G$ to $g$. That is,
$$\begin{align*}
\\
\langle g \rangle = \{g^k \in G : k \in \mathbb{N}\}\\\\
\end{align*}$$
If $\langle g \rangle = G$, then $g$ is said to be a **generator** for $G$. Any group containing a generator is said to be a **cyclic group**.

*Example* $\quad$ We claim that $3$ is a generator of $\mathbb{Z}_7^*$. We'll check the first six powers of $3$ modulo $7$ to verify that they yield every element of $\mathbb{Z}_7^*$.

$$\begin{align*}
3^1 &\equiv 3 \mod 7\\
3^2 &\equiv 2 \mod 7 \\
3^3 &\equiv 6 \mod 7 \\
3^4 &\equiv 4 \mod 7 \\
3^5 &\equiv 5 \mod 7 \\
3^6 &\equiv 1 \mod 7
\end{align*}$$

So indeed $\langle 3 \rangle = \mathbb{Z}_7^*$.

$$$$

Now follows the most important result in the context of the Diffie-Hellman key exchange:

> *Theorem 5.* $\quad$ If $p$ is prime, then $\mathbb{Z}_p^*$ is a cyclic group.

We omit proofs of any of the preceding theorems or claims as they are beyond the scope of this project. In any case, we are now in a position to describe the problem central to the security of the Diffie-Hellman key exchange.

$$$$

*Question: How do we find generators?*

The answer is unsatisfying. There is no known way to find generators analytically. We have to check each element of a group manually to determine which are its generators, if there are any.

---
$$$$
### The Discrete Log Problem

Let $p$ be prime, let $g$ be a generator of $\mathbb{Z}_p^*$, and let $h$ be any element of $\mathbb{Z}_p^*$. Then the following congruence has a solution $x$ in $G$.

$$\begin{align*}
g^x \equiv h \mod p
\end{align*}$$

The unique solution $x$ satisfying $1 \leq x < p$ of this congruence is called the **discrete logarithm of $h$ to the base $g$ modulo $p$**, and we write $x = \log_g{h}$. Note the similarities between the standard logarithm, but do not confuse them. Although the discrete log shares many properties with the normal, real-valued log function, they otherwise behave very differently. 

Let us now state the discrete log problem.

> *The Discrete Log Problem.* $\quad$ Suppose that $p$ is prime, $g$ is a generator for $\mathbb{Z}_p^*$, and $h$ is an element of $\mathbb{Z}_p^*$. Compute $\log_g{h}$. That is, solve
$$\begin{align*}\\
g^x \equiv h \mod p\\
\end{align*}$$

It turns out that this problem is computationally intractable if $p$ is very large (several hundred digits are required these days). Moreover, an attacker would need to solve this problem to obtain the key which would have been shared between two parties. Now we can describe the key exchange itself.

---
$$$$
## The Diffie-Hellman Protocol

Suppose that Alice and Bob have agreed upon a symmetric key cryptosystem to use to send messages to each other, but they have no means of securely transmitting a key to each other securely. Perhaps they are in different locations, and the only forms of communication they have with each other are monitored by Eve.

To get around this, Alice and Bob will execute the following steps:

1.   They use these unsecure lines of communication to agree on a large prime number $p$ and a generator for $\mathbb{Z}_p^*$. 
2.   Alice chooses a secret integer $a$. She computes $g$ to the power $a$ and transmits to Bob the number $A \equiv g^a \mod p$.
3.   At the same time, Bob chooses his own secret integer $b$. He computes $g$ to the power $b$ and then transmits the number $B \equiv g^b \mod p$ to Alice.
4.   After receiving $A$ from Alice, Bob computes $A^b \mod p$. Bob now destroys (or forgets) the integer $b$.
5.   Likewise, after Alice receives $B$ from Bob, she computes $B^a \mod p$. Alice is now free to destroy (or forget) $a$.
6.   It turns out that $A^b \mod p$ is *exactly the same* as $B^a \mod p$! This is their shared secret key $k$.

Let's have a closer look so we can see why this works. In step 3, Bob computes $B \equiv g^b \mod p$ and transmits it to Alice. Then in step 5, Alice takes her own secret integer $a$ with the integer Bob sent her and computes

$$\begin{align*}
B^a \equiv (g^b)^a \mod p \tag{I}
\end{align*}$$

Recall that in normal arithmetic, $(x^y)^z = x^{yz}$. It can be shown that the same rule applies in modular arithmetic. So congruence $\textrm{(I)}$ becomes

$$\begin{align*}
B^a \equiv (g^b)^a \equiv g^{ba} \equiv g^{ab} \mod p
\end{align*}$$

Now, in step 2, Alice computed $A \equiv g^a \mod p$ and transmitted it to Bob. Then in step 4, in a manner identical to Alice but with different values, Bob computed

$$\begin{align*}
A^b \equiv (g^a)^b \equiv g^{ab} \mod p
\end{align*}$$

These are the same! So the secret key is $k = g^{ab}$, and provided Alice and Bob agreed on a way to use it to form a particular key for their previously agreed upon cryptosystem, they are now in a position to send each other encrypted messages.

We present two Python functions in the code cell below:

1.   Given an integer $n$, **checkPrime()** will check for primality of $n$ and return the appropriate TRUE or FALSE value.
2.   Given an integer $L$ (the variable name is lowerBound), **DH_publicKey()** will return the smallest prime number $p$ satisfying $p \geq L$ and a generator $g$ for $\mathbb{Z}_p^*$.



In [62]:
# Import the necessary packages.
from math import sqrt
from math import floor
import random
import time

# Define a function called checkPrime() which accepts an integer n and returns
# True if n is prime and False otherwise.
def checkPrime(n):
  if n < 2:
    return False
  if (n == 2) or (n == 3):
    return True
  if ((n % 2) == 0) or ((n % 3) == 0):
    return False
  upperBound = floor(sqrt(n))
  k = 1
  while (6*k - 1) <= upperBound:
    if ((n % (6*k - 1)) == 0) or ((n % (6*k + 1)) == 0):
      return False
    k = k + 1
  return True

# Define a function called DH_publicKey which accepts an integer lowerBound and
# returns a prime number at least as large as lowerBound and a random generator
# for the multiplicative group of integers modulo the prime.
def DH_publicKey(lowerBound):
  start = time.time()
  prime = lowerBound
  while checkPrime(prime) == False:
    prime = prime + 1
  phi = prime - 1
  generator = -1
  while generator == -1:
    base = random.randint(2, prime)
    product = base
    for exponent in range(2, phi):
      product = (base*product) % prime
      if product == 1:
        break
      if exponent == phi - 1:
        generator = base
        end = time.time()
        print(end - start)
        return prime, generator


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Choose a lower bound for the prime.
lowerBound = 7918

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE

# Call DH_publicKey() on lowerBound and print the result to the command window.
print(DH_publicKey(lowerBound))



0.00471806526184082
(7919, 6947)


---
$$$$
## Example: The Diffie-Hellman Protocol

Let's do a concrete example to verify that this works. First, Alice and Bob agree on the prime $p = 7919$. They have to find a generator for $\mathbb{Z}_{7919}^*$, which is computationally easy. They settle on $g = 105$.

Next, Alice and Bob choose their secret exponents, say $a = 5430$ and $b = 3992$. Alice and Bob compute:

$$\begin{align*}
A &\equiv g^a \equiv 105^{5430} \equiv 3011 \mod 7919 \\\\
B &\equiv g^b \equiv 105^{3992} \equiv 1355 \mod 7919
\end{align*}$$

Alice transmits $A$ to Bob and Bob sends $B$ to Alice. Then Alice and Bob compute:

$$\begin{align*}
B^a &\equiv 1355^{5430} \equiv 704 \mod 7919 \\\\
A^b &\equiv 3011^{3992} \equiv 704 \mod 7919
\end{align*}$$

So their shared secret integer is $k = 704$. You'll notice that Alice and Bob lack precise control over precisely *what* their secret integer will be. Our lack of understanding of the behavior of multiplicative groups is the property that admits secrecy.

It may not be clear why breaking this exchange is equivalent to solving a DLP, but we will return to this later. 

Below are two Python functions called **DH_transmit()** and **DH_compute** which respectively compute the integer A (or B) and compute the key after the exchange is performed. 

In [86]:
def DH_transmit(secretInteger, prime, generator):
  numberToTransmit = pow(generator, secretInteger, prime)
  return numberToTransmit

def DH_compute(secretInteger, prime, receivedInteger):
  key = pow(receivedInteger, secretInteger, prime)
  return key

# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

secretInteger = 5430
prime = 7919
generator = 105

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE

print(DH_transmit(secretInteger, prime, generator))

# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

receivedInteger = 1355

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE

print(DH_compute(secretInteger, prime, receivedInteger))

3011
704


---
$$$$
## Cryptanalysis of the Diffie-Hellman Key Exchange

We now turn our attention to the problem of cryptanalyzing the Diffie-Hellman key exchange. Re-examining the six steps in the protocol reveals that Eve will have access to four pieces of information:

1.   The prime modulus $p$.
2.   The generator $g$ of $\mathbb{Z}_p^*$.
3.   The integer $A \equiv g^a \mod p$ which Alice sent to Bob.
4.   The integer $B \equiv g^b \mod p$ which Bob sent to Alice.

Eve's task is to deduce $k \equiv g^{ab} \mod p$ from this. Well, since Eve has knowledge of $g$, $A$, and $B$, she must determine either $a$ or $b$. She can do this by solving one of the congruences

$$\begin{align*}
g^a &\equiv A \mod p \\
g^b &\equiv B \mod p\\\\
\end{align*}$$

Recall that the DLP asks for the solution $x$ of the congruence $g^x \equiv h \mod p$. This is exactly the problem that Eve must solve! So if we find a general solution to the DLP, we'll have discovered a method to cryptanalyze the Diffie-Hellman protocol.

The simplest publicly available method for doing this is called Shanks' Baby-step Giant-step algorithm. There are other, more efficient methods, but they are somewhat more complicated and so not suitable for our purposes.

---
$$$$
### Shanks' Baby-step Giant-step Algorithm

We'll first briefly introduce some new notation.

> Definition. $\quad$ The **ceiling function** is a function with signature $\lceil \cdot \rceil : \mathbb{R} \rightarrow \mathbb{Z}$ and rule
$$\begin{align*}\\
x \mapsto \min\left\{ n \in \mathbb{Z} : n \geq x \right\}\\\\
\end{align*}$$
In plain English, the ceiling function accepts a real number (i.e. decimal number) $x$ and returns the smallest integer which is at least as large as $x$. 

*Examples:* $\quad \lceil \pi \rceil = 4$, $\quad \lceil -34.2 \rceil = -34$, $\quad \lceil 5 \rceil = 5$

Shanks' algorithm for solving $g^x \equiv h \mod p$ proceeds as follows.

1.    Compute $N = \lceil \sqrt{p} \rceil$.
2.    Compute $g^j$ for every $0 \leq j < N$ and store them in a list $J = (g^0, g^1, g^2, \ldots, g^{N-1})$.
3.    Compute $h g^{-iN}$ for every $0 \leq i < N-1$ and store them in a list $I = (h, h g^{-N}, h g^{-2N}, \ldots, h g^{-(N-1)N})$.
4.    Two entries from these lists, one from each, will contain the same value. Assume $g^j$ and $hg^{-iN}$ are these values.
5.    We solve the following congruence and obtain the solution $x = \log_g(h)$ of the DLP:

$$\begin{align*}
&& g^j &\equiv hg^{-iN} \mod p \\\\
&\Longrightarrow & g^j\cdot g^{iN} &\equiv hg^{-iN}g^{iN} \mod p\\\\
&\Longrightarrow & g^{j + iN} &\equiv h \mod p \\\\
&\Longrightarrow & \log_g(h) &\equiv j + iN \mod p
\end{align*}$$

Since $j, i$, and $N$ are all known quantities, we're done. The amount of time this algorithm requires to find a solution is proportional to $\sqrt{p}$. This is why using a large prime leads to increased security.

Keep in mind that the Diffie-Hellman key exchange is NOT a cryptosystem - it is a means to enable secure exchange of a key for use in a symmetric key cryptosystem. The security of messages exchanged with a cryptosystem using a key generated by the Diffie-Hellman protocol is determined by two factors:

1.   The size of the prime number used to generate the key.
2.   The security of the cryptosystem that key was used for.

If Alice and Bob chose to use their key in, say, a Caesar cipher, it doesn't matter how secure their key exchange was; as you saw earlier this week, the Caesar cipher is easy to cryptanalyze.

Below we present a function **DH_attack()** which executes Shanks' Baby-step Giant-step attack on the discrete log problem. Check that the attack produces Alice and Bob's secret integers from before.

In [96]:
# Import sqrt and ceil (ceiling function).
from math import sqrt
from math import ceil

# The function DH_attack() accepts a prime number, a generator for the
# multiplicative group modulo the prime, and a number h and outputs the discrete 
# log of h to the base "generator."
def DH_attack(prime, generator, h):

  # STEP 1: Compute N = ceil(sqrt(prime)).
  N = ceil(sqrt(prime))

  # STEP 2: Compute the list J.
  J = []
  product = 1
  for j in range(N):
    J.append(product)
    product = (product*generator) % prime
  
  # STEP 3: Compute the list I.
  I = []
  product = h
  generatorInverse = inverseMod(generator, prime)
  generatorN = pow(generatorInverse, N, prime)
  for i in range(N):
    I.append(product)
    product = (product*generatorN) % prime

  # STEP 4: Find the collision of I and J and return the indices of the 
  # list entries that match (i.e. the values of i and j that produce matches).
  for j in range(N):
    if J[j] in I:
      jIndex = j
      iIndex = I.index(J[j])
      break
  
  # STEP 5: Compute x = j + iN modulo p
  return (jIndex + iIndex*N) % prime


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Define variables to store the prime number, the generator, and the desired h.
# Here we use h = B, the value Bob transmitted to Alice.
prime = 7919
generator = 105
h = 1355

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Print the discrete log of h to the base g to the command window.
print(DH_attack(prime, generator, h))


# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Now check that the attack worked for h = A, the value Alice sent to Bob.
h = 3011

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


print(DH_attack(prime, generator, h))

3992
5430


---
$$$$
## Using the Key Generated by the Diffie-Hellman Protocol

The Diffie-Hellman protocol produces a shared secret element of a multiplicative group of prime order. It is not immediately obvious how this could be applied to one of the cryptosystems we've already seen.

Well, first of all, it would be nice if the security was completely determined by the size of the prime and was not dependent on the security of the cryptosystem the generated key is used with. For this reason, we will employ the Vernam cipher - recall that the Vernam cipher is perfectly secret provided several conditions are met. Using the Vernam cipher will ensure that the secrecy will depend only on the size of the prime $p$. 

Our technique will be as follows. Assume that there are $N$ letters to be enciphered.

1. Alice and Bob publicly agree on a large prime $p$ and a generator $g$ of $\mathbb{Z}_p^*$. 

2. For each letter to be enciphered, Alice and Bob select a random integer between $0$ and $p-1$ to serve as their secret numbers. They form two lists:

$$\begin{align*}
a = (a_1, a_2, a_3, \ldots, a_N) \qquad {\textrm{and}} \qquad b = (b_1, b_2, b_3, \ldots, b_N)
\end{align*}$$

3. For each element of these lists, Alice and Bob execute the Diffie-Hellman protocol as prescribed in a previous section. They will have a list of integers in $\mathbb{Z}_p^*$ of the following form:

$$\begin{align*}
L = (g^{a_1b_1}, g^{a_2b_2}, g^{a_3b_3}, \ldots, g^{a_N b_N}) \mod p
\end{align*}$$

4. Finally, Alice and Bob take the least residue classes of these integers modulo $p$ and reduce them modulo $26$; say $k_i \equiv \left(g^{a_i b_i} \mod p\right) \mod 26$. They then form a list $k$ of length $N$:

$$\begin{align*}
k = (k_1, k_2, k_3, \ldots, k_N)
\end{align*}$$

5. The list $k$ is their shared one-time pad.

*Note: If for some reason Alice and Bob wanted to use a relatively small prime $p$, they would probably want to ensure that $26$ divides $p-1$. This condition would ensure that the entries in the key would have a uniform distribution - or, at worst, they would only fail to have a uniform distribution because they didn't select their $a_i$ and $b_i$ randomly.*

We present three functions below to this end:

1. **DH_secretIntegers()** accepts plaintext of length $N$ and a prime number and returns a list of random integers with length $N$.
2. **DH_multi_transmit()** accepts a list of secret integers, a prime $p$, and a generator for $\mathbb{Z}_p^*$ and returns a list of numbers ready for public transmission.
3. **DH_vernam_key()** accepts a prime $p$, a list of secret integers, and a list of numbers received via public transmission from a partner and returns a Vernam cipher key.

In [101]:
# Import the Python package random to assist in selecting secret integers.
import random

# The function DH_secretIntegers() randomly selects elements of the multiplicative
# group modulo the prime number you input and return a list of secret integers.
def DH_secretIntegers(plaintext, prime):
  N = len(plaintext)
  secretIntegers = []
  for index in range(N):
    secretIntegers.append(random.randint(0,prime - 1))
  return secretIntegers


# The function DH_multi_transmit() accepts a list of secret integers, a prime
# number, and a generator of the multiplicative group modulo that prime and 
# returns a the list of integers modulo the prime prescribed by the Diffie-
# Hellman protocol.
def DH_multi_transmit(secretIntegers, prime, generator):
  numbersToTransmit = []
  for index in range(len(secretIntegers)):
    numbersToTransmit.append(DH_transmit(secretIntegers[index], prime, generator))
  return numbersToTransmit


# The fucntion DH_vernam_key() accepts a prime number, a list of secret integers,
# and a list of integers you received from a partner and returns a one-time pad.
def DH_vernam_key(prime, secretIntegers, receivedIntegers):
  DHkey = []
  vernamKey = []
  if len(secretIntegers) != len(receivedIntegers):
    print('The length of the list of your secret integers is not equal to the length of the list of the integers you received.')
    return
  for index in range(len(secretIntegers)):
    DHkey.append(DH_compute(secretIntegers[index], prime, receivedIntegers[index]))
  for index in range(len(secretIntegers)):
    vernamKey.append(DHkey[index] % 26)
  return vernamKey



# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# STEP 0: Alice decides on a message to send to Bob.
plaintext = 'It\'s not unusual to be loved by anyone.'

# STEP 1: Alice and Bob publicly agree on a prime and generator.
prime = 7919
generator = 105
print(DH_secretIntegers(plaintext, prime))


# STEP 2: Alice and Bob randomly select a list of secret integers of the same
# length and at least as long as the message.

# This is a list of secret integers your project leader generated for Alice 
# using the function DH_secretIntegers.
secretIntegersA = [4792, 2609, 5643, 5945, 7499, 1763, 3326, 275, 7340, 3006, 5488, 2518, 4490, 3910, 1179, 1955, 3814, 4360, 6452, 5841, 5799, 193, 293, 4764, 6545, 573, 3563, 6610, 5034, 7251, 2787, 6174, 4525, 3402, 2598, 3238, 2807, 2369, 1489]

# This is a list of secret integers your project leader generated for Bob
# using the function above.
secretIntegersB = [4260, 4349, 3842, 7842, 938, 5966, 66, 4782, 6579, 6037, 668, 6987, 5632, 492, 6406, 7127, 6080, 2409, 7139, 886, 6853, 434, 3063, 1937, 4774, 6158, 4209, 5652, 5023, 5418, 4717, 6269, 1610, 735, 2694, 1243, 4811, 1723, 1335]


# STEP 3/4: Alice and Bob compute the list of numbers to be transmitted and do so.
# Recall that this is done via exponentiating the generator with the secret 
# integers, one at a time, modulo the prime. 

# Print the list of integers Alice will transmit to Bob.
print(DH_multi_transmit(secretIntegersA, prime, generator))

# Print the list of integers Bob will transmit to Alice.
print(DH_multi_transmit(secretIntegersB, prime, generator))


# This is a list of the integers Alice received from Bob generated by your
# project leader.
receivedIntegersA = [76, 1431, 5312, 3786, 4008, 1679, 6736, 5880, 6560, 4619, 1089, 4578, 7010, 4225, 570, 4404, 3813, 468, 6838, 3244, 6638, 5910, 6922, 680, 921, 3216, 4325, 2645, 7315, 2465, 1493, 7, 1643, 2223, 2021, 3274, 4674, 4080, 3828]

# This is a list of the integers Bob received from Alice generated by your
# project leader.
receivedIntegersB = [3775, 2871, 3914, 6735, 866, 2746, 2723, 6130, 5588, 2339, 6782, 460, 5780, 588, 461, 2727, 4498, 4972, 4213, 252, 6751, 6342, 7697, 5221, 7558, 4176, 6121, 6179, 5101, 5803, 1511, 1663, 3722, 4366, 6275, 6635, 3592, 580, 1003]

# STEP 3/4: Alice and Bob exponentiate the list of integers they received from
# each other modulo the prime. They then reduce that list modulo 26.

# This is the Vernam cipher key Alice would have computed assuming the lists
# are as given above.
print(DH_vernam_key(prime, secretIntegersA, receivedIntegersA))

# This is the Vernam cipher key Bob would have computed assuming the lists
# are as given above.
print(DH_vernam_key(prime, secretIntegersB, receivedIntegersB))

keyA = DH_vernam_key(prime, secretIntegersA, receivedIntegersA)
keyB = DH_vernam_key(prime, secretIntegersB, receivedIntegersB)



# This is the plaintext given above encrypted using the Vernam cipher and the
# one-time pad Alice generated.
print(vernam_encrypt(plaintext, keyA))

# This is the plaintext given above encrypted using the Vernam cipher and the
# one-time pad Bob generated.
print(vernam_encrypt(plaintext, keyB))


[5994, 16, 5975, 5867, 5086, 4130, 3221, 1060, 4393, 2882, 7581, 3713, 6936, 1499, 2229, 6029, 5782, 5093, 5599, 6967, 4159, 675, 2155, 2234, 5636, 7466, 5675, 1921, 1484, 4368, 4846, 5856, 1299, 2242, 5181, 4710, 7709, 2440, 5873]
[3775, 2871, 3914, 6735, 866, 2746, 2723, 6130, 5588, 2339, 6782, 460, 5780, 588, 461, 2727, 4498, 4972, 4213, 252, 6751, 6342, 7697, 5221, 7558, 4176, 6121, 6179, 5101, 5803, 1511, 1663, 3722, 4366, 6275, 6635, 3592, 580, 1003]
[76, 1431, 5312, 3786, 4008, 1679, 6736, 5880, 6560, 4619, 1089, 4578, 7010, 4225, 570, 4404, 3813, 468, 6838, 3244, 6638, 5910, 6922, 680, 921, 3216, 4325, 2645, 7315, 2465, 1493, 7, 1643, 2223, 2021, 3274, 4674, 4080, 3828]
[12, 2, 19, 24, 24, 0, 22, 18, 11, 16, 16, 5, 4, 5, 22, 15, 18, 11, 6, 19, 2, 5, 23, 18, 9, 25, 5, 18, 16, 10, 13, 9, 19, 15, 22, 22, 23, 24, 4]
[12, 2, 19, 24, 24, 0, 22, 18, 11, 16, 16, 5, 4, 5, 22, 15, 18, 11, 6, 19, 2, 5, 23, 18, 9, 25, 5, 18, 16, 10, 13, 9, 19, 15, 22, 22, 23, 24, 4]
UVLLM TQFFI KFPYK QWWUO

---
$$$$
## Exercise: Cryptanalysis of the Vernam Cipher with Diffie-Hellman Key

Below is a Python function called **DH_vernam_attack()** which accepts a string of ciphertext encrypted by the Vernam cipher using a key generated via the Diffie-Hellman protocol with a known prime $p$, generator $g$, and two lists of once-exponentiated integers modulo $p$ sent between two parties; it returns the decrypted plaintext. begin\align{ag*}\\
{\textrm{DH_vernam_attack(ciphertext, prime, generator,A, B)}} \qquad \longrightarrow \qquad {\textrm{plaintext}end\alignnd{align*}$$

The prime $p$ and generator $g$ are:

$$\begin{align*}
p = 130330637 \qquad {\textrm{and}} \qquad g = 90456641
\end{align*}$$

Use this function to cryptanalyze the following ciphertext given the prime, generator, and list of integers modulo $p$.

> [Integers mod $p$ received by Alice from Bob.](https://drive.google.com/file/d/11yKDMJQSa84-i8Cgk_oxSODfi0w5e1JX/view?usp=sharing)

> [Integers mod $p$ received by Bob from Alice.](https://drive.google.com/file/d/1Rs-HmAsgux1nbQ0K0kq8AQDi7i52VPLQ/view?usp=sharing)

> [Ciphertext](https://drive.google.com/file/d/1D0cDIAMM9iMViwbUFk2ktmHXflQUJ1ja/view?usp=sharing)

*Question: Suppose you only saw the ciphertext, prime, and generator. Would you have any way of determining the once-exponentiated lists of integers mod $p$ transmitted between Alice and Bob? If so, what cryptanalytic techniques that you've learned so far might help you do this?*

In [113]:
# The arguments of the function DH_vernam_attack() are as follows:
#
# CIPHERTEXT    - The Vernam ciphertext to be broken.
# PRIME         - The prime number p from the DHP public key.
# GENERATOR     - The generator of the multiplicative group modulo p from the
#                 DHP public key.
# A             - The list of integers transmitted to Alice from Bob.
# B             - The list of integers transmitted to Bob from Alice.
#
# The function DH_vernam_attack() uses Shanks' Baby-step Giant-step algorithm
# to attack the discrete log problem associated to each ciphertext letter and
# attempts to decrypt the ciphertext with vernam_decrypt().

def DH_vernam_attack(ciphertext, prime, generator, A, B):
  secretIntegersA = []
  for h in A:
    secretIntegersA.append(DH_attack(prime, generator, h))
  vernamKey = DH_vernam_key(prime, secretIntegersA, B)
  return vernam_decrypt(ciphertext, vernamKey)
  

# EDIT BELOW THIS LINE
#-------------------------------------------------------------------------------

# Initiate variables to store the input arguments.
ciphertext = 'BUEZA VYXUQ YRHDJ AOCAJ SSYHB YNKWZ TWBPZ'
prime = 130330637
generator = 90456641
A = [71939035, 79026566, 51149780, 60425569, 15814472, 52029089, 72737465, 84090345, 1565058, 129877542, 4524873, 128137683, 54334672, 16387630, 15481489, 33945065, 102837120, 114507855, 116289811, 81505534, 48876153, 122366931, 16721960, 15846695, 43466710, 65822520, 113658021, 36221683, 84708340, 258020, 94660203, 120716179, 100657258, 118180836, 22843419, 677767]
B = [38439570, 12717067, 122679091, 104660588, 77747585, 15033249, 37226622, 38462314, 94287130, 35487339, 57301575, 76213919, 110282195, 98162207, 49039168, 56054031, 114631064, 38836271, 115195316, 124265321, 99094729, 5055055, 12392823, 112753878, 53248722, 72376270, 90833767, 34573998, 10192100, 31390428, 88551002, 13337688, 21622235, 1966488, 44471495, 122936739]

#-------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE


# Execute the attack and print the result to the command window.
print(DH_vernam_attack(ciphertext, prime, generator, A, B))

wehol dthes etrut hstob eself evide nt
